In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
import os
import json

from llama_index.embeddings import LangchainEmbedding
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    KnowledgeGraphIndex,
    ServiceContext,
)

from llama_index import set_global_service_context

from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore

import logging
import sys

from IPython.display import Markdown, display
from dotenv import load_dotenv
load_dotenv()


logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))



llm = ChatOpenAI(temperature=0.1, max_tokens = 600, request_timeout=600, max_retries=10,
                        openai_api_key=os.getenv('MISTRAL_7B_API_KEY'),
                        openai_api_base=os.getenv('MISTRAL_7B_API_BASE'),
                        )
 
# embedding

model_name = "sentence-transformers/all-distilroberta-v1"
model_kwargs = {'device': 'cpu'} # mps:
encode_kwargs = {'normalize_embeddings': False}


myEmbedding = HuggingFaceEmbeddings(
                model_name=model_name,
                model_kwargs=model_kwargs,
                encode_kwargs=encode_kwargs)        


embed_model = LangchainEmbedding(myEmbedding)

service_context = ServiceContext.from_defaults(
   llm=llm,
    embed_model=embed_model,)



set_global_service_context(service_context)

In [ ]:
#replace the XXX with your own nenula keys
os.environ["NEBULA_USER"] = "XXX"
os.environ["NEBULA_PASSWORD"] = "XXX"
os.environ[
    "XXX"
] = "XXX"  

space_name = "XXX"
edge_types, rel_prop_names = ["relationship"], [
    "relationship"
]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg



In [ ]:

doc = SimpleDirectoryReader("File_Dir", recursive=True).load_data()

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)


kg_index = KnowledgeGraphIndex.from_documents(
    #documents,
    doc,
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    service_context=service_context,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    include_embeddings=True,
    verbose=True,
)


vector_index = VectorStoreIndex.from_documents(doc)


In [ ]:
# import QueryBundle
from llama_index import QueryBundle

# import NodeWithScore
from llama_index.schema import NodeWithScore

# Retrievers
from llama_index.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
    KGTableRetriever,
)

from typing import List


class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both Vector search and Knowledge Graph search"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        kg_retriever: KGTableRetriever,
        mode: str = "OR",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._kg_retriever = kg_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        kg_nodes = self._kg_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        kg_ids = {n.node.node_id for n in kg_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in kg_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(kg_ids)
        else:
            retrieve_ids = vector_ids.union(kg_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes
    
    

In [ ]:
from llama_index import get_response_synthesizer
from llama_index.query_engine import RetrieverQueryEngine

# create custom retriever
vector_retriever = VectorIndexRetriever(index=vector_index)
kg_retriever = KGTableRetriever(
    index=kg_index, retriever_mode="keyword", include_text=False
)


# create response synthesizer
response_synthesizer = get_response_synthesizer(
    service_context=service_context,
    response_mode="tree_summarize",
)


custom_retriever = CustomRetriever(vector_retriever, kg_retriever)

custom_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer,
)

In [ ]:
from llama_index.evaluation import (
    CorrectnessEvaluator,
    FaithfulnessEvaluator,
    RelevancyEvaluator,
    SemanticSimilarityEvaluator,
)
import tqdm

import pandas as pd




def ResponseEvaluationMetrics(Queries_method):
    relevance_scores = {
        #'Query': [],
        'Vector Index': [],
        'Knowledge Graph': [],
        'Vector + Knowledge': []
    }
    
    correctness_scores = {
        #'Query': [],
        'Vector Index': [],
        'Knowledge Graph': [],
        'Vector + Knowledge': []
    }
    
    faithfulness_scores = {
        #'Query': [],
        'Vector Index': [],
        'Knowledge Graph': [],
        'Vector + Knowledge': []
    }

    response_length = {
        #'Query': [],
        'Vector Index': [],
        'Knowledge Graph': [],
        'Vector + Knowledge': []
    }

    for query in Queries_method:
        responseB = custom_query_engine.query(query)
        responseK = kg_keyword_query_engine.query(query)
        responseV = vector_query_engine.query(query)

        # Extract content for references
        ref_content_V = responseV.source_nodes[0].node.get_content()[:1000] if responseV.source_nodes else ""
        ref_content_K = responseK.source_nodes[0].node.get_content()[:1000] if responseK.source_nodes else ""
        ref_content_B = responseB.source_nodes[0].node.get_content()[:1000] if responseB.source_nodes else ""

        # Calculate response lengths
        response_length_V = len(str(responseV))
        response_length_K = len(str(responseK))
        response_length_B = len(str(responseB))

        # Evaluation based on Relevance
        evaluatorR = RelevancyEvaluator(service_context=service_context)
        evalR_V = evaluatorR.evaluate_response(query=query, response=responseV)
        evalR_K = evaluatorR.evaluate_response(query=query, response=responseK)
        evalR_B = evaluatorR.evaluate_response(query=query, response=responseB)

        #relevance_scores['Query'].append(query)
        relevance_scores['Vector Index'].append(evalR_V.score)
        relevance_scores['Knowledge Graph'].append(evalR_K.score)
        relevance_scores['Vector + Knowledge'].append(evalR_B.score)

        #response_length['Query'].append(query)
        response_length['Vector Index'].append(response_length_V)
        response_length['Knowledge Graph'].append(response_length_K)
        response_length['Vector + Knowledge'].append(response_length_B)

        # Evaluation based on Correctness
        evaluatorC = CorrectnessEvaluator(service_context=service_context)
        evalC_V = evaluatorC.evaluate(
            query=query,
            response=str(responseV),
            reference=ref_content_V,
        )
        evalC_K = evaluatorC.evaluate(
            query=query,
            response=str(responseK),
            reference=ref_content_K,
        )
        evalC_B = evaluatorC.evaluate(
            query=query,
            response=str(responseB),
            reference=ref_content_B,
        )

        #correctness_scores['Query'].append(query)
        correctness_scores['Vector Index'].append(evalC_V.score)
        correctness_scores['Knowledge Graph'].append(evalC_K.score)
        correctness_scores['Vector + Knowledge'].append(evalC_B.score)

        # Evaluation based on Faithfulness
        evaluatorF = FaithfulnessEvaluator(service_context=service_context)
        evalF_V = evaluatorF.evaluate_response(
            query=query,
            response=responseV,
            reference=ref_content_V,
        )
        evalF_K = evaluatorF.evaluate_response(
            query=query,
            response=responseK,
            reference=ref_content_K,
        )
        evalF_B = evaluatorF.evaluate_response(
            query=query,
            response=responseB,
            reference=ref_content_B,
        )

        #faithfulness_scores['Query'].append(query)
        faithfulness_scores['Vector Index'].append(evalF_V.score)
        faithfulness_scores['Knowledge Graph'].append(evalF_K.score)
        faithfulness_scores['Vector + Knowledge'].append(evalF_B.score)

    # Create DataFrames
    FinalEvalR = pd.DataFrame(relevance_scores)
    FinalEvalC = pd.DataFrame(correctness_scores)
    FinalEvalF = pd.DataFrame(faithfulness_scores)
    FinalResponseLength = pd.DataFrame(response_length)
    
    return FinalEvalR, FinalEvalC, FinalEvalF, FinalResponseLength




## Normal Queries

In [ ]:
eas_queries = [
    "What year was Abraham Lincoln born?",
    "Who was Abraham Lincoln's vice president during his first term?",
    "Which speech did Abraham Lincoln deliver that starts with 'Four score and seven years ago'?",
    "What are the basic anatomical features of an ant?",
    "How many species of ants are there globally?",
    "What is the primary diet of ants?",
    "What is the official script used to write Arabic?",
    "Name three countries where Arabic is the official language?",
    "What is the basic word order in an Arabic sentence?",
    "What is the primary role of bees in the ecosystem?",
    "Name the most well-known species of bees.",
    "How do bees communicate with each other?",
    "What order do beetles belong to?",
    "How many species of beetles are there?",
    "What is the primary diet of a ladybird beetle?",
    "What is the capital city of China?",
    "Name one famous historical site in Beijing.",
    "What is the population of Beijing?",
    "What stage comes after the caterpillar in the butterfly lifecycle?",
    "Name one common species of butterfly.",
    "What do butterflies typically eat?"
    "What period did Calvin Coolidge became a president?",
    "What was Calvin Coolidge profession?",
    "Who was Calvin Coolidge?",
    "What is the capital city of Canada?",
    "Name three provinces in Canada.",
    "What are the official languages of Canada?",
    "What is a cello?",
    "How many strings does a cello have?",
    "Name one famous cellist.",
    "What type of scripts can be used to write Chinese?",
    "How many tones are there in Mandarin Chinese?",
    "Name one Chinese dialect",
    "What is another name for a cougar?",
    "Where are cougars commonly found?",
    "What is the primary diet of a cougar?",
    "What is a cymbal?",
    "Name three types of cymbals.",
    "What materials are cymbals typically made from?",
    "What is the capital of Bangladesh?",
    "Name one river in Dhaka?",
    "What is Dhaka known of?",
    "What order do dragonflies belong to?",
    "How many pairs of wings do dragonflies have?",
    "What is the primary habitat of dragonflies?",
    "What family do ducks belong to?",
    "What is the primary diet of ducks?",
    "Name three types of ducks.",
    "What is the capital of Egypt?",
    "Name one major river in Egypt.",
    "What is Egypt's official language?",
    "What was El Greco's real name?",
    "Where was El Greco born?",
    "Name one famous painting by El Greco.",
    "How many species of elephants are there?",
    "What is the primary diet of elephants?",
    "Name one physical characteristic unique to elephants.",
    "What family of languages does English belong to?",
    "Name two countries where English is an official language.",
    "What historical event significantly influenced the development of Middle English?",
    "What is the capital of Finland?",
    "Name one official language of Finland.",
    "What is the population of Finland?",
    "What language family does Finnish belong to?",
    "Name one country other than Finland where Finnish is spoken.",
    "How many cases does the Finnish language have?",
    "What family of musical instruments does the flute belong to?",
    "How is sound produced in a flute?",
    "Name one type of flute.",
    "How many species of foxes are there?",
    "What is the primary habitat of the red fox?",
    "Name one characteristic physical feature of foxes.",
    "What language family does French belong to?",
    "Name two countries where French is an official language.",
    "What is the origin of the French language?",
    "When did Gerald Ford serve as President of the United States?",
    "Who was Gerald Ford's vice president?",
    "What event led to Gerald Ford becoming president?",
    "What language family does German belong to?",
    "Name two countries where German is an official language.",
    "What is the standard form of the German language called?",
    "What is the capital of Ghana?",
    "Name one major river in Ghana.",
    "What is Ghana's official language?",
    "What is the primary diet of giant pandas?",
    "Where are giant pandas naturally found?",
    "What is one physical characteristic unique to giant pandas?",
    "What is the scientific name of the giraffe?",
    "How tall can male giraffes grow?",
    "What is the diet of giraffes?",
    "What is the scientific name of the gray wolf?",
    "Where are gray wolves commonly found?",
    "What is the primary diet of gray wolves?",
    "When did Grover Cleveland serve as President of the United States?",
    "What is unique about Grover Cleveland's terms in office?",
    "Who was Grover Cleveland's vice president?",
    "What family of musical instruments does the guitar belong to?",
    "How many strings does a standard guitar have?",
    "Name one famous guitarist.",
    "When was Gustav Klimt born?",
    "What art movement was Gustav Klimt associated with?",
    "Name one famous painting by Gustav Klimt.",
    "What is the capital city of Indonesia?",
    "Name one major island in Indonesia.",
    "What is the official language of Indonesia?",
    "Where was Isaac Newton born?",
    "What is Isaac Newton  known for?",
    "Name one  work published by Isaac Newton.",
    "When was Jackson Pollock born?",
    "What art movement was Jackson Pollock associated with?",
    "Name one famous painting by Jackson Pollock.",
    "When did James Monroe serve as President of the United States?",
    "What is James Monroe best known for?",
    "Name one major event that occurred during Monroe's presidency.",
    "When was James Watt born?",
    "What is James Watt best known for?",
    "Name one major improvement James Watt made to the steam engine.",
    "What writing systems are used in the Japanese language?",
    "How many main dialects are there in Japanese?",
    "What is the standard form of Japanese called?",
    "When did John Adams serve as President of the United States?",
    "What is John Adams best known for?",
    "Who was John Adams' vice president?",
    "What family do kangaroos belong to?",
    "Name two species of kangaroos.",
    "What is the primary diet of kangaroos?",
    "What is the scientific name of the koala?",
    "Where are koalas naturally found?",
    "What is the primary diet of koalas?",
    "What writing systems are used in the Korean language?",
    "How many main dialects are there in Korean?",
    "What is the standard form of Korean called?",
    "When was Leonardo da Vinci born?",
    "What is Leonardo da Vinci best known for?",
    "Name one famous painting by Leonardo da Vinci.",
    "What is the scientific name of the leopard?",
    "Where are leopards commonly found?",
    "What is the diet of leopards?",
    "What is the capital of Liechtenstein?",
    "What is the official language of Liechtenstein?",
    "What is the population of Liechtenstein?",
    "What is the capital of Peru?",
    "Name one famous landmark in Lima.",
    "What is the population of Lima?",
    "What is the scientific name of the lion?",
    "Where are lions commonly found?",
    "What is the primary diet of lions?",
    "How many species of lizards are there?",
    "What is the primary diet of most lizards?",
    "Name one unique feature of lizards.",
    "What family do clawed lobsters belong to?",
    "Name two types of lobsters.",
    "What is the primary diet of lobsters?",
    "What is the capital city of the United Kingdom?",
    "Name one famous landmark in London.",
    "What river runs through London?",
    "What type of musical instrument is a lyre?",
    "Name one ancient civilization that used the lyre.",
    "What is the primary material used for lyre strings?",
    "What language family does Malay belong to?",
    "Name one country where Malay is an official language.",
    "What script is used to write Malay?",
    "When was Michelangelo born?",
    "What is Michelangelo best known for?",
    "Name one famous sculpture by Michelangelo.",
    "When did Millard Fillmore serve as President of the United States?",
    "Who was Millard Fillmore's vice president?",
    "What event led to Millard Fillmore becoming president?",
    "When was Norman Rockwell born?",
    "What is Norman Rockwell best known for?",
    "Name one famous painting by Norman Rockwell.",
    "How many arms does an octopus have?",
    "What is the primary diet of octopuses?",
    "Where are octopuses commonly found?",
    "What family do otters belong to?",
    "Name one type of otter.",
    "What is the primary diet of otters?",
    "When was Pablo Picasso born?",
    "What art movement is Pablo Picasso associated with?",
    "Name one famous painting by Pablo Picasso.",
    "How many species of penguins are there?",
    "What is the primary diet of penguins?",
    "Where are penguins commonly found?",
    "How many keys does a standard piano have?",
    "What are the main types of pianos?",
    "Who is known as the 'father of the modern piano'?",
    "When was Pierre-Auguste Renoir born?",
    "What art movement is Renoir associated with?",
    "Name one famous painting by Renoir.",
    "When was Piet Mondrian born?",
    "What art movement is Mondrian associated with?",
    "Name one famous painting by Mondrian.",
    "What is the scientific name of the polar bear?",
    "Where are polar bears commonly found?",
    "What is the primary diet of polar bears?",
    "What language family does Portuguese belong to?",
    "Name one country where Portuguese is an official language.",
    "What script is used to write Portuguese?",
    "What is the capital of Qatar?",
    "Name one major industry in Qatar.",
    "What is the official language of Qatar?",
    "What is the capital of Singapore?",
    "Name one major industry in Singapore.",
    "What is the official language of Singapore?",
    "What language family does Spanish belong to?",
    "Name two countries where Spanish is an official language.",
    "What script is used to write Spanish?",
    "How fast can one learn swahili?",
    "Name one country where Swahili is an official language.",
    "What script is used to write Swahili?",
    "What family do swans belong to?",
    "Name two species of swans."
    ]


FinalEvalR, FinalEvalC, FinalEvalF = ResponseEvaluationMetrics(Queries_method=eas_queries)



eas_queries_2=[ "What is the primary diet of swans?",
    "What language family does Swedish belong to?",
    "Name one country where Swedish is an official language.",
    "What script is used to write Swedish?",
    "When did Theodore Roosevelt serve as President of the United States?",
    "Who was Theodore Roosevelt's vice president?",
    "What event led to Theodore Roosevelt becoming president?",
    "What is the scientific name of the tiger?",
    "Where are tigers commonly found?",
    "What is the primary diet of tigers?",
    "What type of musical instrument is a trumpet?",
    "How many valves does a standard trumpet have?",
    "What is the primary material used to make trumpets?",
    "What language family does Turkish belong to?",
    "Name one country where Turkish is an official language.",
    "What script is used to write Turkish?",
    "How many species of turtles are there?",
    "What is the primary diet of most turtles?",
    "Name one unique feature of turtles.",
    "What is Ulysses S. Grant known for?",
    "Who was Ulysses S. Grant's vice president?",
    "What was Ulysses S. Grant's role during the Civil War?",
    "What is the capital of Uruguay?",
    "Name one major industry in Uruguay.",
    "What is the official language of Uruguay?",
    "What language family does Vietnamese belong to?",
    "Name one country where Vietnamese is an official language.",
    "What script is used to write Vietnamese?",
    "When was Vincent van Gogh born?",
    "What art movement is Vincent van Gogh associated with?",
    "Name one famous painting by Vincent van Gogh.",
    "How many strings does a standard violin have?",
    "What is the primary material used to make violin bows?",
    "Name one famous violinist.",
    "When did Woodrow Wilson serve as President of the United States?",
    "Who was Woodrow Wilson's vice president?",
    "What major global event occurred during Wilson's presidency?",
    "What type of musical instrument is a xylophone?",
    "What is the material used to make xylophone bars?",
    "Name one culture that uses the xylophone.",
    "How many species of zebras are there?",
    "What is the primary diet of zebras?",
    "Where are zebras commonly found?"]

FinalEvalR_easy2, FinalEvalC_easy2, FinalEvalF_easy2 = ResponseEvaluationMetrics(Queries_method=eas_queries_2)


FinalR_easy=pd.concat([FinalEvalR, FinalEvalR_easy2], axis=0)
FinalC_easy=pd.concat([FinalEvalC, FinalEvalC_easy2], axis=0)
FinalF_easy=pd.concat([FinalEvalF, FinalEvalF_easy2], axis=0)



In [ ]:
medium_queries_1 = [
    "What were the key points of the Emancipation Proclamation issued by Abraham Lincoln?",
    "Describe the events leading up to Abraham Lincoln's assassination.",
    "How did Abraham Lincoln's early life influence his political career?",
    "Explain the social structure of an ant colony.",
    "How do ants communicate with each other?",
    "What are the predators of ants in the wild?",
    "Explain the differences between Modern Standard Arabic and Classical Arabic.",
    "How has the Arabic language influenced other languages?",
    "Describe the role of Arabic in the context of Islam.",
    "Explain the process of pollination and how bees contribute to it.",
    "What are the threats to bee populations?",
    "Describe the lifecycle of a honey bee.",
    "Explain the significance of the hard exoskeleton in beetles.",
    "What are the main differences between beetle larvae and adult beetles?",
    "Describe the role of beetles in the decomposition process.",
    "Explain the significance of the Forbidden City in Beijing's history.",
    "How has Beijing's economy evolved in the past two decades?",
    "Describe the impact of air pollution on Beijing's environment.",
    "Explain the process of metamorphosis in butterflies.",
    "How do butterflies contribute to pollination?",
    "Describe the role of butterflies in their ecosystems.",
    "Describe the key domestic policies of Calvin Coolidge's administration.",
    "How did Calvin Coolidge handle the Boston Police Strike?",
    "What was Calvin Coolidge's approach to foreign policy?",
    "Describe the significance of the Canadian Charter of Rights and Freedoms.",
    "How does Canada's healthcare system differ from that of the United States?",
    "Explain the economic importance of the natural resources sector in Canada.",
    "Explain the role of the cello in a string quartet.",
    "Describe the differences between a cello and a double bass.",
    "What is the standard tuning of a cello?",
    "Explain the differences between Simplified and Traditional Chinese characters.",
    "How has the Chinese language influenced other East Asian languages?",
    "Describe the role of Chinese characters in Japanese and Korean writing systems.",
    "Describe the physical characteristics of a cougar.",
    "Explain the hunting techniques by cougars.",
    "What are the threats to cougar populations?",
    "Explain the role of cymbals in an orchestra.",
    "Describe the differences between crash cymbals and ride cymbals.",
    "How are cymbals used in a drum kit?",
    "Explain the historical significance of Dhaka.",
    "How did the partition of India in 1947 affect Dhaka?",
    "Describe the economic activities in Dhaka.",
    "Explain the lifecycle of a dragonfly.",
    "Describe the differences between dragonflies and damselflies.",
    "How do dragonflies contribute to their ecosystems?",
    "Describe the main difference between diving ducks and dabbling ducks.",
    "How do ducks' beaks help them filter food from water?",
    "What are the main predators of ducks?",
    "Describe the significance of the Suez Canal.",
    "How did the ancient Egyptian civilization influence modern Egypt?",
    "What are the major economic activities in Egypt?",
    "Describe the artistic style of El Greco.",
    "How did El Greco's work influence the Baroque movement?",
    "What are the main themes in El Greco's paintings?",
    "Describe the social structure of elephant herds.",
    "How do elephants communicate with each other?",
    "What are the main threats to elephant populations in the wild?"]


FinalEvalR_med1, FinalEvalC_med1, FinalEvalF_med1 = ResponseEvaluationMetrics(Queries_method=medium_queries_1)


medium_queries_2=["Describe the differences between old English and modern English.",
    "How has the British Empire contributed to the spread of English globally?",
    "What are the main characteristics of English as a global lingua franca?",
    "Describe the Finnish education system.",
    "How does Finland's healthcare system compare to other Nordic countries?",
    "What are the major industries in Finland?",
    "Describe the phonological characteristics of Finnish.",
    "How has the Finnish language influenced other languages in the region?",
    "What are the main differences between Finnish and Estonian?",
    "Describe the historical development of the flute.",
    "How does the embouchure affect the sound of a flute?",
    "What are the main differences between a flute and a recorder?",
    "Describe the diet of foxes.",
    "How do foxes adapt to urban environments?",
    "What are the main threats to fox populations?",
    "Describe the influence of Latin on the French language.",
    "How did French become a global language?",
    "What are the main differences between Parisian French and Quebec French?",
    "Describe the domestic policies of Gerald Ford's administration.",
    "How did Gerald Ford handle the aftermath of the Watergate scandal?",
    "What were the key challenges faced by Gerald Ford during his presidency?",
    "Describe the differences between High German and Low German dialects.",
    "How has the German language influenced English?",
    "What are the main grammatical features of German?",
    "Describe the significance of the Ashanti Empire in Ghana's history.",
    "How did Ghana achieve independence from the United Kingdom?",
    "What are the major economic activities in Ghana?",
    "Describe the habitat of giant pandas.",
    "How do giant pandas contribute to their ecosystem?",
    "What are the main threats to giant panda populations?",
    "Describe the social structure of giraffe herds.",
    "How do giraffes use their necks in combat?",
    "What are the main threats to giraffe populations in the wild?",
    "Describe the social structure of gray wolf packs.",
    "How do gray wolves communicate with each other?",
    "What are the main threats to gray wolf populations?",
    "Describe the domestic policies of Grover Cleveland's administration.",
    "How did Grover Cleveland handle economic issues during his presidency?",
    "What were the key challenges faced by Grover Cleveland during his presidency?",
    "Describe the differences between acoustic and electric guitars.",
    "How has the guitar evolved over time?",
    "What are the main techniques used in playing the guitar?",
    "Describe the main themes in Gustav Klimt's artwork.",
    "How did Klimt's work influence the Vienna Secession movement?",
    "What are the characteristics of Klimt's 'Golden Phase'?",
    "Describe the significance of Indonesia's independence from the Netherlands.",
    "How does Indonesia's geography affect its cultural diversity?",
    "What are the main economic activities in Indonesia?",
    "Describe the significance of Newton's laws of motion.",
    "How did Newton contribute to the field of optics?",
    "What were the main ideas in Newton's 'Principia Mathematica'?",
    "Describe the main characteristics of Pollock's 'drip technique'.",
    "How did Pollock's work influence abstract expressionism?",
    "What were the personal struggles that Pollock faced during his life?",
    "Describe the main principles of the Monroe Doctrine.",
    "How did Monroe's presidency contribute to the Era of Good Feelings?",
    "What were the key challenges faced by Monroe during his presidency?",
    "Describe the significance of Watt's separate condenser in steam engines.",
    "How did Watt's inventions contribute to the Industrial Revolution?",
    "What were the main challenges Watt faced in developing his steam engine?",
    "Describe the main grammatical features of Japanese.",
    "How has the Japanese language evolved over time?",
    "What are the differences between Hiragana and Katakana?",
    "Describe the main domestic policies of John Adams' administration.",
    "How did John Adams handle the XYZ Affair?",
    "What were the key challenges faced by John Adams during his presidency?",
    "Describe the locomotion of kangaroos.",
    "How do kangaroos adapt to their environment?",
    "What are the main threats to kangaroo populations?",
    "Describe the social behavior of koalas.",
    "How do koalas adapt to their environment?",
    "What are the main threats to koala populations?",
    "Describe the main grammatical features of Korean.",
    "How has the Korean language evolved over time?",
    "What are the differences between Hangul and Hanja?",
    "Describe the main themes in Leonardo da Vinci's artwork.",
    "How did Leonardo da Vinci contribute to the field of anatomy?",
    "What were the main ideas in Leonardo da Vinci's notebooks?",
    "Describe the social structure of leopards.",
    "How do leopards adapt to their environment?",
    "What are the main threats to leopard populations?",
    "Describe the political system of Liechtenstein.",
    "How does Liechtenstein's economy compare to other European countries?",
    "What are the major industries in Liechtenstein?",
    "Describe the historical significance of Lima.",
    "How does Lima's economy contribute to Peru's GDP?",
    "What are the main cultural activities in Lima?",
    "Describe the social structure of lion prides.",
    "How do lions communicate with each other?",
    "What are the main threats to lion populations?",
    "Describe the habitat requirements of lizards.",
    "How do lizards adapt to their environment?",
    "What are the main threats to lizard populations?",
    "Describe the molting process in lobsters.",
    "How do lobsters adapt to their environment?",
    "What are the main threats to lobster populations?",
    "Describe the historical significance of the Tower of London.",
    "How does London's economy contribute to the UK's GDP?",
    "What are the main cultural activities in London?",
    "Describe the main structural components of a lyre.",
    "How did the lyre contribute to ancient Greek culture?",
    "What are the differences between a lyre and a harp?",
    "Describe the main grammatical features of Malay.",
    "How has the Malay language evolved over time?",
    "What are the differences between Malay and Indonesian?",
    "Describe the main themes in Michelangelo's artwork.",
    "How did Michelangelo contribute to the Renaissance?",
    "What were the main techniques used by Michelangelo in his sculptures?",
    "Describe the domestic policies of Millard Fillmore's administration.",
    "How did Millard Fillmore handle the issue of slavery during his presidency?",
    "What were the challenges faced by Millard Fillmore during his presidency?",
    "Describe the main themes in Rockwell's artwork.",
    "How did Rockwell's work reflect American society?",
    "What were the main techniques used by Rockwell in his paintings?",
    "Describe the intelligence of octopuses.",
    "How do octopuses adapt to their environment?",
    "What are the main threats to octopus populations?",
    "Describe the social behavior of otters.",
    "How do otters adapt to their environment?",
    "What are the main threats to otter populations?",
    "Describe the main themes in Picasso's artwork.",
    "How did Picasso's work influence modern art?",
    "What were the main periods of Picasso's artistic career?",
    "Describe the breeding behavior of penguins.",
    "How do penguins adapt to their environment?",
    "What are the main threats to penguin populations?",
    "Describe the mechanism of sound production in a piano.",
    "How has the design of the piano evolved over time?",
    "What are the main differences between a grand piano and an upright piano?",
    "Describe the main themes in Renoir's artwork.",
    "How did Renoir's work influence the Impressionist movement?",
    "What were the main techniques used by Renoir in his paintings?",
    "Describe the main themes in Mondrian's artwork.",
    "How did Mondrian's work influence modern art?",
    "What were the main techniques used by Mondrian in his paintings?",
    "Describe the social behavior of polar bears.",
    "How do polar bears adapt to their environment?",
    "What are the threats to polar bear populations?",
    "Describe the main grammatical features of Portuguese.",
    "How has the Portuguese language evolved over time?",
    "What are the differences between European Portuguese and Brazilian Portuguese?",
    "Describe the historical significance of Qatar's independence.",
    "How does Qatar's economy compare to other Gulf countries?",
    "What are the main cultural activities in Qatar?",
    "Describe the significance of Singapore's independence.",
    "How does Singapore's economy compare to other Southeast Asian countries?",
    "What are the cultural activities in Singapore?",
    "Describe the grammatical features of Spanish.",
    "How has the Spanish language evolved over time?",
    "What are the differences between Castilian Spanish and Latin American Spanish?",
    "Describe the main grammatical features of Swahili.",
    "How has the Swahili language evolved over time?",
    "What are the differences between Swahili and other Bantu languages?",
    "Describe the social behavior of swans.",
    "How do swans adapt to their environment?",
    "What are the main threats to swan populations?",
    "Describe the main grammatical features of Swedish.",
    "How has the Swedish language evolved over time?",
    "What are the differences between Swedish and Danish?",
    "Describe the domestic policies of Theodore Roosevelt's administration.",
    "How did Theodore Roosevelt handle the issue of conservation during his presidency?",
    "What were the key challenges faced by Theodore Roosevelt during his presidency?",
    "Describe the social behavior of tigers.",
    "How do tigers adapt to their environment?",
    "What are the main threats to tiger populations?",
    "Describe the mechanism of sound production in a trumpet.",
    "How has the design of the trumpet evolved over time?",
    "What are the main differences between a trumpet and a cornet?",
    "Describe the main grammatical features of Turkish.",
    "How has the Turkish language evolved over time?",
    "What are the differences between Turkish and Azerbaijani?",
    "Describe the habitat requirements of turtles.",
    "How do turtles adapt to their environment?",
    "What are the main threats to turtle populations?",
    "Describe the domestic policies of Ulysses S. Grant's administration.",
    "How did Grant handle the issue of Reconstruction during his presidency?",
    "What were the key challenges faced by Grant during his presidency?",
    "Describe the historical significance of Uruguay's independence.",
    "How does Uruguay's economy compare to other South American countries?",
    "What are the main cultural activities in Uruguay?",
    "Describe the main grammatical features of Vietnamese.",
    "How has the Vietnamese language evolved over time?",
    "What are the differences between Northern and Southern Vietnamese dialects?",
    "Describe the main themes in van Gogh's artwork.",
    "How did van Gogh's work influence modern art?",
    "What were the main periods of van Gogh's artistic career?",
    "Describe the mechanism of sound production in a violin.",
    "How has the design of the violin evolved over time?",
    "What are the main differences between a violin and a viola?",
    "Describe the domestic policies of Woodrow Wilson's administration.",
    "How did Wilson handle the issue of World War I during his presidency?",
    "What were the key challenges faced by Wilson during his presidency?",
    "Describe the mechanism of sound production in a xylophone.",
    "How has the design of the xylophone evolved over time?",
    "What are the main differences between a xylophone and a marimba?",
    "Describe the social behavior of zebras.",
    "How do zebras adapt to their environment?",
    "What are the main threats to zebra populations?"
]


FinalEvalR_med2, FinalEvalC_med2, FinalEvalF_med2 = ResponseEvaluationMetrics(Queries_method=medium_queries_2)

FinalR_medium=pd.concat([FinalEvalR_med1, FinalEvalR_med2], axis=0)
FinalC_medium=pd.concat([FinalEvalC_med1, FinalEvalC_med2], axis=0)
FinalF_medium=pd.concat([FinalEvalF_med1, FinalEvalF_med2], axis=0)

In [ ]:


hard_queries_1 =[
    "Analyze Abraham Lincoln's impact on the Civil War strategies of the Union.",
    "Compare and contrast Abraham Lincoln's political views with those of his contemporaries.",
    "Discuss the legal and constitutional challenges Abraham Lincoln faced during his presidency.",
    "Describe the various roles ants play in their ecosystems.",
    "Analyze the evolutionary adaptations that have made ants successful.",
    "Compare the reproductive strategies of different ant species.",
    "Analyze the morphological structure of Arabic verbs.",
    "Discuss the challenges and strategies in teaching Arabic as a second language.",
    "Compare and contrast the various dialects of Arabic spoken across different regions.",
    "Analyze the impact of Colony Collapse Disorder on global agriculture.",
    "Discuss the differences in behavior and social structure between honey bees and bumblebees.",
    "Compare the efficiency of bees as pollinators to other insects.",
    "Analyze the evolutionary adaptations of beetles that have led to their diversity.",
    "Discuss the ecological impact of beetles on agriculture.",
    "Compare the reproductive strategies of different beetle species.",
    "Analyze the role of Beijing in China's political landscape.",
    "Discuss the cultural significance of Beijing's traditional architecture.",
    "Compare the urban development of Beijing with other major global cities.",
    "Analyze the impact of habitat loss on butterfly populations.",
    "Discuss the differences in lifecycle and behavior between butterflies and moths.",
    "Compare the migration patterns of different butterfly species."
    "Analyze the economic impact of Calvin Coolidge's tax policies.",
    "Compare Calvin Coolidge's presidency with that of his predecessor, Warren G. Harding.",
    "Discuss the long-term effects of Calvin Coolidge's administration on the Republican Party.",
    "Analyze the impact of Canadian multiculturalism policies on its society.",
    "Discuss the role of Canada in international peacekeeping missions.",
    "Compare and contrast the political systems of Canada and the United Kingdom.",
    "Analyze the development of cello technique from the Baroque to the modern era.",
    "Discuss the contributions of Pablo Casals to cello performance.",
    "Compare the use of the cello in classical and contemporary music.",
    "Analyze the linguistic structure of Mandarin Chinese grammar.",
    "Discuss the challenges of translating Chinese idioms into English.",
    "Compare the phonological systems of Mandarin and Cantonese.",
    "Analyze the role of cougars in their ecosystems.",
    "Discuss the conservation strategies employed to protect cougars.",
    "Compare the behavior of cougars with that of other big cats like lions and tigers.",
    "Analyze the evolution of cymbal manufacturing techniques.",
    "Discuss the cultural significance of cymbals in different musical traditions.",
    "Compare the sound characteristics of different cymbal alloys.",
    "Analyze the urban challenges faced by Dhaka due to rapid population growth.",
    "Discuss the impact of climate change on Dhaka's infrastructure.",
    "Compare the development of Dhaka with that of other major South Asian cities.",
    "Analyze the evolutionary adaptations of dragonflies for flight.",
    "Discuss the cultural significance of dragonflies in different societies.",
    "Compare the hunting strategies of dragonflies with those of other aerial predators."
    ]

FinalEvalR_Hard1, FinalEvalC_Hard1, FinalEvalF_Hard1 = ResponseEvaluationMetrics(Queries_method=hard_queries_1)


hard_queries_2=[ "Analyze the migratory patterns of ducks in the Northern Hemisphere.",
    "Discuss the importance of ducks in agriculture.",
    "Compare and contrast the plumage differences between male and female ducks.",
    "Analyze the political changes in Egypt over the past century.",
    "Discuss the impact of tourism on Egypt's economy.",
    "Compare the cultural differences between urban and rural areas in Egypt.",
    "Analyze the use of color in El Greco's artwork.",
    "Discuss the significance of El Greco's contributions to Renaissance art.",
    "Compare El Greco's style with that of his contemporaries.",
    "Analyze the role of elephants in their ecosystems.",
    "Discuss the conservation efforts to protect elephants.",
    "Compare the behaviors of African and Asian elephants.",
    "Analyze the impact of English on linguistic diversity worldwide.",
    "Discuss the challenges of teaching English as a second language.",
    "Compare the phonological differences between British and American English.",
    "Analyze the impact of Finland's welfare state on its economy.",
    "Discuss the role of technology and innovation in Finland's development.",
    "Compare the cultural practices of the Sami people with those of other Finnish communities.",
    "Analyze the grammatical structure of Finnish verbs.",
    "Discuss the challenges of translating Finnish literature into English.",
    "Compare the dialects of Finnish spoken in different regions.",
    "Analyze the acoustical properties of the flute.",
    "Discuss the role of the flute in various musical traditions.",
    "Compare the techniques used in playing the Western concert flute and the Indian bansuri.",
    "Analyze the role of foxes in folklore and culture.",
    "Discuss the evolutionary adaptations of foxes for hunting.",
    "Compare the behaviors of different fox species in their natural habitats.",
    "Analyze the impact of French on other languages worldwide.",
    "Discuss the challenges of learning French for non-native speakers.",
    "Compare the phonological systems of French and Italian.",
    "Analyze the impact of Gerald Ford's economic policies.",
    "Discuss the significance of Gerald Ford's pardon of Richard Nixon.",
    "Compare Gerald Ford's presidency with that of his successor, Jimmy Carter.",
    "Analyze the historical development of the German language.",
    "Discuss the challenges of learning German for English speakers.",
    "Compare the phonological systems of German and Dutch.",
    "Analyze the impact of Ghana's colonial history on its modern society.",
    "Discuss the role of Ghana in African politics.",
    "Compare the cultural practices of different ethnic groups in Ghana."]

FinalEvalR_Hard2, FinalEvalC_Hard2, FinalEvalF_Hard2 = ResponseEvaluationMetrics(Queries_method=hard_queries_2)





hard_queries_3=["Analyze the conservation efforts to protect giant pandas.",
    "Discuss the impact of climate change on giant panda habitats.",
    "Compare the behaviors of giant pandas in the wild and in captivity.",
    "Analyze the evolutionary adaptations of giraffes for their height.",
    "Discuss the role of giraffes in their ecosystems.",
    "Compare the different subspecies of giraffes and their geographic distributions.",
    "Analyze the role of gray wolves in their ecosystems.",
    "Discuss the conservation efforts to protect gray wolves.",
    "Compare the behaviors of gray wolves in different regions.",
    "Analyze the impact of Grover Cleveland's economic policies.",
    "Discuss the significance of Grover Cleveland's vetoes.",
    "Compare Grover Cleveland's presidency with that of his predecessors.",
    "Analyze the role of the guitar in different musical genres.",
    "Discuss the impact of guitar technology on modern music.",
    "Compare the playing styles of different famous guitarists.",
    "Analyze the impact of Gustav Klimt's work on modern art.",
    "Discuss the role of eroticism in Klimt's paintings.",
    "Compare the techniques used by Klimt in his portraits and landscapes.",
    "Analyze the political changes in Indonesia since its independence.",
    "Discuss the impact of tourism on Indonesia's economy.",
    "Compare the cultural practices of different ethnic groups in Indonesia.",
    "Analyze the impact of Newton's work on the Scientific Revolution.",
    "Discuss the challenges Newton faced in his scientific career.",
    "Compare the scientific contributions of Newton and Galileo.",
    "Analyze the impact of Jackson Pollock's work on modern art.",
    "Discuss the role of automatism in Pollock's paintings.",
    "Compare Pollock's techniques with those of other abstract expressionists.",
    "Analyze the impact of the Monroe Doctrine on American foreign policy.",
    "Discuss the significance of the Missouri Compromise during Monroe's presidency.",
    "Compare Monroe's presidency with that of his predecessor, James Madison.",
    "Analyze the impact of James Watt's work on modern engineering.",
    "Discuss the collaboration between James Watt and Matthew Boulton.",
    "Compare the steam engines of James Watt and Thomas Newcomen.",
    "Analyze the influence of Chinese on the Japanese writing system.",
    "Discuss the challenges of translating Japanese literature into English.",
    "Compare the phonological systems of Japanese and Korean.",
    "Analyze the impact of the Alien and Sedition Acts on Adams' presidency.",
    "Discuss the significance of John Adams' contributions to American independence.",
    "Compare John Adams' presidency with that of his successor, Thomas Jefferson.",
    "Analyze the role of kangaroos in Australian culture.",
    "Discuss the impact of habitat loss on kangaroo populations.",
    "Compare the different species of kangaroos and their geographic distributions.",
    "Analyze the conservation efforts to protect koalas.",
    "Discuss the impact of climate change on koala habitats.",
    "Compare the behaviors of koalas in the wild and in captivity.",
    "Analyze the influence of Chinese on the Korean writing system.",
    "Discuss the challenges of translating Korean literature into English.",
    "Compare the phonological systems of Korean and Japanese.",
    "Analyze the impact of Leonardo da Vinci's work on the Renaissance.",
    "Discuss the challenges Leonardo da Vinci faced in his scientific career.",
    "Compare the artistic techniques of Leonardo da Vinci and Michelangelo.",
    "Analyze the role of leopards in their ecosystems.",
    "Discuss the conservation efforts to protect leopards.",
    "Compare the behaviors of leopards in different regions.",
    "Analyze the impact of Liechtenstein's banking sector on its economy.",
    "Discuss the role of Liechtenstein in international relations.",
    "Compare the cultural practices of Liechtenstein with those of neighboring countries.",
    "Analyze the urban development of Lima over the past century.",
    "Discuss the impact of tourism on Lima's economy.",
    "Compare the social issues in Lima with those in other major South American cities.",
    "Analyze the role of lions in their ecosystems.",
    "Discuss the conservation efforts to protect lions.",
    "Compare the behaviors of lions in different regions.",
    "Analyze the evolutionary adaptations of lizards.",
    "Discuss the role of lizards in their ecosystems.",
    "Compare the behaviors of different lizard species in their natural habitats.",
    "Analyze the impact of lobster fishing on marine ecosystems.",
    "Discuss the economic importance of lobsters in global seafood markets.",
    "Compare the anatomical differences between clawed lobsters and spiny lobsters.",
    "Analyze the impact of the Industrial Revolution on London's development.",
    "Discuss the role of London in global finance.",
    "Compare the social issues in London with those in other major European cities."
]


FinalEvalR_Hard3, FinalEvalC_Hard3, FinalEvalF_Hard3 = ResponseEvaluationMetrics(Queries_method=hard_queries_3)



hard_queries_4=["Analyze the evolution of the lyre in different cultures.",
    "Discuss the role of the lyre in ancient rituals and ceremonies.",
    "Discuss the playing techniques of lyre and lute.",
    "Analyze the influence of Arabic on the Malay language.",
    "Discuss the challenges of translating Malay literature into English.",
    "Compare the phonological systems of Malay and Javanese.",
    "Analyze the impact of Michelangelo's work on art history.",
    "Discuss the challenges Michelangelo faced during his career.",
    "Compare the artistic techniques of Michelangelo and Leonardo da Vinci.",
    "Analyze the impact of Millard Fillmore's foreign policies.",
    "Discuss the significance of the Compromise of 1850 during Fillmore's presidency.",
    "Compare Fillmore's presidency with that of his predecessor, Zachary Taylor.",
    "Analyze the impact of Norman Rockwell's work on American culture.",
    "Discuss the challenges Rockwell faced during his career.",
    "Compare the artistic styles of Norman Rockwell and other American illustrators.",
    "Analyze the role of octopuses in marine ecosystems.",
    "Discuss the significance of octopus camouflage abilities.",
    "Compare the behaviors of different octopus species in their natural habitats.",
    "Analyze the role of otters in their ecosystems.",
    "Discuss the conservation efforts to protect otters.",
    "Compare the behaviors of different otter species in their natural habitats.",
    "Analyze the impact of Cubism on Picasso's work.",
    "Discuss the significance of Picasso's 'Guernica'.",
    "Compare the techniques used by Picasso and Georges Braque in their Cubist works.",
    "Analyze the role of penguins in their ecosystems.",
    "Discuss the impact of climate change on penguin habitats.",
    "Compare the behaviors of different penguin species in their natural habitats.",
    "Analyze the role of the piano in classical music.",
    "Discuss the impact of piano technology on modern music.",
    "Compare the playing techniques of different famous pianists.",
    "Analyze the impact of Renoir's work on modern art.",
    "Discuss the challenges Renoir faced during his career.",
    "Compare the artistic techniques of Renoir and Claude Monet.",
    "Analyze the impact of Mondrian's work on abstract art.",
    "Discuss the significance of Mondrian's 'Composition with Red, Blue, and Yellow'.",
    "Compare the artistic techniques of Mondrian and Theo van Doesburg.",
    "Analyze the role of polar bears in their ecosystems.",
    "Discuss the impact of climate change on polar bear habitats.",
    "Compare the behaviors of polar bears in different regions.",
    "Analyze the influence of Latin on the Portuguese language.",
    "Discuss the challenges of translating Portuguese literature into English.",
    "Compare the phonological systems of Portuguese and Spanish.",
    "Analyze the impact of Qatar's natural gas reserves on its economy.",
    "Discuss the role of Qatar in international politics.",
    "Compare the social development of Qatar with that of neighboring countries.",
    "Analyze the impact of Singapore's port on its economy.",
    "Discuss the role of Singapore in international trade.",
    "Compare the social development of Singapore with that of neighboring countries."]


FinalEvalR_Hard4, FinalEvalC_Hard4, FinalEvalF_Hard4 = ResponseEvaluationMetrics(Queries_method=hard_queries_4)



hard_queries_5=[
    "Analyze the influence of Arabic on the Spanish language.",
    "Discuss the challenges of translating Spanish literature into English.",
    "Compare the phonological systems of Spanish and Portuguese.",
    "Analyze the influence of Arabic on the Swahili language.",
    "Discuss the challenges of translating Swahili literature into English.",
    "Compare the phonological systems of Swahili and Somali.",
    "Analyze the role of swans in their ecosystems.",
    "Discuss the significance of swans in mythology and culture.",
    "Compare the behaviors of different swan species in their natural habitats.",
    "Analyze the influence of German on the Swedish language.",
    "Discuss the challenges of translating Swedish literature into English.",
    "Compare the phonological systems of Swedish and Norwegian.",
    "Analyze the impact of Theodore Roosevelt's foreign policies.",
    "Discuss the significance of the Panama Canal during Roosevelt's presidency.",
    "Compare Roosevelt's presidency with that of William Howard Taft.",
    "Analyze the role of tigers in their ecosystems.",
    "Discuss the impact of poaching on tiger populations.",
    "Compare the behaviors of different tiger subspecies in their natural habitats.",
    "Analyze the role of the trumpet in classical music.",
    "Discuss the impact of trumpet technology on modern music.",
    "Compare the playing techniques of different famous trumpeters.",
    "Analyze the influence of Arabic on the Turkish language.",
    "Discuss the challenges of translating Turkish literature into English.",
    "Compare the phonological systems of Turkish and Persian.",
    "Analyze the evolutionary adaptations of turtles.",
    "Discuss the role of turtles in their ecosystems.",
    "Compare the behaviors of different turtle species in their natural habitats.",
    "Analyze the impact of Grant's foreign policies.",
    "Discuss the significance of the Civil Rights Act of 1875 during Grant's presidency.",
    "Compare Grant's presidency with that of his predecessor, Andrew Johnson.",
    "Analyze the impact of agriculture on Uruguay's economy.",
    "Discuss the role of Uruguay in international politics.",
    "Compare the social development of Uruguay with that of neighboring countries.",
    "Analyze the influence of Chinese on the Vietnamese language.",
    "Discuss the challenges of translating Vietnamese literature into English.",
    "Compare the phonological systems of Vietnamese and Thai.",
    "Analyze the impact of van Gogh's mental health on his work.",
    "Discuss the significance of van Gogh's 'Starry Night'.",
    "Compare the techniques used by van Gogh and Paul Gauguin.",
    "Analyze the role of the violin in classical music.",
    "Discuss the impact of violin technology on modern music.",
    "Compare the playing techniques of different famous violinists.",
    "Analyze the impact of Wilson's foreign policies.",
    "Discuss the significance of the League of Nations during Wilson's presidency.",
    "Compare Wilson's presidency with that of his predecessor, William Howard Taft.",
    "Analyze the role of the xylophone in orchestral music.",
    "Discuss the impact of xylophone technology on modern music.",
    "Compare the playing techniques of different famous xylophonists.",
    "Analyze the role of zebras in their ecosystems.",
    "Discuss the impact of habitat loss on zebra populations.",
    "Compare the behaviors of different zebra species in their natural habitats."
]
    

FinalEvalR_Hard5, FinalEvalC_Hard5, FinalEvalF_Hard5 = ResponseEvaluationMetrics(Queries_method=hard_queries_5)


FinalR_hard=pd.concat([FinalEvalR_Hard1, FinalEvalR_Hard2, FinalEvalR_Hard3, FinalEvalR_Hard4, FinalEvalR_Hard5], axis=0)
FinalC_hard=pd.concat([FinalEvalC_Hard1, FinalEvalC_Hard2, FinalEvalC_Hard3, FinalEvalC_Hard4, FinalEvalC_Hard5], axis=0)
FinalF_hard=pd.concat([FinalEvalF_Hard1, FinalEvalF_Hard2, FinalEvalF_Hard3, FinalEvalF_Hard4, FinalEvalF_Hard5], axis=0)


## Queries with few keywords Present

In [ ]:
easy_queriesN1 = [
    "In which year was the 16th President of the USA born?",
    "Who served as the vice president during the first term of the 16th President?",
    "Which notable address starts with 'Four score and seven years ago'?",
    "What are the basic physical features of Formicidae?",
    "How many types of Formicidae exist globally?",
    "What constitutes the primary diet of Formicidae?",
    "Which script is used in regions where a Semitic language is spoken?",
    "List three countries where the principal language belongs to the Afroasiatic family.",
    "What is the typical word order in sentences of the primary language spoken in the Middle East?",
    "What crucial role do Apis mellifera play in nature?",
    "Identify the most well-known species of Apis.",
    "How do Apis mellifera communicate with one another?",
    "Which taxonomic order includes Coleoptera?",
    "How many species of Coleoptera have been identified?",
    "What do Coccinellidae primarily consume?",
    "Which city is the capital of the People's Republic?",
    "Mention a renowned historic site in the capital of the People's Republic.",
    "What is the population size of the capital of the People's Republic?",
    "After the larval stage, what is the next phase in the Lepidoptera life cycle?",
    "Identify a commonly known species of Lepidoptera.",
    "What do Lepidoptera typically consume?"
]

FinalEvalR_easyN1, FinalEvalC_easyN1, FinalEvalF_easyN1, FinalResponseLength_easyN1 = ResponseEvaluationMetrics(Queries_method=easy_queriesN1)


easy_queriesN11 = [
    "What was the moniker of the 30th President of the United States?",
    "When did the 30th President of the United States serve?",
    "Who was the vice president during the tenure of the 30th President?",
    "What is the capital city of the country north of the United States?",
    "Name three provinces in the country north of the United States.",
    "What are the official languages of the country north of the United States?",
    "What is a bowed string instrument with four strings that is larger than a violin?",
    "How many strings does a bowed string instrument with four strings have?",
    "Name one famous player of the bowed string instrument with four strings.",
    "What script is used to write the language spoken by the majority in China?",
    "How many tones are there in the language spoken by the majority in China?",
    "Name one dialect of the language spoken by the majority in China besides the main one.",
    "What is another name for a large feline found in the Americas?",
    "Where are large felines found in the Americas commonly located?",
    "What is the primary diet of large felines found in the Americas?",
    "What is a percussion instrument made of metal that produces a loud crash when struck?",
    "Name three types of percussion instruments made of metal.",
    "What materials are percussion instruments made of metal typically made from?",
    "What is the capital of the South Asian country that has the city known as the 'City of Mosques'?",
    "What river is the 'City of Mosques' located on?",
    "What is the 'City of Mosques' known as?",
    "What order do insects with two pairs of large wings belong to?",
    "How many pairs of wings do insects with two pairs of large wings have?",
    "What is the primary habitat of insects with two pairs of large wings?"
]



FinalEvalR_easyN11, FinalEvalC_easyN11, FinalEvalF_easyN11, FinalResponseLength_easyN11 = ResponseEvaluationMetrics(Queries_method=easy_queriesN11)



easy_queriesN2 = [
    "What avian family do these waterfowl belong to?",
    "What is the main food source for these waterfowl?",
    "List three varieties of these waterfowl.",
    "What is the administrative center of the African country known for pyramids?",
    "Name a significant waterway in the African country known for pyramids.",
    "What is the national language of the African country known for pyramids?",
    "What was the birth name of the artist known as El Greco?",
    "In which country was the artist known as El Greco born?",
    "Name one renowned artwork by the artist known as El Greco.",
    "How many distinct kinds of the largest land mammals exist?",
    "What is the main food source for the largest land mammals?",
    "Identify a unique physical trait of the largest land mammals.",
    "To which language family does the global lingua franca belong?",
    "Name two nations where the global lingua franca is an official language.",
    "What historical event greatly influenced the transition from Old to Middle English?",
    "What is the administrative center of the Nordic country with thousands of lakes?",
    "Name one official language of the Nordic country with thousands of lakes.",
    "What is the population of the Nordic country with thousands of lakes?",
    "To which language family does the Uralic tongue of the Nordic country belong?",
    "Name one country other than the Nordic one where the Uralic tongue is spoken.",
    "How many grammatical cases are there in the Uralic tongue of the Nordic country?",
    "To which family of musical instruments does the instrument played by blowing across a hole belong?",
    "How is sound generated in the instrument played by blowing across a hole?",
    "Name one variety of the instrument played by blowing across a hole other than the Western concert type.",
    "How many distinct kinds of small, wild canines are there?",
    "What is the primary habitat of the most widespread small, wild canine?",
    "Identify a distinctive physical feature of small, wild canines."
]


FinalEvalR_easyN2, FinalEvalC_easyN2, FinalEvalF_easyN2, FinalResponseLength_easyN2 = ResponseEvaluationMetrics(Queries_method=easy_queriesN2)


easy_queriesN3 = [
    "What language family does the official tongue of France belong to?",
    "Name two countries where the official tongue of France is used.",
    "What is the origin of the official tongue of France?",
    "When did the 38th President of the United States serve?",
    "Who was the vice president during the 38th President's tenure?",
    "What event led to the 38th President becoming president?",
    "What language family does the official tongue of Germany belong to?",
    "Name two countries where the official tongue of Germany is used.",
    "What is the standard form of the official tongue of Germany called?",
    "What is the capital of the West African nation known for gold?",
    "Name one major waterway in the West African nation known for gold.",
    "What is the national language of the West African nation known for gold?",
    "What is the primary diet of the black-and-white bear species?",
    "Where are the black-and-white bear species naturally found?",
    "What is one physical characteristic unique to the black-and-white bear species?",
    "What is the scientific name of the tallest land animal?",
    "How tall can male members of the tallest land animal species grow?",
    "What is the primary diet of the tallest land animals?",
    "What is the scientific name of the largest wild canid?",
    "Where are the largest wild canids commonly found?",
    "What is the primary diet of the largest wild canids?",
    "When did the 22nd and 24th President of the United States serve?",
    "What is unique about the 22nd and 24th President's terms in office?",
    "Who was the vice president during the 22nd and 24th President's tenure?",
    "What family of musical instruments does the stringed instrument played by strumming belong to?",
    "How many strings does a standard stringed instrument played by strumming have?",
    "Name one famous musician known for playing the stringed instrument."
]


FinalEvalR_easyN3, FinalEvalC_easyN3, FinalEvalF_easyN3, FinalResponseLength_easyN3 = ResponseEvaluationMetrics(Queries_method=easy_queriesN3)


easy_queriesN4 = [
    "When was the artist known for 'The Kiss' born?",
    "What art movement was the artist known for 'The Kiss' associated with?",
    "Name one famous painting by the artist known for 'The Kiss'.",
    "What is the capital city of the Southeast Asian archipelago nation?",
    "Name one major island in the Southeast Asian archipelago nation.",
    "What is the official language of the Southeast Asian archipelago nation?",
    "When was the scientist known for the laws of motion born?",
    "What is the scientist known for the laws of motion best known for?",
    "Name one major work published by the scientist known for the laws of motion.",
    "When was the artist known for his 'drip technique' born?",
    "What art movement was the artist known for his 'drip technique' associated with?",
    "Name one famous painting by the artist known for his 'drip technique'.",
    "When did the 5th President of the United States serve?",
    "What is the 5th President of the United States best known for?",
    "Name one major event that occurred during the 5th President's tenure.",
    "When was the inventor known for improving the steam engine born?",
    "What is the inventor known for improving the steam engine best known for?",
    "Name one major improvement the inventor known for the steam engine made.",
    "What writing systems are used in the primary language of Japan?",
    "How many main dialects are there in the primary language of Japan?",
    "What is the standard form of the primary language of Japan called?",
    "When did the 2nd President of the United States serve?",
    "What is the 2nd President of the United States best known for?",
    "Who was the vice president during the 2nd President's tenure?"
]


FinalEvalR_easyN4, FinalEvalC_easyN4, FinalEvalF_easyN4, FinalResponseLength_easyN4 = ResponseEvaluationMetrics(Queries_method=easy_queriesN4)


easy_queriesN5 = [
    "What marsupial family do these hopping animals belong to?",
    "Name two species of these hopping animals.",
    "What is the primary diet of these hopping animals?",
    "What is the scientific name of the tree-dwelling marsupial found in Australia?",
    "Where are the tree-dwelling marsupials found in Australia naturally located?",
    "What is the primary diet of the tree-dwelling marsupials found in Australia?",
    "What writing systems are used in the primary language of South Korea?",
    "How many main dialects are there in the primary language of South Korea?",
    "What is the standard form of the primary language of South Korea called?",
    "When was the Renaissance artist known for 'The Last Supper' born?",
    "What is the Renaissance artist known for 'The Last Supper' best known for?",
    "Name one famous painting by the Renaissance artist known for 'The Last Supper'.",
    "What is the scientific name of the large spotted cat?",
    "Where are the large spotted cats commonly found?",
    "What is the primary diet of the large spotted cats?",
    "What is the capital of the small European principality known for its banks?",
    "What is the official language of the small European principality known for its banks?",
    "What is the population of the small European principality known for its banks?",
    "What is the capital of the South American country known for Machu Picchu?",
    "Name one famous landmark in the capital of the South American country known for Machu Picchu.",
    "What is the population of the capital of the South American country known for Machu Picchu?",
    "What is the scientific name of the large feline known as the 'king of the jungle'?",
    "Where are the large felines known as the 'king of the jungle' commonly found?",
    "What is the primary diet of the large felines known as the 'king of the jungle'?",
    "How many species of small, scaly reptiles are there?",
    "What is the primary diet of most small, scaly reptiles?",
    "Name one unique feature of small, scaly reptiles."
]


FinalEvalR_easyN5, FinalEvalC_easyN5, FinalEvalF_easyN5, FinalResponseLength_easyN5 = ResponseEvaluationMetrics(Queries_method=easy_queriesN5)


easy_queriesN6 = [
    "What family do clawed marine crustaceans belong to?",
    "Name two types of marine crustaceans with claws.",
    "What is the primary diet of marine crustaceans with claws?",
    "What is the capital city of the United Kingdom?",
    "Name one famous landmark in the capital city of the United Kingdom.",
    "What river runs through the capital city of the United Kingdom?",
    "What type of musical instrument is a harp-like instrument with strings?",
    "Name one ancient civilization that used the harp-like instrument with strings.",
    "What is the primary material used for the strings of the harp-like instrument?",
    "What language family does the official tongue of Malaysia belong to?",
    "Name one country where the official tongue of Malaysia is an official language.",
    "What script is used to write the official tongue of Malaysia?",
    "When was the Renaissance artist known for 'David' born?",
    "What is the Renaissance artist known for 'David' best known for?",
    "Name one famous sculpture by the Renaissance artist known for 'David'.",
    "When did the 13th President of the United States serve?",
    "Who was the vice president during the tenure of the 13th President?",
    "What event led to the 13th President becoming president?",
    "When was the American illustrator known for 'The Saturday Evening Post' born?",
    "What is the American illustrator known for 'The Saturday Evening Post' best known for?",
    "Name one famous painting by the American illustrator known for 'The Saturday Evening Post'.",
    "How many arms does a soft-bodied marine animal with eight limbs have?",
    "What is the primary diet of soft-bodied marine animals with eight limbs?",
    "Where are soft-bodied marine animals with eight limbs commonly found?",
    "What family do semi-aquatic mammals with webbed feet belong to?",
    "Name one type of semi-aquatic mammal with webbed feet.",
    "What is the primary diet of semi-aquatic mammals with webbed feet?"
]


FinalEvalR_easyN6, FinalEvalC_easyN6, FinalEvalF_easyN6, FinalResponseLength_easyN6 = ResponseEvaluationMetrics(Queries_method=easy_queriesN6)




easy_queriesN7 = [
    "When was the artist known for 'Guernica' born?",
    "What art movement is the artist known for 'Guernica' associated with?",
    "Name one famous painting by the artist known for 'Guernica'.",
    "How many species of flightless seabirds are there?",
    "What is the primary diet of flightless seabirds?",
    "Where are flightless seabirds commonly found?",
    "How many keys does a standard keyboard instrument have?",
    "What are the main types of keyboard instruments?",
    "Who is known as the 'father of the modern keyboard instrument'?",
    "When was the Impressionist artist known for 'Luncheon of the Boating Party' born?",
    "What art movement is the Impressionist artist known for 'Luncheon of the Boating Party' associated with?",
    "Name one famous painting by the Impressionist artist known for 'Luncheon of the Boating Party'.",
    "When was the De Stijl artist known for 'Composition with Red, Blue, and Yellow' born?",
    "What art movement is the De Stijl artist known for 'Composition with Red, Blue, and Yellow' associated with?",
    "Name one famous painting by the De Stijl artist known for 'Composition with Red, Blue, and Yellow'.",
    "What is the scientific name of the large white bear found in the Arctic?",
    "Where are large white bears found in the Arctic commonly located?",
    "What is the primary diet of large white bears found in the Arctic?",
    "What language family does the official tongue of Portugal belong to?",
    "Name one country where the official tongue of Portugal is an official language.",
    "What script is used to write the official tongue of Portugal?",
    "What is the capital of the Middle Eastern country known for its natural gas reserves?",
    "Name one major industry in the Middle Eastern country known for its natural gas reserves.",
    "What is the official language of the Middle Eastern country known for its natural gas reserves?",
    "What is the capital of the Southeast Asian city-state known for its port?",
    "Name one major industry in the Southeast Asian city-state known for its port.",
    "What is the official language of the Southeast Asian city-state known for its port?"
]

FinalEvalR_easyN7, FinalEvalC_easyN7, FinalEvalF_easyN7, FinalResponseLength_easyN7 = ResponseEvaluationMetrics(Queries_method=easy_queriesN7)



easy_queriesN8 = [
    "What language family does the official tongue of Spain belong to?",
    "Name two countries where the official tongue of Spain is an official language.",
    "What script is used to write the official tongue of Spain?",
    "What language family does the lingua franca of East Africa belong to?",
    "Name one country where the lingua franca of East Africa is an official language.",
    "What script is used to write the lingua franca of East Africa?",
    "What family do large waterfowl with long necks belong to?",
    "Name two species of large waterfowl with long necks.",
    "What is the primary diet of large waterfowl with long necks?",
    "What language family does the native tongue of Sweden belong to?",
    "Name one country where the native tongue of Sweden is an official language.",
    "What script is used to write the native tongue of Sweden?",
    "When did the 26th President of the United States serve?",
    "Who was the vice president during the tenure of the 26th President?",
    "What event led to the 26th President becoming president?",
    "What is the scientific name of the large striped feline?",
    "Where are large striped felines commonly found?",
    "What is the primary diet of large striped felines?",
    "What type of musical instrument is a brass wind instrument with three valves?",
    "How many valves does a standard brass wind instrument have?",
    "What is the primary material used to make brass wind instruments?",
    "What language family does the official tongue of Turkey belong to?",
    "Name one  official language of Turkey.",
    "What script is used to write the official language of Turkey?",
    "How many species of hard-shelled reptiles are there?",
    "What is the primary diet of most hard-shelled reptiles?",
    "Name one unique feature of hard-shelled reptiles."
]


FinalEvalR_easyN8, FinalEvalC_easyN8, FinalEvalF_easyN8, FinalResponseLength_easyN8 = ResponseEvaluationMetrics(Queries_method=easy_queriesN8)



easy_queriesN9 = [
    "When did the 18th President of the United States serve?",
    "Who was the vice president during the tenure of the 18th President?",
    "What was the role of the 18th President during the Civil War?",
    "What is the capital of the South American country known for its beef industry?",
    "Name one major industry in the South American country known for its beef industry.",
    "What is the official language of the South American country known for its beef industry?",
    "What language family does the official tongue of Vietnam belong to?",
    "Name one country where the official tongue of Vietnam is an official language.",
    "What script is used to write the official tongue of Vietnam?",
    "When was the post-impressionist artist known for 'Starry Night' born?",
    "What art movement is the post-impressionist artist known for 'Starry Night' associated with?",
    "Name one famous painting by the post-impressionist artist known for 'Starry Night'.",
    "How many strings does a standard bowed string instrument have?",
    "What is the primary material used to make bows for bowed string instruments?",
    "Name one famous player of the standard bowed string instrument.",
    "When did the 28th President of the United States serve?",
    "Who was the vice president during the tenure of the 28th President?",
    "What major global event occurred during the tenure of the 28th President?",
    "What type of musical instrument is a percussion instrument with wooden bars?",
    "What is the primary material used to make bars for the percussion instrument?",
    "Name one culture that uses the percussion instrument with wooden bars in traditional music.",
    "How many species of striped African equids are there?",
    "What is the primary diet of striped African equids?",
    "Where are striped African equids commonly found?"
]


FinalEvalR_easyN9, FinalEvalC_easyN9, FinalEvalF_easyN9, FinalResponseLength_easyN9 = ResponseEvaluationMetrics(Queries_method=easy_queriesN9)



FinalR_easyN=pd.concat([FinalEvalR_easyN1,FinalEvalR_easyN11, FinalEvalR_easyN2, FinalEvalR_easyN3, FinalEvalR_easyN4, FinalEvalR_easyN5, FinalEvalR_easyN6, FinalEvalR_easyN7,
                        FinalEvalR_easyN8, FinalEvalR_easyN9], axis=0)
FinalC_easyN=pd.concat([FinalEvalC_easyN1,FinalEvalC_easyN11, FinalEvalC_easyN2, FinalEvalC_easyN3, FinalEvalC_easyN4, FinalEvalC_easyN5, FinalEvalC_easyN6, FinalEvalC_easyN7,
                        FinalEvalC_easyN8, FinalEvalC_easyN9], axis=0)
FinalF_easyN=pd.concat([FinalEvalF_easyN1, FinalEvalF_easyN11, FinalEvalF_easyN2, FinalEvalF_easyN3, FinalEvalF_easyN4, FinalEvalF_easyN5, FinalEvalF_easyN6, FinalEvalF_easyN7,
                        FinalEvalF_easyN8, FinalEvalF_easyN9], axis=0)
FinalL_easyN=pd.concat([FinalResponseLength_easyN1, FinalResponseLength_easyN11, FinalResponseLength_easyN2, FinalResponseLength_easyN3, FinalResponseLength_easyN4, FinalResponseLength_easyN5,
                        FinalResponseLength_easyN6, FinalResponseLength_easyN7, FinalResponseLength_easyN8, FinalResponseLength_easyN9], axis=0)

In [ ]:
medium_queriesN1 = [
    "What were the main aspects of the Proclamation issued to free slaves by the 16th President?",
    "Detail the events leading to the assassination of the 16th President.",
    "How did the early years of the 16th President shape his political path?",
    "Describe the hierarchical organization within a Formicidae community.",
    "By what means do Formicidae communicate?",
    "Who are the primary predators of Formicidae in nature?",
    "Differentiate between the modern and classical forms of this Semitic language.",
    "How has this Semitic language impacted other tongues?",
    "What significance does this Semitic language hold in Islamic practice?",
    "Explain how Apis mellifera assist in the pollination process.",
    "What are the major dangers to global Apis mellifera populations?",
    "Outline the developmental stages of Apis mellifera.",
    "Why is the rigid outer shell important for Coleoptera?",
    "How do Coleoptera larvae differ from mature Coleoptera?",
    "What role do Coleoptera play in breaking down organic material?",
    "What is the historical importance of the Forbidden City in the capital of the People's Republic?",
    "How has the economy of the capital of the People's Republic changed over the last twenty years?",
    "What environmental issues arise from air pollution in the capital of the People's Republic?",
    "Explain the transformation process in Lepidoptera.",
    "How do Lepidoptera aid in pollination?",
    "What ecological role do Lepidoptera play?"
]



FinalEvalR_mediumN1, FinalEvalC_mediumN1, FinalEvalF_mediumN1, FinalResponseLength_mediumN1 = ResponseEvaluationMetrics(Queries_method=medium_queriesN1)



medium_queriesN11 = [
    "Describe the key domestic policies of the 30th President's administration.",
    "How did the 30th President handle the Boston Police Strike?",
    "What was the 30th President's approach to foreign policy?",
    "Describe the significance of the Charter of Rights and Freedoms in the country north of the United States.",
    "How does the healthcare system in the country north of the United States differ from that of the United States?",
    "Explain the economic importance of the natural resources sector in the country north of the United States.",
    "Explain the role of the bowed string instrument with four strings in a string quartet.",
    "Describe the differences between a bowed string instrument with four strings and a double bass.",
    "What is the standard tuning of a bowed string instrument with four strings?",
    "Explain the differences between Simplified and Traditional characters in the language spoken by the majority in China.",
    "How has the language spoken by the majority in China influenced other East Asian languages?",
    "Describe the role of the script used in the language spoken by the majority in China in Japanese and Korean writing systems.",
    "Describe the physical characteristics of large felines found in the Americas.",
    "Explain the hunting techniques used by large felines found in the Americas.",
    "What are the main threats to large feline populations in the Americas?",
    "Explain the role of percussion instruments made of metal in an orchestra.",
    "Describe the differences between crash percussion instruments made of metal and ride percussion instruments made of metal.",
    "How are percussion instruments made of metal used in a drum kit?",
    "Explain the historical significance of the 'City of Mosques' during the Mughal period.",
    "How did the partition of India in 1947 affect the 'City of Mosques'?",
    "Describe the main economic activities in the 'City of Mosques'.",
    "Explain the lifecycle of insects with two pairs of large wings.",
    "Describe the differences between insects with two pairs of large wings and damselflies.",
    "How do insects with two pairs of large wings contribute to their ecosystems?"
]



FinalEvalR_mediumN11, FinalEvalC_mediumN11, FinalEvalF_mediumN11, FinalResponseLength_mediumN11 = ResponseEvaluationMetrics(Queries_method=medium_queriesN11)



medium_queriesN2 = [
    "Describe the differences between diving and dabbling waterfowl.",
    "How do the beaks of these waterfowl help them filter food from water?",
    "What are the main natural enemies of these waterfowl?",
    "Explain the importance of the man-made waterway connecting the Mediterranean to the Red Sea.",
    "How did the ancient civilization along the Nile impact the modern nation?",
    "What are the major industries in the modern nation along the Nile?",
    "Describe the distinctive style of the artist known as El Greco.",
    "How did the work of El Greco influence the Baroque movement?",
    "What are the predominant themes in the paintings of El Greco?",
    "Describe the social organization of herds of the largest land mammals.",
    "How do the largest land mammals communicate with each other?",
    "What are the primary threats to the largest land mammals in the wild?",
    "Describe the differences between the Old form of the global lingua franca and its modern counterpart.",
    "How did the British Empire contribute to the global spread of the lingua franca?",
    "What are the main features of the global lingua franca as an international language?",
    "Describe the education system in the Nordic country with thousands of lakes.",
    "How does the healthcare system of the Nordic country compare to other countries in the region?",
    "What are the main economic sectors in the Nordic country?",
    "Describe the sound characteristics of the Uralic tongue of the Nordic country.",
    "How has the Uralic tongue of the Nordic country influenced other languages in the area?",
    "What are the main differences between the Uralic tongues of the Nordic country and Estonia?",
    "Outline the historical evolution of the instrument played by blowing across a hole.",
    "How does the embouchure affect the sound produced by the instrument played by blowing across a hole?",
    "What are the main differences between the instrument played by blowing across a hole and a recorder?",
    "Describe the dietary habits of small, wild canines.",
    "How do small, wild canines adapt to living in urban environments?",
    "What are the main threats to small, wild canines?"
]


FinalEvalR_mediumN2, FinalEvalC_mediumN2, FinalEvalF_mediumN2, FinalResponseLength_mediumN2 = ResponseEvaluationMetrics(Queries_method=medium_queriesN2)



medium_queriesN3 = [
    "Describe the influence of Latin on the official tongue of France.",
    "How did the official tongue of France become a global language?",
    "What are the main differences between Parisian and Quebec versions of the official tongue of France?",
    "Describe the domestic policies of the 38th President's administration.",
    "How did the 38th President handle the aftermath of the political scandal leading to his presidency?",
    "What were the key challenges faced by the 38th President during his tenure?",
    "Describe the differences between High and Low dialects of the official tongue of Germany.",
    "How has the official tongue of Germany influenced English?",
    "What are the main grammatical features of the official tongue of Germany?",
    "Describe the significance of the Ashanti Empire in the history of the West African nation known for gold.",
    "How did the West African nation known for gold achieve independence from the UK?",
    "What are the major economic activities in the West African nation known for gold?",
    "Describe the habitat requirements of the black-and-white bear species.",
    "How do the black-and-white bear species contribute to their ecosystem?",
    "What are the main threats to the black-and-white bear species populations?",
    "Describe the social structure of herds of the tallest land animals.",
    "How do the tallest land animals use their necks in combat?",
    "What are the main threats to the tallest land animals in the wild?",
    "Describe the social structure of packs of the largest wild canids.",
    "How do the largest wild canids communicate with each other?",
    "What are the main threats to the largest wild canid populations?",
    "Describe the domestic policies of the 22nd and 24th President's administration.",
    "How did the 22nd and 24th President handle economic issues during his presidency?",
    "What were the key challenges faced by the 22nd and 24th President during his presidency?",
    "Describe the differences between acoustic and electric versions of the stringed instrument played by strumming.",
    "How has the stringed instrument played by strumming evolved over time?",
    "What are the main techniques used in playing the stringed instrument?"
]


FinalEvalR_mediumN3, FinalEvalC_mediumN3, FinalEvalF_mediumN3, FinalResponseLength_mediumN3 = ResponseEvaluationMetrics(Queries_method=medium_queriesN3)

medium_queriesN4 = [
    "Describe the main themes in the artwork of the artist known for 'The Kiss'.",
    "How did the work of the artist known for 'The Kiss' influence the Vienna Secession movement?",
    "What are the characteristics of the 'Golden Phase' of the artist known for 'The Kiss'?",
    "Describe the significance of the Southeast Asian archipelago nation's independence from the Netherlands.",
    "How does the geography of the Southeast Asian archipelago nation affect its cultural diversity?",
    "What are the main economic activities in the Southeast Asian archipelago nation?",
    "Describe the significance of the laws of motion proposed by the scientist known for his work on gravity.",
    "How did the scientist known for his work on gravity contribute to the field of optics?",
    "What were the main ideas in 'Principia Mathematica' by the scientist known for his work on gravity?",
    "Describe the main characteristics of the 'drip technique' used by the artist known for his abstract paintings.",
    "How did the work of the artist known for his abstract paintings influence abstract expressionism?",
    "What were the personal struggles faced by the artist known for his 'drip technique'?",
    "Describe the main principles of the doctrine named after the 5th President of the United States.",
    "How did the 5th President's tenure contribute to the Era of Good Feelings?",
    "What were the key challenges faced by the 5th President during his tenure?",
    "Describe the significance of the separate condenser invented by the person known for improving the steam engine.",
    "How did the inventions of the person known for improving the steam engine contribute to the Industrial Revolution?",
    "What were the main challenges faced by the inventor of the improved steam engine?",
    "Describe the main grammatical features of the primary language of Japan.",
    "How has the primary language of Japan evolved over time?",
    "What are the differences between Hiragana and Katakana?",
    "Describe the main domestic policies of the 2nd President's administration.",
    "How did the 2nd President handle the XYZ Affair?",
    "What were the key challenges faced by the 2nd President during his tenure?"
]


FinalEvalR_mediumN4, FinalEvalC_mediumN4, FinalEvalF_mediumN4, FinalResponseLength_mediumN4 = ResponseEvaluationMetrics(Queries_method=medium_queriesN4)




medium_queriesN5 = [
    "Describe the locomotion of the hopping marsupials.",
    "How do the hopping marsupials adapt to their environment?",
    "What are the main threats to the hopping marsupial populations?",
    "Describe the social behavior of the tree-dwelling marsupials.",
    "How do the tree-dwelling marsupials adapt to their environment?",
    "What are the main threats to the tree-dwelling marsupial populations?",
    "Describe the main grammatical features of the primary language of South Korea.",
    "How has the primary language of South Korea evolved over time?",
    "What are the differences between Hangul and Hanja?",
    "Describe the main themes in the artwork of the Renaissance artist known for 'The Last Supper'.",
    "How did the Renaissance artist known for 'The Last Supper' contribute to the field of anatomy?",
    "What were the main ideas in the notebooks of the Renaissance artist known for 'The Last Supper'?",
    "Describe the social structure of the large spotted cats.",
    "How do the large spotted cats adapt to their environment?",
    "What are the main threats to the large spotted cat populations?",
    "Describe the political system of the small European principality known for its banks.",
    "How does the economy of the small European principality compare to other European countries?",
    "What are the major industries in the small European principality known for its banks?",
    "Describe the historical significance of the capital of the South American country known for Machu Picchu.",
    "How does the economy of the capital of the South American country known for Machu Picchu contribute to the country's GDP?",
    "What are the main cultural activities in the capital of the South American country known for Machu Picchu?",
    "Describe the social structure of prides of the large felines known as the 'king of the jungle'.",
    "How do the large felines known as the 'king of the jungle' communicate with each other?",
    "What are the main threats to the populations of the large felines known as the 'king of the jungle'?",
    "Describe the habitat requirements of small, scaly reptiles.",
    "How do small, scaly reptiles adapt to their environment?",
    "What are the main threats to the small, scaly reptile populations?"
]

FinalEvalR_mediumN5, FinalEvalC_mediumN5, FinalEvalF_mediumN5, FinalResponseLength_mediumN5 = ResponseEvaluationMetrics(Queries_method=medium_queriesN5)



medium_queriesN6 = [
    "Describe the molting process in clawed marine crustaceans.",
    "How do clawed marine crustaceans adapt to their environment?",
    "What are the main threats to clawed marine crustacean populations?",
    "Describe the historical significance of the Tower of London.",
    "How does the economy of the capital city of the United Kingdom contribute to the UK's GDP?",
    "What are the main cultural activities in the capital city of the United Kingdom?",
    "Describe the main structural components of a harp-like instrument with strings.",
    "How did the harp-like instrument contribute to ancient Greek culture?",
    "What are the differences between a harp-like instrument and a harp?",
    "Describe the main grammatical features of the official tongue of Malaysia.",
    "How has the official tongue of Malaysia evolved over time?",
    "What are the differences between the official tongue of Malaysia and Indonesian?",
    "Describe the main themes in the artwork of the Renaissance artist known for 'David'.",
    "How did the Renaissance artist known for 'David' contribute to the Renaissance?",
    "What were the main techniques used by the Renaissance artist known for 'David' in his sculptures?",
    "Describe the domestic policies of the 13th President's administration.",
    "How did the 13th President handle the issue of slavery during his presidency?",
    "What were the key challenges faced by the 13th President during his presidency?",
    "Describe the main themes in the artwork of the American illustrator known for 'The Saturday Evening Post'.",
    "How did the work of the American illustrator known for 'The Saturday Evening Post' reflect American society?",
    "What were the main techniques used by the American illustrator known for 'The Saturday Evening Post' in his paintings?",
    "Describe the intelligence of soft-bodied marine animals with eight limbs.",
    "How do soft-bodied marine animals with eight limbs adapt to their environment?",
    "What are the main threats to soft-bodied marine animal populations?",
    "Describe the social behavior of semi-aquatic mammals with webbed feet.",
    "How do semi-aquatic mammals with webbed feet adapt to their environment?",
    "What are the main threats to semi-aquatic mammal populations?"
]

FinalEvalR_mediumN6, FinalEvalC_mediumN6, FinalEvalF_mediumN6, FinalResponseLength_mediumN6 = ResponseEvaluationMetrics(Queries_method=medium_queriesN6)



medium_queriesN7 = [
    "Describe the main themes in the artwork of the artist known for 'Guernica'.",
    "How did the work of the artist known for 'Guernica' influence modern art?",
    "What were the main periods of the artistic career of the artist known for 'Guernica'?",
    "Describe the breeding behavior of flightless seabirds.",
    "How do flightless seabirds adapt to their environment?",
    "What are the main threats to flightless seabird populations?",
    "Describe the mechanism of sound production in a keyboard instrument.",
    "How has the design of the keyboard instrument evolved over time?",
    "What are the main differences between a grand keyboard instrument and an upright keyboard instrument?",
    "Describe the main themes in the artwork of the Impressionist artist known for 'Luncheon of the Boating Party'.",
    "How did the work of the Impressionist artist known for 'Luncheon of the Boating Party' influence the Impressionist movement?",
    "What were the main techniques used by the Impressionist artist known for 'Luncheon of the Boating Party' in his paintings?",
    "Describe the main themes in the artwork of the De Stijl artist known for 'Composition with Red, Blue, and Yellow'.",
    "How did the work of the De Stijl artist known for 'Composition with Red, Blue, and Yellow' influence modern art?",
    "What were the main techniques used by the De Stijl artist known for 'Composition with Red, Blue, and Yellow' in his paintings?",
    "Describe the social behavior of large white bears found in the Arctic.",
    "How do large white bears found in the Arctic adapt to their environment?",
    "What are the main threats to the populations of large white bears found in the Arctic?",
    "Describe the main grammatical features of the official tongue of Portugal.",
    "How has the official tongue of Portugal evolved over time?",
    "What are the differences between European Portuguese and Brazilian Portuguese?",
    "Describe the historical significance of the independence of the Middle Eastern country known for its natural gas reserves.",
    "How does the economy of the Middle Eastern country known for its natural gas reserves compare to other Gulf countries?",
    "What are the main cultural activities in the Middle Eastern country known for its natural gas reserves?",
    "Describe the historical significance of the independence of the Southeast Asian city-state known for its port.",
    "How does the economy of the Southeast Asian city-state known for its port compare to other Southeast Asian countries?",
    "What are the main cultural activities in the Southeast Asian city-state known for its port?"
]


FinalEvalR_mediumN7, FinalEvalC_mediumN7, FinalEvalF_mediumN7, FinalResponseLength_mediumN7 = ResponseEvaluationMetrics(Queries_method=medium_queriesN7)



medium_queriesN8 = [
    "Describe the main grammatical features of the official tongue of Spain.",
    "How has the official tongue of Spain evolved over time?",
    "What are the differences between the Castilian dialect and the Latin American dialect of the official tongue of Spain?",
    "Describe the main grammatical features of the lingua franca of East Africa.",
    "How has the lingua franca of East Africa evolved over time?",
    "What are the differences between the lingua franca of East Africa and other Bantu languages?",
    "Describe the social behavior of large waterfowl with long necks.",
    "How do large waterfowl with long necks adapt to their environment?",
    "What are the main threats to large waterfowl with long neck populations?",
    "Describe the main grammatical features of the native tongue of Sweden.",
    "How has the native tongue of Sweden evolved over time?",
    "What are the differences between the native tongue of Sweden and Danish?",
    "Describe the domestic policies of the 26th President's administration.",
    "How did the 26th President handle the issue of conservation during his presidency?",
    "What were the key challenges faced by the 26th President during his presidency?",
    "Describe the social behavior of large striped felines.",
    "How do large striped felines adapt to their environment?",
    "What are the main threats to large striped feline populations?",
    "Describe the mechanism of sound production in a brass wind instrument.",
    "How has the design of the brass wind instrument evolved over time?",
    "What are the main differences between a brass wind instrument and a cornet?",
    "Describe the main grammatical features of the official tongue of Turkey.",
    "How has the official tongue of Turkey evolved over time?",
    "What are the differences between the official tongue of Turkey and Azerbaijani?",
    "Describe the habitat requirements of hard-shelled reptiles.",
    "How do hard-shelled reptiles adapt to their environment?",
    "What are the main threats to hard-shelled reptile populations?"
]


FinalEvalR_mediumN8, FinalEvalC_mediumN8, FinalEvalF_mediumN8, FinalResponseLength_mediumN8 = ResponseEvaluationMetrics(Queries_method=medium_queriesN8)



medium_queriesN9 = [
    "Describe the domestic policies of the 18th President's administration.",
    "How did the 18th President handle the issue of Reconstruction during his presidency?",
    "What were the key challenges faced by the 18th President during his presidency?",
    "Describe the historical significance of the independence of the South American country known for its beef industry.",
    "How does the economy of the South American country known for its beef industry compare to other South American countries?",
    "What are the main cultural activities in the South American country known for its beef industry?",
    "Describe the main grammatical features of the official tongue of Vietnam.",
    "How has the official tongue of Vietnam evolved over time?",
    "What are the differences between Northern and Southern dialects of the official tongue of Vietnam?",
    "Describe the main themes in the artwork of the post-impressionist artist known for 'Starry Night'.",
    "How did the work of the post-impressionist artist known for 'Starry Night' influence modern art?",
    "What were the main periods of the artistic career of the post-impressionist artist known for 'Starry Night'?",
    "Describe the mechanism of sound production in a bowed string instrument.",
    "How has the design of the bowed string instrument evolved over time?",
    "What are the main differences between a bowed string instrument and a viola?",
    "Describe the domestic policies of the 28th President's administration.",
    "How did the 28th President handle the issue of World War I during his presidency?",
    "What were the key challenges faced by the 28th President during his presidency?",
    "Describe the mechanism of sound production in a percussion instrument with wooden bars.",
    "How has the design of the percussion instrument with wooden bars evolved over time?",
    "What are the main differences between a percussion instrument with wooden bars and a marimba?",
    "Describe the social behavior of striped African equids.",
    "How do striped African equids adapt to their environment?",
    "What are the main threats to striped African equid populations?"
]

FinalEvalR_mediumN9, FinalEvalC_mediumN9, FinalEvalF_mediumN9, FinalResponseLength_mediumN9 = ResponseEvaluationMetrics(Queries_method=medium_queriesN9)



FinalR_mediumN=pd.concat([FinalEvalR_mediumN1,FinalEvalR_mediumN11, FinalEvalR_mediumN2, FinalEvalR_mediumN3, FinalEvalR_mediumN4, FinalEvalR_mediumN5, FinalEvalR_mediumN6, FinalEvalR_mediumN7,
                        FinalEvalR_mediumN8, FinalEvalR_mediumN9], axis=0)
FinalC_mediumN=pd.concat([FinalEvalC_mediumN1,FinalEvalC_mediumN11, FinalEvalC_mediumN2, FinalEvalC_mediumN3, FinalEvalC_mediumN4, FinalEvalC_mediumN5, FinalEvalC_mediumN6, FinalEvalC_mediumN7,
                        FinalEvalC_mediumN8, FinalEvalC_mediumN9], axis=0)
FinalF_mediumN=pd.concat([FinalEvalF_mediumN1, FinalEvalF_mediumN11, FinalEvalF_mediumN2, FinalEvalF_mediumN3, FinalEvalF_mediumN4, FinalEvalF_mediumN5, FinalEvalF_mediumN6, FinalEvalF_mediumN7,
                        FinalEvalF_mediumN8, FinalEvalF_mediumN9], axis=0)
FinalL_mediumN=pd.concat([FinalResponseLength_mediumN1,FinalResponseLength_mediumN11, FinalResponseLength_mediumN2, FinalResponseLength_mediumN3, FinalResponseLength_mediumN4, FinalResponseLength_mediumN5,
                        FinalResponseLength_mediumN6, FinalResponseLength_mediumN7, FinalResponseLength_mediumN8, FinalResponseLength_mediumN9], axis=0)

In [ ]:

hard_queriesN1 = [
    "Evaluate the influence of the 16th President on Union military tactics during the Civil War.",
    "Contrast the political beliefs of the 16th President with those of his peers.",
    "Discuss the legal challenges faced by the 16th President during his term.",
    "Detail the various ecological functions performed by Formicidae.",
    "Examine the evolutionary traits that make Formicidae successful.",
    "Compare the reproductive methods of different Formicidae species.",
    "Analyze the structure of verbs in this Semitic language.",
    "What strategies are involved in teaching this Semitic language as a foreign language?",
    "Contrast the various dialects of this Semitic language spoken in different regions.",
    "Evaluate the impact of Colony Collapse Disorder on agriculture.",
    "How do the behaviors and social structures of Apis mellifera and Bombus differ?",
    "Compare the effectiveness of Apis mellifera as pollinators to other insects.",
    "Analyze the evolutionary traits contributing to the diversity of Coleoptera.",
    "Discuss the ecological impact of Coleoptera on farming.",
    "Compare the reproductive strategies of various Coleoptera species.",
    "Evaluate the political significance of the capital of the People's Republic.",
    "What cultural importance does traditional architecture hold in the capital of the People's Republic?",
    "Compare the urban development of the capital of the People's Republic to other major global cities.",
    "Evaluate the effects of habitat loss on Lepidoptera populations.",
    "How do the lifecycles and behaviors of Lepidoptera and nocturnal Lepidoptera differ?",
    "Compare the migration patterns of different Lepidoptera species."
]


FinalEvalR_hardN1, FinalEvalC_hardN1, FinalEvalF_hardN1, FinalResponseLength_hardN1 = ResponseEvaluationMetrics(Queries_method=hard_queriesN1)



hard_queriesN11 = [
    "Analyze the economic impact of the 30th President's tax policies.",
    "Compare the presidency of the 30th President with that of his predecessor, Warren G. Harding.",
    "Discuss the long-term effects of the 30th President's administration on the Republican Party.",
    "Analyze the impact of multiculturalism policies in the country north of the United States on its society.",
    "Discuss the role of the country north of the United States in international peacekeeping missions.",
    "Compare and contrast the political systems of the country north of the United States and the United Kingdom.",
    "Analyze the development of technique for the bowed string instrument with four strings from the Baroque to the modern era.",
    "Discuss the contributions of Pablo Casals to the performance of the bowed string instrument with four strings.",
    "Compare the use of the bowed string instrument with four strings in classical and contemporary music.",
    "Analyze the linguistic structure of grammar in the language spoken by the majority in China.",
    "Discuss the challenges of translating idioms from the language spoken by the majority in China into English.",
    "Compare the phonological systems of the language spoken by the majority in China and Cantonese.",
    "Analyze the role of large felines found in the Americas in their ecosystems.",
    "Discuss the conservation strategies employed to protect large felines found in the Americas.",
    "Compare the behavior of large felines found in the Americas with that of other big cats like lions and tigers.",
    "Analyze the evolution of manufacturing techniques for percussion instruments made of metal.",
    "Discuss the cultural significance of percussion instruments made of metal in different musical traditions.",
    "Compare the sound characteristics of different alloys used in percussion instruments made of metal.",
    "Analyze the urban challenges faced by the 'City of Mosques' due to rapid population growth.",
    "Discuss the impact of climate change on the infrastructure of the 'City of Mosques'.",
    "Compare the development of the 'City of Mosques' with that of other major South Asian cities.",
    "Analyze the evolutionary adaptations of insects with two pairs of large wings for flight.",
    "Discuss the cultural significance of insects with two pairs of large wings in different societies.",
    "Compare the hunting strategies of insects with two pairs of large wings with those of other aerial predators."
]


FinalEvalR_hardN11, FinalEvalC_hardN11, FinalEvalF_hardN11, FinalResponseLength_hardN11 = ResponseEvaluationMetrics(Queries_method=hard_queriesN11)



hard_queriesN2 = [
    "Analyze the migratory patterns of waterfowl in the Northern Hemisphere.",
    "Discuss the economic significance of waterfowl in agriculture.",
    "Compare and contrast the plumage differences between male and female waterfowl.",
    "Examine the political developments in the African country known for pyramids over the last hundred years.",
    "Discuss the impact of tourism on the economy of the African country known for pyramids.",
    "Compare the cultural differences between urban and rural areas in the African country known for pyramids.",
    "Analyze the use of color in the artworks of El Greco.",
    "Discuss the importance of El Greco's contributions to Renaissance art.",
    "Compare the artistic style of El Greco with that of his contemporaries.",
    "Evaluate the role of the largest land mammals in their ecosystems.",
    "Discuss the efforts to conserve the largest land mammals.",
    "Compare the behaviors of the largest land mammals in Africa and Asia.",
    "Assess the impact of the global lingua franca on linguistic diversity worldwide.",
    "Discuss the difficulties in teaching the global lingua franca as a second language.",
    "Compare the phonological differences between British and American forms of the global lingua franca.",
    "Evaluate the impact of the welfare state on the economy of the Nordic country.",
    "Discuss the role of technology and innovation in the development of the Nordic country.",
    "Compare the cultural practices of the indigenous Sami people with those of other communities in the Nordic country.",
    "Analyze the grammatical structure of verbs in the Uralic tongue of the Nordic country.",
    "Discuss the challenges of translating literature from the Uralic tongue of the Nordic country into the global lingua franca.",
    "Compare the dialects of the Uralic tongue spoken in different regions of the Nordic country.",
    "Examine the acoustical properties of the instrument played by blowing across a hole.",
    "Discuss the role of the instrument played by blowing across a hole in various musical traditions.",
    "Compare the playing techniques of the Western concert variety and the Indian bamboo variety of the instrument played by blowing across a hole.",
    "Evaluate the role of small, wild canines in folklore and culture.",
    "Discuss the evolutionary adaptations that enable small, wild canines to hunt effectively.",
    "Compare the behaviors of different species of small, wild canines in their natural habitats."
]


FinalEvalR_hardN2, FinalEvalC_hardN2, FinalEvalF_hardN2, FinalResponseLength_hardN2 = ResponseEvaluationMetrics(Queries_method=hard_queriesN2)


hard_queriesN3 = [
    "Analyze the impact of the official tongue of France on other languages worldwide.",
    "Discuss the challenges of learning the official tongue of France for non-native speakers.",
    "Compare the phonological systems of the official tongues of France and Italy.",
    "Analyze the impact of the 38th President's economic policies.",
    "Discuss the significance of the 38th President's pardon of his predecessor.",
    "Compare the 38th President's tenure with that of his successor, the 39th President.",
    "Analyze the historical development of the official tongue of Germany.",
    "Discuss the challenges of learning the official tongue of Germany for English speakers.",
    "Compare the phonological systems of the official tongues of Germany and the Netherlands.",
    "Analyze the impact of the colonial history of the West African nation known for gold on its modern society.",
    "Discuss the role of the West African nation known for gold in African politics.",
    "Compare the cultural practices of different ethnic groups in the West African nation known for gold.",
    "Analyze the conservation efforts to protect the black-and-white bear species.",
    "Discuss the impact of climate change on the habitats of the black-and-white bear species.",
    "Compare the behaviors of the black-and-white bear species in the wild and in captivity.",
    "Analyze the evolutionary adaptations of the tallest land animals for their height.",
    "Discuss the role of the tallest land animals in their ecosystems.",
    "Compare the different subspecies of the tallest land animals and their geographic distributions.",
    "Analyze the role of the largest wild canids in their ecosystems.",
    "Discuss the conservation efforts to protect the largest wild canids.",
    "Compare the behaviors of the largest wild canids in different regions.",
    "Analyze the impact of the 22nd and 24th President's economic policies.",
    "Discuss the significance of the 22nd and 24th President's vetoes.",
    "Compare the 22nd and 24th President's tenure with that of his predecessors.",
    "Analyze the role of the stringed instrument played by strumming in different musical genres.",
    "Discuss the impact of technology on the modern stringed instrument played by strumming.",
    "Compare the playing styles of different famous musicians known for playing the stringed instrument."
]


FinalEvalR_hardN3, FinalEvalC_hardN3, FinalEvalF_hardN3, FinalResponseLength_hardN3 = ResponseEvaluationMetrics(Queries_method=hard_queriesN3)



hard_queriesN3 = [
    "Analyze the impact of the official tongue of France on other languages worldwide.",
    "Discuss the challenges of learning the official tongue of France for non-native speakers.",
    "Compare the phonological systems of the official tongues of France and Italy.",
    "Analyze the impact of the 38th President's economic policies.",
    "Discuss the significance of the 38th President's pardon of his predecessor.",
    "Compare the 38th President's tenure with that of his successor, the 39th President.",
    "Analyze the historical development of the official tongue of Germany.",
    "Discuss the challenges of learning the official tongue of Germany for English speakers.",
    "Compare the phonological systems of the official tongues of Germany and the Netherlands.",
    "Analyze the impact of the colonial history of the West African nation known for gold on its modern society.",
    "Discuss the role of the West African nation known for gold in African politics.",
    "Compare the cultural practices of different ethnic groups in the West African nation known for gold.",
    "Analyze the conservation efforts to protect the black-and-white bear species.",
    "Discuss the impact of climate change on the habitats of the black-and-white bear species.",
    "Compare the behaviors of the black-and-white bear species in the wild and in captivity.",
    "Analyze the evolutionary adaptations of the tallest land animals for their height.",
    "Discuss the role of the tallest land animals in their ecosystems.",
    "Compare the different subspecies of the tallest land animals and their geographic distributions.",
    "Analyze the role of the largest wild canids in their ecosystems.",
    "Discuss the conservation efforts to protect the largest wild canids.",
    "Compare the behaviors of the largest wild canids in different regions.",
    "Analyze the impact of the 22nd and 24th President's economic policies.",
    "Discuss the significance of the 22nd and 24th President's vetoes.",
    "Compare the 22nd and 24th President's tenure with that of his predecessors.",
    "Analyze the role of the stringed instrument played by strumming in different musical genres.",
    "Discuss the impact of technology on the modern stringed instrument played by strumming.",
    "Compare the playing styles of different famous musicians known for playing the stringed instrument."
]


FinalEvalR_hardN3, FinalEvalC_hardN3, FinalEvalF_hardN3, FinalResponseLength_hardN3 = ResponseEvaluationMetrics(Queries_method=hard_queriesN3)





hard_queriesN4 = [
    "Analyze the impact of the work of the artist known for 'The Kiss' on modern art.",
    "Discuss the role of eroticism in the paintings of the artist known for 'The Kiss'.",
    "Compare the techniques used by the artist known for 'The Kiss' in his portraits and landscapes.",
    "Analyze the political changes in the Southeast Asian archipelago nation since its independence.",
    "Discuss the impact of tourism on the economy of the Southeast Asian archipelago nation.",
    "Compare the cultural practices of different ethnic groups in the Southeast Asian archipelago nation.",
    "Analyze the impact of the work of the scientist known for gravity on the Scientific Revolution.",
    "Discuss the challenges faced by the scientist known for gravity in his scientific career.",
    "Compare the scientific contributions of the scientist known for gravity and Galileo.",
    "Analyze the impact of the work of the artist known for his 'drip technique' on modern art.",
    "Discuss the role of automatism in the paintings of the artist known for his 'drip technique'.",
    "Compare the techniques of the artist known for his 'drip technique' with those of other abstract expressionists.",
    "Analyze the impact of the doctrine named after the 5th President of the United States on American foreign policy.",
    "Discuss the significance of the Missouri Compromise during the tenure of the 5th President.",
    "Compare the presidency of the 5th President with that of his predecessor, James Madison.",
    "Analyze the impact of the work of the inventor known for improving the steam engine on modern engineering.",
    "Discuss the collaboration between the inventor known for improving the steam engine and Matthew Boulton.",
    "Compare the steam engines of the inventor known for improving the steam engine and Thomas Newcomen.",
    "Analyze the influence of Chinese on the writing system of the primary language of Japan.",
    "Discuss the challenges of translating literature from the primary language of Japan into English.",
    "Compare the phonological systems of the primary languages of Japan and Korea.",
    "Analyze the impact of the Alien and Sedition Acts on the presidency of the 2nd President.",
    "Discuss the significance of the 2nd President's contributions to American independence.",
    "Compare the presidency of the 2nd President with that of his successor, Thomas Jefferson."
]


FinalEvalR_hardN4, FinalEvalC_hardN4, FinalEvalF_hardN4, FinalResponseLength_hardN4 = ResponseEvaluationMetrics(Queries_method=hard_queriesN4)



hard_queriesN5 = [
    "Analyze the role of the hopping marsupials in Australian culture.",
    "Discuss the impact of habitat loss on the hopping marsupial populations.",
    "Compare the different species of the hopping marsupials and their geographic distributions.",
    "Analyze the conservation efforts to protect the tree-dwelling marsupials.",
    "Discuss the impact of climate change on the habitats of the tree-dwelling marsupials.",
    "Compare the behaviors of the tree-dwelling marsupials in the wild and in captivity.",
    "Analyze the influence of Chinese on the writing system of the primary language of South Korea.",
    "Discuss the challenges of translating literature from the primary language of South Korea into English.",
    "Compare the phonological systems of the primary languages of South Korea and Japan.",
    "Analyze the impact of the work of the Renaissance artist known for 'The Last Supper' on the Renaissance.",
    "Discuss the challenges faced by the Renaissance artist known for 'The Last Supper' in his scientific career.",
    "Compare the artistic techniques of the Renaissance artist known for 'The Last Supper' and Michelangelo.",
    "Analyze the role of the large spotted cats in their ecosystems.",
    "Discuss the conservation efforts to protect the large spotted cats.",
    "Compare the behaviors of the large spotted cats in different regions.",
    "Analyze the impact of the banking sector on the economy of the small European principality.",
    "Discuss the role of the small European principality in international relations.",
    "Compare the cultural practices of the small European principality with those of neighboring countries.",
    "Analyze the urban development of the capital of the South American country known for Machu Picchu over the past century.",
    "Discuss the impact of tourism on the economy of the capital of the South American country known for Machu Picchu.",
    "Compare the social issues in the capital of the South American country known for Machu Picchu with those in other major South American cities.",
    "Analyze the role of the large felines known as the 'king of the jungle' in their ecosystems.",
    "Discuss the conservation efforts to protect the large felines known as the 'king of the jungle'.",
    "Compare the behaviors of the large felines known as the 'king of the jungle' in different regions.",
    "Analyze the evolutionary adaptations of small, scaly reptiles.",
    "Discuss the role of small, scaly reptiles in their ecosystems.",
    "Compare the behaviors of different species of small, scaly reptiles in their natural habitats."
]

FinalEvalR_hardN5, FinalEvalC_hardN5, FinalEvalF_hardN5, FinalResponseLength_hardN5 = ResponseEvaluationMetrics(Queries_method=hard_queriesN5)



hard_queriesN6 = [
    "Analyze the impact of fishing clawed marine crustaceans on marine ecosystems.",
    "Discuss the economic importance of clawed marine crustaceans in global seafood markets.",
    "Compare the anatomical differences between clawed marine crustaceans and spiny marine crustaceans.",
    "Analyze the impact of the Industrial Revolution on the development of the capital city of the United Kingdom.",
    "Discuss the role of the capital city of the United Kingdom in global finance.",
    "Compare the social issues in the capital city of the United Kingdom with those in other major European cities.",
    "Analyze the evolution of the harp-like instrument in different cultures.",
    "Discuss the role of the harp-like instrument in ancient rituals and ceremonies.",
    "Compare the playing techniques of the harp-like instrument and the lute.",
    "Analyze the influence of Arabic on the official tongue of Malaysia.",
    "Discuss the challenges of translating literature from the official tongue of Malaysia into English.",
    "Compare the phonological systems of the official tongue of Malaysia and Javanese.",
    "Analyze the impact of the work of the Renaissance artist known for 'David' on art history.",
    "Discuss the challenges faced by the Renaissance artist known for 'David' during his career.",
    "Compare the artistic techniques of the Renaissance artist known for 'David' and Leonardo da Vinci.",
    "Analyze the impact of the foreign policies of the 13th President.",
    "Discuss the significance of the Compromise of 1850 during the 13th President's tenure.",
    "Compare the presidency of the 13th President with that of his predecessor, Zachary Taylor.",
    "Analyze the impact of the work of the American illustrator known for 'The Saturday Evening Post' on American culture.",
    "Discuss the challenges faced by the American illustrator known for 'The Saturday Evening Post' during his career.",
    "Compare the artistic styles of the American illustrator known for 'The Saturday Evening Post' and other American illustrators.",
    "Analyze the role of soft-bodied marine animals with eight limbs in marine ecosystems.",
    "Discuss the significance of the camouflage abilities of soft-bodied marine animals with eight limbs.",
    "Compare the behaviors of different species of soft-bodied marine animals with eight limbs in their natural habitats.",
    "Analyze the role of semi-aquatic mammals with webbed feet in their ecosystems.",
    "Discuss the conservation efforts to protect semi-aquatic mammals with webbed feet.",
    "Compare the behaviors of different species of semi-aquatic mammals with webbed feet in their natural habitats."
]


FinalEvalR_hardN6, FinalEvalC_hardN6, FinalEvalF_hardN6, FinalResponseLength_hardN6 = ResponseEvaluationMetrics(Queries_method=hard_queriesN6)


hard_queriesN7 = [
    "Analyze the impact of Cubism on the work of the artist known for 'Guernica'.",
    "Discuss the significance of 'Guernica' by the artist known for it.",
    "Compare the techniques used by the artist known for 'Guernica' and Georges Braque in their Cubist works.",
    "Analyze the role of flightless seabirds in their ecosystems.",
    "Discuss the impact of climate change on the habitats of flightless seabirds.",
    "Compare the behaviors of different species of flightless seabirds in their natural habitats.",
    "Analyze the role of the keyboard instrument in classical music.",
    "Discuss the impact of keyboard instrument technology on modern music.",
    "Compare the playing techniques of different famous keyboard instrument players.",
    "Analyze the impact of the work of the Impressionist artist known for 'Luncheon of the Boating Party' on modern art.",
    "Discuss the challenges faced by the Impressionist artist known for 'Luncheon of the Boating Party' during his career.",
    "Compare the artistic techniques of the Impressionist artist known for 'Luncheon of the Boating Party' and Claude Monet.",
    "Analyze the impact of the work of the De Stijl artist known for 'Composition with Red, Blue, and Yellow' on abstract art.",
    "Discuss the significance of 'Composition with Red, Blue, and Yellow' by the De Stijl artist known for it.",
    "Compare the artistic techniques of the De Stijl artist known for 'Composition with Red, Blue, and Yellow' and Theo van Doesburg.",
    "Analyze the role of large white bears found in the Arctic in their ecosystems.",
    "Discuss the impact of climate change on the habitats of large white bears found in the Arctic.",
    "Compare the behaviors of large white bears found in the Arctic in different regions.",
    "Analyze the influence of Latin on the official tongue of Portugal.",
    "Discuss the challenges of translating literature from the official tongue of Portugal into English.",
    "Compare the phonological systems of the official tongue of Portugal and Spanish.",
    "Analyze the impact of the natural gas reserves on the economy of the Middle Eastern country known for them.",
    "Discuss the role of the Middle Eastern country known for its natural gas reserves in international politics.",
    "Compare the social development of the Middle Eastern country known for its natural gas reserves with that of neighboring countries.",
    "Analyze the impact of the port on the economy of the Southeast Asian city-state known for it.",
    "Discuss the role of the Southeast Asian city-state known for its port in international trade.",
    "Compare the social development of the Southeast Asian city-state known for its port with that of neighboring countries."
]

FinalEvalR_hardN7, FinalEvalC_hardN7, FinalEvalF_hardN7, FinalResponseLength_hardN7 = ResponseEvaluationMetrics(Queries_method=hard_queriesN7)



hard_queriesN8 = [
    "Analyze the influence of Arabic on the official tongue of Spain.",
    "Discuss the challenges of translating literature from the official tongue of Spain into English.",
    "Compare the phonological systems of the official tongue of Spain and Portuguese.",
    "Analyze the influence of Arabic on the lingua franca of East Africa.",
    "Discuss the challenges of translating literature from the lingua franca of East Africa into English.",
    "Compare the phonological systems of the lingua franca of East Africa and Somali.",
    "Analyze the role of large waterfowl with long necks in their ecosystems.",
    "Discuss the significance of large waterfowl with long necks in mythology and culture.",
    "Compare the behaviors of different species of large waterfowl with long necks in their natural habitats.",
    "Analyze the influence of German on the native tongue of Sweden.",
    "Discuss the challenges of translating literature from the native tongue of Sweden into English.",
    "Compare the phonological systems of the native tongue of Sweden and Norwegian.",
    "Analyze the impact of the foreign policies of the 26th President.",
    "Discuss the significance of the Panama Canal during the tenure of the 26th President.",
    "Compare the presidency of the 26th President with that of his successor, William Howard Taft.",
    "Analyze the role of large striped felines in their ecosystems.",
    "Discuss the impact of poaching on large striped feline populations.",
    "Compare the behaviors of different subspecies of large striped felines in their natural habitats.",
    "Analyze the role of the brass wind instrument in classical music.",
    "Discuss the impact of brass wind instrument technology on modern music.",
    "Compare the playing techniques of different famous brass wind instrument players.",
    "Analyze the influence of Arabic on the official tongue of Turkey.",
    "Discuss the challenges of translating literature from the official tongue of Turkey into English.",
    "Compare the phonological systems of the official tongue of Turkey and Persian.",
    "Analyze the evolutionary adaptations of hard-shelled reptiles.",
    "Discuss the role of hard-shelled reptiles in their ecosystems.",
    "Compare the behaviors of different species of hard-shelled reptiles in their natural habitats."
]


FinalEvalR_hardN8, FinalEvalC_hardN8, FinalEvalF_hardN8, FinalResponseLength_hardN8 = ResponseEvaluationMetrics(Queries_method=hard_queriesN8)



hard_queriesN9 = [
    "Analyze the impact of the foreign policies of the 18th President.",
    "Discuss the significance of the Civil Rights Act of 1875 during the tenure of the 18th President.",
    "Compare the presidency of the 18th President with that of his predecessor, Andrew Johnson.",
    "Analyze the impact of agriculture on the economy of the South American country known for its beef industry.",
    "Discuss the role of the South American country known for its beef industry in international politics.",
    "Compare the social development of the South American country known for its beef industry with that of neighboring countries.",
    "Analyze the influence of Chinese on the official tongue of Vietnam.",
    "Discuss the challenges of translating literature from the official tongue of Vietnam into English.",
    "Compare the phonological systems of the official tongue of Vietnam and Thai.",
    "Analyze the impact of the mental health of the post-impressionist artist known for 'Starry Night' on his work.",
    "Discuss the significance of 'Starry Night' by the post-impressionist artist known for it.",
    "Compare the techniques used by the post-impressionist artist known for 'Starry Night' and Paul Gauguin.",
    "Analyze the role of the bowed string instrument in classical music.",
    "Discuss the impact of bowed string instrument technology on modern music.",
    "Compare the playing techniques of different famous players of the bowed string instrument.",
    "Analyze the impact of the foreign policies of the 28th President.",
    "Discuss the significance of the League of Nations during the tenure of the 28th President.",
    "Compare the presidency of the 28th President with that of his predecessor, William Howard Taft.",
    "Analyze the role of the percussion instrument with wooden bars in orchestral music.",
    "Discuss the impact of percussion instrument technology on modern music.",
    "Compare the playing techniques of different famous players of the percussion instrument with wooden bars.",
    "Analyze the role of striped African equids in their ecosystems.",
    "Discuss the impact of habitat loss on striped African equid populations.",
    "Compare the behaviors of different species of striped African equids in their natural habitats."
]


FinalEvalR_hardN9, FinalEvalC_hardN9, FinalEvalF_hardN9, FinalResponseLength_hardN9 = ResponseEvaluationMetrics(Queries_method=hard_queriesN9)


FinalR_hardN=pd.concat([FinalEvalR_hardN1, FinalEvalR_hardN11, FinalEvalR_hardN2, FinalEvalR_hardN3, FinalEvalR_hardN4, FinalEvalR_hardN5, FinalEvalR_hardN6, FinalEvalR_hardN7,
                        FinalEvalR_hardN8, FinalEvalR_hardN9], axis=0)
FinalC_hardN=pd.concat([FinalEvalC_hardN1, FinalEvalC_hardN11, FinalEvalC_hardN2, FinalEvalC_hardN3, FinalEvalC_hardN4, FinalEvalC_hardN5, FinalEvalC_hardN6, FinalEvalC_hardN7,
                        FinalEvalC_hardN8, FinalEvalC_hardN9], axis=0)
FinalF_hardN=pd.concat([FinalEvalF_hardN1,FinalEvalF_hardN11, FinalEvalF_hardN2, FinalEvalF_hardN3, FinalEvalF_hardN4, FinalEvalF_hardN5, FinalEvalF_hardN6, FinalEvalF_hardN7,
                        FinalEvalF_hardN8, FinalEvalF_hardN9], axis=0)
FinalL_hardN=pd.concat([FinalResponseLength_hardN1, FinalResponseLength_hardN11, FinalResponseLength_hardN2, FinalResponseLength_hardN3, FinalResponseLength_hardN4, FinalResponseLength_hardN5,
                        FinalResponseLength_hardN6, FinalResponseLength_hardN7, FinalResponseLength_hardN8, FinalResponseLength_hardN9], axis=0)
#print(FinalEvalR_easyN.mean())


## More Recognized

In [ ]:
easy_queriesNN1 = [
    "In which exact year was Abraham Lincoln, the 16th President of the United States who led the country through the Civil War and issued the Emancipation Proclamation, born, considering his early life in Kentucky and his rise in Illinois politics?",
    "Who served as Abraham Lincoln's vice president during his first term as President of the United States, before the American Civil War ended and before Andrew Johnson took over during the Reconstruction era?",
    "Which famous speech, known for its opening line 'Four score and seven years ago,' did Abraham Lincoln deliver during the dedication of the Soldiers' National Cemetery in Gettysburg, Pennsylvania, which redefined the principles of liberty and equality in the United States?",
    "What are the basic anatomical features and physical characteristics of an ant, an insect that is part of the Formicidae family, known for its eusocial behavior, division of labor within colonies, and ability to lift objects many times their own body weight?",
    "Globally, how many recognized species of ants are there, including variations such as carpenter ants, fire ants, and leafcutter ants, and how do these species vary in terms of their ecological roles, habitats, and social structures?",
    "What is the primary diet of ants, considering the diverse species such as omnivorous ants, herbivorous ants, and predatory ants, and the varying food sources they rely on in different environments, including their role as scavengers and decomposers?",
    "What is the official script used to write the Arabic language, a Semitic language that is the liturgical language of Islam, has influenced many other languages across the Middle East and North Africa, and is written in the Arabic alphabet, which includes 28 letters?",
    "Name three countries where Arabic, a Semitic language with a rich literary and cultural history, is the official language, and discuss the dialectical variations in these countries, such as the differences between Egyptian Arabic, Levantine Arabic, and Gulf Arabic.",
    "What is the basic word order in an Arabic sentence, considering the typical subject-verb-object structure, the use of inflection in the Arabic language, and how different dialects might alter this structure in spoken Arabic across various regions?",
    "What is the primary role of bees in the ecosystem, particularly in terms of their contribution to pollination, the production of honey and beeswax, and their essential role in supporting agriculture by pollinating crops like fruits, vegetables, and nuts?",
    "Name the most well-known species of bees, such as the Western honeybee (Apis mellifera), and discuss their importance in both commercial pollination and honey production, as well as the social structure of their hives, which includes workers, drones, and a queen bee.",
    "How do bees, particularly honeybees, communicate with each other within the hive, including the use of the 'waggle dance' to convey information about the location of food sources, and the release of pheromones to coordinate activities within the colony?",
    "To which order of insects do beetles belong, considering their classification as Coleoptera, their distinctive hard exoskeletons (elytra), and the fact that they represent the largest order of insects with a vast diversity of species globally?",
    "How many species of beetles, the largest order of insects, are there worldwide, including well-known types such as ladybugs (ladybird beetles), weevils, and dung beetles, and what are some examples of their ecological diversity and roles in various ecosystems?",
    "What is the primary diet of a ladybird beetle, an insect known for its beneficial role in controlling aphid populations in gardens and agricultural fields, and how do these beetles contribute to natural pest control in ecosystems?",
    "What is the capital city of China, a country with a long history of dynastic rule, including the Ming and Qing dynasties, and a rapidly growing economy that has become a global powerhouse in the 21st century?",
    "Name one famous historical site in Beijing, the capital of China, such as the Forbidden City or the Great Wall of China, that reflects the city's imperial past, its significance in Chinese culture, and its status as a UNESCO World Heritage site.",
    "What is the current population of Beijing, China, and how has the city's population changed over the past few decades as a result of rapid urbanization, economic growth, and migration from rural areas to urban centers?",
    "What stage comes after the caterpillar in the lifecycle of a butterfly, a process that includes complete metamorphosis involving the stages of egg, larva (caterpillar), pupa (chrysalis), and adult butterfly, and how does this transformation occur?",
    "Name one common species of butterfly, such as the Monarch butterfly (Danaus plexippus), and describe its lifecycle, migratory patterns across continents, and the role it plays in pollination and as a symbol in various cultures.",
    "What do butterflies typically eat, considering their diet during both the caterpillar and adult stages, including their reliance on nectar from flowers as adults and various plant leaves as larvae, and how do they contribute to pollination?"
]


FinalEvalR_easyNN1, FinalEvalC_easyNN1, FinalEvalF_easyNN1, FinalResponseLength_easyNN1 = ResponseEvaluationMetrics(Queries_method=easy_queriesNN1)


easy_queriesNN2 = [
    "What was Calvin Coolidge's nickname during his presidency, reflecting his reputation for being a man of few words and his role in promoting small government and laissez-faire economics in the 1920s?",
    "During which years did Calvin Coolidge serve as President of the United States, following the death of Warren G. Harding, and how did his policies reflect the economic boom and social changes of the Roaring Twenties?",
    "Who served as Calvin Coolidge's vice president, helping to oversee domestic policies such as tax cuts, the reduction of government debt, and the promotion of business interests during Coolidge's administration?",
    "What is the capital city of Canada, a country known for its diverse culture, bilingualism in English and French, and its federal system of government, and how does this city serve as the political and cultural hub of the nation?",
    "Name three provinces in Canada, each known for its unique geography, culture, and economic contributions, and describe the differences in regional identities and their roles in the Canadian Confederation.",
    "What are the official languages of Canada, a country with a strong tradition of bilingualism, and how do these languages reflect the country's colonial history and ongoing efforts to promote cultural diversity and inclusivity?",
    "What is a cello, a stringed musical instrument known for its deep, rich tone, and its role in both orchestral and chamber music, and how does it compare in size and pitch to other instruments in the violin family?",
    "How many strings does a cello, an essential instrument in Western classical music, typically have, and what are the names of these strings, each of which is tuned a fifth apart to produce its distinctive range?",
    "Name one famous cellist, known for their contributions to the development of cello technique and repertoire, and discuss their influence on both classical and contemporary music through performances and recordings.",
    "What script is used to write Chinese, one of the oldest writing systems still in use, and how does this logographic script differ from alphabetic systems in terms of structure, usage, and historical development?",
    "How many tones are there in Mandarin Chinese, the official language of China, and how do these tones affect the meaning of words, distinguishing Mandarin from other Chinese dialects and influencing its pronunciation?",
    "Name one Chinese dialect besides Mandarin, such as Cantonese or Shanghainese, and discuss its regional significance, linguistic features, and how it reflects the cultural diversity within China's vast population.",
    "What is another name for a cougar, a large feline species known for its adaptability across various habitats in the Americas, and how does this name vary by region, reflecting the cougar's wide distribution?",
    "Where are cougars commonly found, considering their extensive range from Canada to South America, and how do they adapt to different environments, from forests and mountains to deserts and urban areas?",
    "What is the primary diet of a cougar, a solitary predator that plays a key role in its ecosystem, and how does its hunting behavior and prey selection vary across its range, influencing its interactions with other species?",
    "What is a cymbal, a percussion instrument that produces a distinctive sound when struck, and how is it used in different musical contexts, from orchestral performances to modern drum kits in various genres?",
    "Name three types of cymbals, such as crash, ride, and hi-hat, and describe the differences in their sound, construction, and usage in both traditional and contemporary musical settings.",
    "What materials are cymbals typically made from, and how do these materials, including alloys of copper, tin, and sometimes silver, affect the sound quality, durability, and tone produced by the cymbals?",
    "What is the capital of Bangladesh, a densely populated country in South Asia with a rich cultural history, and how does this city serve as the political, economic, and cultural center of the nation?",
    "What river is Dhaka, the capital of Bangladesh, located on, and how does this river influence the city's economy, transportation, and historical significance as a hub of trade and culture in the region?",
    "What is Dhaka known as the 'City of', reflecting its role in Bangladesh's history, culture, and economy, and how does this nickname capture the city's vibrant and dynamic character despite its challenges?",
    "What order do dragonflies belong to, an ancient group of insects known for their aerial agility and predatory behavior, and how does this classification reflect their evolutionary history and ecological role?",
    "How many pairs of wings do dragonflies have, and how do these wings contribute to their flight capabilities, allowing them to hover, change direction rapidly, and catch prey in mid-air with remarkable precision?",
    "What is the primary habitat of dragonflies, including their reliance on freshwater ecosystems like ponds, lakes, and rivers for breeding, and how do these habitats support their lifecycle and ecological interactions?"
]


FinalEvalR_easyNN2, FinalEvalC_easyNN2, FinalEvalF_easyNN2, FinalResponseLength_easyNN2 = ResponseEvaluationMetrics(Queries_method=easy_queriesNN2)



easy_queriesNN3 = [
    "What family do ducks belong to within the bird classification, and how does this family relate to other waterfowl like geese and swans in terms of habitat and behavior?",
    "What is the primary diet of ducks, and how does their feeding behavior vary between different species, particularly between surface feeders like dabbling ducks and deeper feeders like diving ducks?",
    "Name three types of ducks commonly found in both the Northern and Southern Hemispheres, and describe their distinguishing physical characteristics, habitats, and migratory patterns.",
    "What is the capital of Egypt, and how does its geographic location on the Nile River influence its role as a cultural, political, and economic hub in both historical and modern contexts?",
    "Name one major river in Egypt other than the Nile, and discuss its significance in ancient Egyptian civilization as well as its current role in the country’s agriculture and economy.",
    "What is Egypt's official language, and how has it been influenced by historical events such as the Arab conquest and subsequent cultural exchanges with neighboring regions?",
    "What was El Greco's real name, and how did his Greek heritage influence his artistic style, particularly in the context of the Spanish Renaissance?",
    "Where was El Greco born, and how did the cultural and artistic environment of his birthplace influence his later work, especially in his adoption of Mannerist techniques?",
    "Name one famous painting by El Greco, and discuss how this work exemplifies his unique style, characterized by elongated figures, dramatic use of color, and spiritual intensity.",
    "How many species of elephants are there, and what are the key differences in their physical characteristics, habitats, and social structures, particularly between African and Asian elephants?",
    "What is the primary diet of elephants, and how does their feeding behavior impact the ecosystems they inhabit, particularly in terms of vegetation management and seed dispersal?",
    "Name one physical characteristic unique to elephants that distinguishes them from other large mammals, and explain how this feature has evolved to support their survival in various habitats.",
    "What family of languages does English belong to within the Indo-European language family, and how does its relationship with other Germanic languages influence its vocabulary and grammar?",
    "Name two countries where English is an official language, and discuss how historical events like colonization and globalization have contributed to the spread of English around the world.",
    "What historical event significantly influenced the development of Middle English, particularly in terms of vocabulary and grammar, and how did this event shape the evolution of the English language?",
    "What is the capital of Finland, and how does its location in Northern Europe influence its role in international trade, politics, and cultural exchanges within the Nordic region?",
    "Name one official language of Finland, and discuss the historical and cultural factors that have contributed to its official status, including its relationship with other languages in the region.",
    "What is the population of Finland, and how does demographic trends like urbanization and migration impact the country’s economy, social structure, and cultural identity?",
    "What language family does Finnish belong to, and how does its linguistic structure differ from that of other major languages in Europe, particularly those in the Indo-European family?",
    "Name one country other than Finland where Finnish is spoken, and discuss the historical and cultural reasons for the presence of Finnish speakers in that country.",
    "How many cases does the Finnish language have, and how does this complex system of noun inflection affect sentence structure, meaning, and the overall learning process for non-native speakers?",
    "What family of musical instruments does the flute belong to, and how has its design evolved from ancient times to the modern Western concert flute used in classical and contemporary music?",
    "How is sound produced in a flute, and what role does the player's embouchure, air stream, and finger positioning play in controlling pitch, volume, and tone quality?",
    "Name one type of flute other than the Western concert flute, and describe its unique characteristics, historical origins, and the musical traditions in which it is commonly used.",
    "How many species of foxes are there globally, and how do their physical and behavioral adaptations allow them to thrive in diverse environments ranging from arctic tundras to urban areas?",
    "What is the primary habitat of the red fox, and how has this adaptable species managed to colonize a wide range of ecosystems, including both natural and human-modified landscapes?",
    "Name one characteristic physical feature of foxes, and explain how this feature contributes to their survival strategies, particularly in terms of hunting, camouflage, and communication."
]


FinalEvalR_easyNN3, FinalEvalC_easyNN3, FinalEvalF_easyNN3, FinalResponseLength_easyNN3 = ResponseEvaluationMetrics(Queries_method=easy_queriesNN3)


easy_queriesNN4 = [
    "What language family does French belong to within the broader Indo-European family, and how does its classification relate to other Romance languages like Italian and Spanish in terms of linguistic evolution and shared vocabulary?",
    "Name two countries where French is an official language, and discuss the historical and cultural factors, such as colonization and the Francophonie, that have contributed to the spread of French as an official language in these regions.",
    "What is the origin of the French language, particularly in terms of its evolution from Latin during the Roman Empire, and how did historical events like the Frankish invasions influence its development into Old French?",
    "When did Gerald Ford serve as President of the United States, and what were the key political and social challenges he faced during his time in office, particularly in the aftermath of the Watergate scandal?",
    "Who was Gerald Ford's vice president, and how did his background and political experience influence the administration's domestic and foreign policies during a period of economic uncertainty and geopolitical tension?",
    "What event led to Gerald Ford becoming president, and how did this unprecedented political transition affect the American public's trust in the government and the overall functioning of the U.S. political system?",
    "What language family does German belong to within the Indo-European family, and how does its classification as a West Germanic language relate to other Germanic languages like English and Dutch in terms of shared linguistic features?",
    "Name two countries where German is an official language, and discuss how historical, cultural, and economic ties have contributed to the prominence of the German language in these countries' education systems and official communications.",
    "What is the standard form of the German language called, and how does it serve as a unifying linguistic standard across different German-speaking regions, particularly in terms of education, media, and official discourse?",
    "What is the capital of Ghana, and how does its role as a political, economic, and cultural center reflect Ghana's history as a leading nation in West Africa, particularly in terms of its contributions to the Pan-African movement?",
    "Name one major river in Ghana, and discuss its significance in the country's history, economy, and culture, particularly in terms of its role in agriculture, transportation, and traditional ceremonies.",
    "What is Ghana's official language, and how does its adoption as the official language reflect Ghana's colonial history, as well as its current efforts to promote national unity and cultural identity?",
    "What is the primary diet of giant pandas, and how has their specialized diet of bamboo influenced their evolutionary adaptations, behavior, and conservation status in the mountainous regions of China?",
    "Where are giant pandas naturally found, and how does their habitat in the bamboo forests of China contribute to their unique ecological role and the challenges they face in terms of habitat loss and climate change?",
    "What is one physical characteristic unique to giant pandas that distinguishes them from other bear species, and how has this feature evolved to support their bamboo-based diet and arboreal lifestyle?",
    "What is the scientific name of the giraffe, and how does this classification reflect the giraffe's evolutionary history and its relationship to other members of the Giraffidae family?",
    "How tall can male giraffes grow, and how does their height contribute to their feeding behavior, social structure, and survival in the savannas and woodlands of Africa?",
    "What is the primary diet of giraffes, and how does their preference for acacia leaves influence their role in the ecosystem, particularly in terms of vegetation management and interactions with other herbivores?",
    "What is the scientific name of the gray wolf, and how does this classification reflect the wolf's evolutionary relationship with other canids, as well as its adaptations to diverse habitats across the Northern Hemisphere?",
    "Where are gray wolves commonly found, and how have their populations adapted to different environments, from forests to tundras, and what are the ecological roles they play in these ecosystems?",
    "What is the primary diet of gray wolves, and how does their role as apex predators influence the structure and dynamics of the ecosystems they inhabit, particularly in terms of prey populations and trophic cascades?",
    "When did Grover Cleveland serve as President of the United States, and how did his non-consecutive terms reflect the political dynamics of the late 19th century, particularly in terms of party politics and voter sentiment?",
    "What is unique about Grover Cleveland's terms in office, and how did his return to the presidency after a four-year gap influence his second administration's policies and his legacy in American political history?",
    "Who was Grover Cleveland's vice president during each of his terms, and how did their political influence and relationship with Cleveland shape the administration's approach to domestic and foreign policy issues?",
    "What family of musical instruments does the guitar belong to within the broader classification of stringed instruments, and how does its design and playing technique differ from other plucked string instruments like the lute and the banjo?",
    "How many strings does a standard guitar have, and how does the number and tuning of these strings influence the instrument's range, versatility, and use in different musical genres, from classical to rock?",
    "Name one famous guitarist, and discuss how their playing style, technique, and contributions to music have influenced the development of the guitar as a key instrument in modern music, particularly in terms of its role in popular culture."
]


FinalEvalR_easyNN4, FinalEvalC_easyNN4, FinalEvalF_easyNN4, FinalResponseLength_easyNN4 = ResponseEvaluationMetrics(Queries_method=easy_queriesNN4)


easy_queriesNN5 = [
    "When was Gustav Klimt born, and how did his upbringing and the cultural environment of Vienna at the time influence his early artistic development and later works?",
    "What art movement was Gustav Klimt associated with, particularly in relation to his role in the Vienna Secession, and how did this movement challenge the traditional art norms of the time?",
    "Name one famous painting by Gustav Klimt, and discuss how this work exemplifies his use of symbolism, intricate patterns, and the innovative techniques that characterized his 'Golden Phase'.",
    "What is the capital city of Indonesia, and how does Jakarta's role as the political, economic, and cultural center of Indonesia reflect the country's diverse cultural heritage and complex history?",
    "Name one major island in Indonesia, and discuss its significance in terms of cultural diversity, natural resources, and its role in the nation's economy and tourism industry.",
    "What is the official language of Indonesia, and how does the use of Bahasa Indonesia as a unifying language reflect the nation's efforts to promote national identity and unity across its diverse population?",
    "When was Isaac Newton born, and how did the scientific environment of 17th-century England influence his groundbreaking contributions to mathematics, physics, and astronomy?",
    "What is Isaac Newton best known for, particularly in terms of his formulation of the laws of motion and universal gravitation, and how did these ideas revolutionize our understanding of the physical world?",
    "Name one major work published by Isaac Newton, and discuss its impact on the development of modern science, particularly in the fields of physics, mathematics, and astronomy.",
    "When was Jackson Pollock born, and how did his early life experiences and the cultural environment of mid-20th-century America shape his development as a leading figure in abstract expressionism?",
    "What art movement was Jackson Pollock associated with, and how did his unique 'drip painting' technique challenge traditional notions of painting and contribute to the evolution of modern art?",
    "Name one famous painting by Jackson Pollock, and discuss how this work exemplifies his innovative use of color, form, and technique in the creation of dynamic, abstract compositions.",
    "When did James Monroe serve as President of the United States, and how did his presidency, known as the 'Era of Good Feelings', reflect the political and social dynamics of the early 19th century?",
    "What is James Monroe best known for, particularly in terms of his foreign policy, and how did the Monroe Doctrine shape America's approach to international relations and its role in the Western Hemisphere?",
    "Name one major event that occurred during Monroe's presidency, and discuss its significance in the context of American political history and Monroe's efforts to promote national unity and security.",
    "When was James Watt born, and how did the Industrial Revolution and the scientific advancements of the 18th century influence his innovations in steam engine technology?",
    "What is James Watt best known for, particularly in terms of his improvements to the steam engine, and how did these innovations contribute to the broader industrialization of Europe and the world?",
    "Name one major improvement James Watt made to the steam engine, and discuss how this advancement revolutionized industrial processes and transportation, paving the way for the modern industrial economy.",
    "What writing systems are used in the Japanese language, and how do these systems, including Kanji, Hiragana, and Katakana, reflect the linguistic and cultural history of Japan?",
    "How many main dialects are there in Japanese, and how do these regional variations reflect the diverse cultural and historical influences across different parts of Japan?",
    "What is the standard form of Japanese called, and how does its use in education, media, and government promote linguistic unity in a country with significant regional dialectal diversity?",
    "When did John Adams serve as President of the United States, and how did his leadership during the early years of the republic shape the development of American political institutions and foreign policy?",
    "What is John Adams best known for, particularly in terms of his role in the American Revolution and his efforts to establish a strong federal government during his presidency?",
    "Who was John Adams' vice president, and how did their relationship influence the administration's approach to key issues such as foreign policy, domestic governance, and party politics?"
]


FinalEvalR_easyNN5, FinalEvalC_easyNN5, FinalEvalF_easyNN5, FinalResponseLength_easyNN5 = ResponseEvaluationMetrics(Queries_method=easy_queriesNN5)



easy_queriesNN6 = [
    "What family do kangaroos belong to, and how does their classification within the Macropodidae family reflect their evolutionary adaptations for hopping and grazing in the diverse environments of Australia?",
    "Name two species of kangaroos, and discuss their distinct physical characteristics, habitats, and the ecological roles they play in the Australian landscape.",
    "What is the primary diet of kangaroos, and how do their specialized digestive systems allow them to thrive on a diet of grasses and shrubs in the often harsh and arid environments of Australia?",
    "What is the scientific name of the koala, and how does its classification within the Phascolarctidae family relate to its unique adaptations for a eucalyptus-based diet and arboreal lifestyle?",
    "Where are koalas naturally found, particularly in terms of their distribution across Australia's eastern states, and how does their habitat preference for eucalyptus forests influence their conservation status?",
    "What is the primary diet of koalas, and how have their highly specialized feeding habits on eucalyptus leaves shaped their behavior, physiology, and vulnerability to environmental changes?",
    "What writing systems are used in the Korean language, particularly in terms of the historical development and use of Hangul, Hanja, and the roles these scripts play in modern Korean literacy and education?",
    "How many main dialects are there in Korean, and how do these regional dialects reflect the linguistic diversity and cultural history of the Korean Peninsula?",
    "What is the standard form of Korean called, and how does its use in government, education, and media promote linguistic unity and cultural identity across South Korea?",
    "When was Leonardo da Vinci born, and how did the intellectual and artistic environment of Renaissance Italy influence his development as one of history's greatest polymaths?",
    "What is Leonardo da Vinci best known for, particularly in terms of his contributions to art, science, and engineering, and how do these diverse achievements reflect the ideals of the Renaissance?",
    "Name one famous painting by Leonardo da Vinci, and discuss how this work exemplifies his mastery of techniques such as chiaroscuro, sfumato, and his deep understanding of human anatomy.",
    "What is the scientific name of the leopard, and how does its classification within the Panthera genus relate to its adaptations for solitary hunting and survival in diverse habitats?",
    "Where are leopards commonly found, particularly in terms of their distribution across Africa and Asia, and how do their adaptability to different environments contribute to their widespread presence?",
    "What is the primary diet of leopards, and how do their hunting strategies, physical adaptations, and opportunistic feeding habits enable them to be one of the most versatile predators in their ecosystems?",
    "What is the capital of Liechtenstein, and how does Vaduz's role as the political, cultural, and economic center of the principality reflect the unique governance and international relationships of Liechtenstein?",
    "What is the official language of Liechtenstein, and how does the use of German as the national language reflect the cultural and historical ties between Liechtenstein and its neighboring German-speaking countries?",
    "What is the population of Liechtenstein, and how does its small population size influence the country's social structure, economy, and governance?",
    "What is the capital of Peru, and how does Lima's role as the cultural, economic, and political heart of Peru reflect the country's rich history and diverse cultural heritage?",
    "Name one famous landmark in Lima, and discuss its historical significance, architectural features, and how it reflects Lima's role in the colonial and modern history of Peru.",
    "What is the population of Lima, and how does the city's large and growing population impact its infrastructure, economy, and social dynamics?",
    "What is the scientific name of the lion, and how does its classification within the Panthera genus relate to its social behavior, hunting strategies, and status as a keystone predator in African ecosystems?",
    "Where are lions commonly found, particularly in terms of their distribution across sub-Saharan Africa, and how do their habitat preferences influence their conservation status and interactions with human populations?",
    "What is the primary diet of lions, and how do their social hunting strategies, physical adaptations, and role as apex predators shape the dynamics of their ecosystems?",
    "How many species of lizards are there, and how does the vast diversity of lizard species reflect their adaptations to a wide range of habitats and ecological niches across the globe?",
    "What is the primary diet of most lizards, and how do their varied feeding habits, including herbivory, insectivory, and carnivory, reflect their ecological roles in different environments?",
    "Name one unique feature of lizards, and discuss how this feature contributes to their survival and adaptation to their respective habitats."
]

FinalEvalR_easyNN6, FinalEvalC_easyNN6, FinalEvalF_easyNN6, FinalResponseLength_easyNN6 = ResponseEvaluationMetrics(Queries_method=easy_queriesNN6)


easy_queriesNN7 = [
    "What family do clawed lobsters belong to, and how does their classification within the Nephropidae family reflect their evolutionary adaptations for living in various marine environments?",
    "Name two types of lobsters, and discuss their distinct physical characteristics, habitats, and roles in the marine ecosystem.",
    "What is the primary diet of lobsters, and how do their feeding habits and scavenging behaviors contribute to their role in the marine food web?",
    "What is the capital city of the United Kingdom, and how does London's historical and cultural significance influence its status as a global city?",
    "Name one famous landmark in London, and discuss its historical significance, architectural features, and role in shaping the cultural identity of the city.",
    "What river runs through London, and how has the Thames River historically contributed to the city's development, economy, and cultural heritage?",
    "What type of musical instrument is a lyre, and how does its classification as a stringed instrument relate to its historical use in ancient music and ceremonies?",
    "Name one ancient civilization that used the lyre, and discuss how this instrument played a role in their cultural and religious practices.",
    "What is the primary material used for lyre strings, and how did the choice of materials influence the sound quality and construction of this ancient instrument?",
    "What language family does Malay belong to, and how does its classification within the Austronesian family reflect its linguistic characteristics and historical spread across Southeast Asia?",
    "Name one country where Malay is an official language, and discuss the cultural and political significance of the language in that country.",
    "What script is used to write Malay, and how has the Jawi script influenced the development of written Malay literature and religious texts?",
    "When was Michelangelo born, and how did the Renaissance period influence his development as a sculptor, painter, and architect?",
    "What is Michelangelo best known for, particularly in terms of his contributions to sculpture and painting, and how do these works exemplify the ideals of the Renaissance?",
    "Name one famous sculpture by Michelangelo, and discuss how it reflects his mastery of anatomy, form, and the portrayal of human emotion.",
    "When did Millard Fillmore serve as President of the United States, and how did the political context of the time shape his presidency and policy decisions?",
    "Who was Millard Fillmore's vice president, and how did their political relationship influence the administration's approach to domestic and foreign policies?",
    "What event led to Millard Fillmore becoming president, and how did this transition of power impact the political landscape of the United States at the time?",
    "When was Norman Rockwell born, and how did the cultural and social climate of early 20th century America influence his development as an illustrator?",
    "What is Norman Rockwell best known for, particularly in terms of his contributions to American art and culture, and how do his works reflect the values and concerns of his time?",
    "Name one famous painting by Norman Rockwell, and discuss how it captures the spirit of American life, culture, and the social issues of its era.",
    "How many arms does an octopus have, and how do their unique physical characteristics and abilities contribute to their survival and predatory behavior in marine environments?",
    "What is the primary diet of octopuses, and how do their feeding habits and hunting strategies reflect their intelligence and adaptability in the ocean?",
    "Where are octopuses commonly found, particularly in terms of their distribution in various marine habitats, and how do their adaptations to these environments influence their behavior and ecology?",
    "What family do otters belong to, and how does their classification within the Mustelidae family reflect their adaptations for life in aquatic environments?",
    "Name one type of otter, and discuss its unique physical characteristics, habitat preferences, and role in the ecosystem.",
    "What is the primary diet of otters, and how do their feeding habits and hunting strategies reflect their role as predators in aquatic ecosystems?"
]


FinalEvalR_easyNN7, FinalEvalC_easyNN7, FinalEvalF_easyNN7, FinalResponseLength_easyNN7 = ResponseEvaluationMetrics(Queries_method=easy_queriesNN7)


easy_queriesNN8 = [
    "When was Pablo Picasso born, and how did his early life and upbringing in Spain influence his later work in the Cubist art movement?",
    "What art movement is Pablo Picasso associated with, and how did his contributions to Cubism redefine the concepts of perspective and form in modern art?",
    "Name one famous painting by Pablo Picasso, and discuss how this work exemplifies his innovative techniques and thematic explorations within the Cubist movement.",
    "How many species of penguins are there, and how do their unique adaptations to cold environments contribute to their survival in diverse ecosystems across the Southern Hemisphere?",
    "What is the primary diet of penguins, and how do their feeding behaviors and foraging strategies reflect their role in the marine food web?",
    "Where are penguins commonly found, and how do their geographic distribution and habitat preferences impact their behavior, breeding, and conservation status?",
    "How many keys does a standard piano have, and how does the keyboard layout facilitate the performance of complex musical compositions across various genres?",
    "What are the main types of pianos, and how do their structural differences and sound production methods influence their use in classical and contemporary music?",
    "Who is known as the 'father of the modern piano,' and how did his innovations in piano design and manufacturing techniques revolutionize the instrument and its role in Western music?",
    "When was Pierre-Auguste Renoir born, and how did the socio-political climate of 19th century France influence his development as a leading figure in the Impressionist movement?",
    "What art movement is Renoir associated with, and how did his focus on light, color, and everyday scenes contribute to the evolution of Impressionism?",
    "Name one famous painting by Renoir, and discuss how this work reflects his mastery of color, light, and the depiction of human emotion within the context of Impressionist art.",
    "When was Piet Mondrian born, and how did his experiences in the Netherlands and subsequent exposure to avant-garde movements shape his contributions to abstract art?",
    "What art movement is Mondrian associated with, and how did his development of Neoplasticism influence the trajectory of abstract art in the 20th century?",
    "Name one famous painting by Mondrian, and discuss how this work exemplifies his use of geometric forms and primary colors to express his philosophical ideas about harmony and order.",
    "What is the scientific name of the polar bear, and how does its classification within the Ursidae family reflect its evolutionary adaptations for survival in Arctic environments?",
    "Where are polar bears commonly found, and how do their behaviors, hunting strategies, and interactions with their environment reflect their role as apex predators in the Arctic?",
    "What is the primary diet of polar bears, and how do their feeding habits and reliance on sea ice for hunting impact their survival and conservation status?",
    "What language family does Portuguese belong to, and how does its classification within the Romance languages reflect its historical development and influence across the Lusophone world?",
    "Name one country where Portuguese is an official language, and discuss how the language's spread through colonization has shaped the cultural and linguistic landscape of that country.",
    "What script is used to write Portuguese, and how has the Latin script's adaptation to the phonological structure of Portuguese influenced its orthography and literary traditions?",
    "What is the capital of Qatar, and how does Doha's rapid urban development and economic growth reflect the broader trends of modernization and globalization in the Gulf region?",
    "Name one major industry in Qatar, and discuss how the country's natural gas reserves have driven its economic diversification and international influence.",
    "What is the official language of Qatar, and how does the use of Arabic in education, government, and media reflect the country's cultural identity and connection to the broader Arab world?",
    "What is the capital of Singapore, and how has the city-state's strategic location and economic policies contributed to its status as a global financial hub?",
    "Name one major industry in Singapore, and discuss how the country's focus on technology and innovation has driven its economic success and global competitiveness.",
    "What is the official language of Singapore, and how does the multilingual policy reflect the country's cultural diversity and efforts to maintain social harmony among its population?"
]


FinalEvalR_easyNN8, FinalEvalC_easyNN8, FinalEvalF_easyNN8, FinalResponseLength_easyNN8 = ResponseEvaluationMetrics(Queries_method=easy_queriesNN8)


easy_queriesNN9 = [
    "What language family does Spanish belong to, and how does its classification within the Romance languages reflect its historical development and spread across the globe?",
    "Name two countries where Spanish is an official language, and discuss how the language's prevalence in these countries has influenced their cultural and social landscapes.",
    "What script is used to write Spanish, and how has the Latin script been adapted to accommodate the phonological characteristics of the Spanish language?",
    "What language family does Swahili belong to, and how does its classification within the Bantu languages reflect its origins and the influence of Arabic and other languages?",
    "Name one country where Swahili is an official language, and discuss how Swahili's status as a lingua franca has shaped its role in communication, trade, and culture in East Africa.",
    "What script is used to write Swahili, and how has the adoption of the Latin script influenced the standardization and dissemination of the Swahili language?",
    "What family do swans belong to, and how does their classification within the Anatidae family reflect their anatomical and behavioral characteristics?",
    "Name two species of swans, and discuss how their geographic distribution, physical adaptations, and behaviors distinguish them from each other.",
    "What is the primary diet of swans, and how do their feeding habits and preferences contribute to their role in aquatic ecosystems?",
    "What language family does Swedish belong to, and how does its classification within the North Germanic languages reflect its historical development and linguistic characteristics?",
    "Name one country where Swedish is an official language, and discuss how the language's official status has influenced education, media, and cultural identity in that country.",
    "What script is used to write Swedish, and how has the Latin script been adapted to represent the unique phonological features of the Swedish language?",
    "When did Theodore Roosevelt serve as President of the United States, and how did his leadership during this period shape the nation's domestic and foreign policies?",
    "Who was Theodore Roosevelt's vice president, and how did their partnership influence the administration's approach to key issues such as conservation, trust-busting, and foreign policy?",
    "What event led to Theodore Roosevelt becoming president, and how did this event impact his subsequent approach to governance and his legacy as a progressive leader?",
    "What is the scientific name of the tiger, and how does its classification within the Panthera genus reflect its evolutionary history and physical adaptations?",
    "Where are tigers commonly found, and how do their habitats, ranging from dense forests to grasslands, influence their hunting strategies, behaviors, and conservation status?",
    "What is the primary diet of tigers, and how do their carnivorous feeding habits and position as apex predators impact their role in maintaining the balance of their ecosystems?",
    "What type of musical instrument is a trumpet, and how does its classification as a brass instrument reflect its method of sound production and use in various musical genres?",
    "How many valves does a standard trumpet have, and how do these valves function to change the pitch and facilitate the performance of complex musical pieces?",
    "What is the primary material used to make trumpets, and how does the choice of material influence the instrument's tone, durability, and suitability for different styles of music?",
    "What language family does Turkish belong to, and how does its classification within the Turkic languages reflect its historical development and linguistic characteristics?",
    "Name one country where Turkish is an official language, and discuss how the language's official status has influenced national identity, education, and media in that country.",
    "What script is used to write Turkish, and how did the adoption of the Latin script in the 20th century impact the language's orthography and accessibility?",
    "How many species of turtles are there, and how does the diversity of these species reflect their adaptations to various habitats, from terrestrial to aquatic environments?",
    "What is the primary diet of most turtles, and how do their feeding habits and preferences vary across different species and influence their role in ecosystems?",
    "Name one unique feature of turtles, and discuss how this feature has contributed to their survival and success across diverse environments and over millions of years."
]

FinalEvalR_easyNN9, FinalEvalC_easyNN9, FinalEvalF_easyNN9, FinalResponseLength_easyNN9 = ResponseEvaluationMetrics(Queries_method=easy_queriesNN9)



easy_queriesNN10 = [
    "When did Ulysses S. Grant serve as President of the United States, and how did his military background influence his leadership style during his presidency?",
    "Who was Ulysses S. Grant's vice president, and how did their partnership shape the policies and direction of the Grant administration?",
    "What was Ulysses S. Grant's role during the Civil War, particularly in terms of his strategies as a Union general and how they contributed to the eventual victory of the Union forces?",
    "What is the capital of Uruguay, and how does Montevideo's status as a cultural and economic hub reflect Uruguay's development as a nation?",
    "Name one major industry in Uruguay, and discuss how this industry contributes to the country's economy and its position within the South American economic landscape.",
    "What is the official language of Uruguay, and how does the use of Spanish reflect the country's colonial history and cultural ties with other Spanish-speaking nations?",
    "What language family does Vietnamese belong to, and how does its classification within the Austroasiatic languages reflect the historical and cultural influences on the Vietnamese language?",
    "Name one country where Vietnamese is an official language, and discuss how the language's status influences national identity, education, and media in that country.",
    "What script is used to write Vietnamese, and how did the adoption of the Latin-based Quoc Ngu script during the French colonial period impact the standardization and accessibility of the Vietnamese language?",
    "When was Vincent van Gogh born, and how did his early life and experiences shape his unique artistic style and themes?",
    "What art movement is Vincent van Gogh associated with, particularly in terms of his contributions to Post-Impressionism and how his work influenced the development of modern art?",
    "Name one famous painting by Vincent van Gogh, and discuss how the themes, techniques, and use of color in this painting exemplify his artistic vision.",
    "How many strings does a standard violin have, and how do the materials and construction of these strings contribute to the instrument's sound and playability?",
    "What is the primary material used to make violin bows, and how do the properties of this material influence the performance and tonal qualities of the violin?",
    "Name one famous violinist, and discuss their contributions to the world of classical music, including their unique playing style and influence on the development of violin technique.",
    "When did Woodrow Wilson serve as President of the United States, and how did his presidency coincide with significant global events such as World War I?",
    "Who was Woodrow Wilson's vice president, and how did their collaboration impact the administration's policies, particularly in areas such as domestic reform and international relations?",
    "What major global event occurred during Wilson's presidency, and how did Wilson's leadership during this event shape the course of American and world history?",
    "What type of musical instrument is a xylophone, and how does its classification as a percussion instrument reflect its method of sound production and use in various musical genres?",
    "What is the primary material used to make xylophone bars, and how do the acoustic properties of this material influence the instrument's tone and suitability for different musical styles?",
    "Name one culture that uses the xylophone in traditional music, and discuss how the instrument's role in this culture reflects its historical and cultural significance.",
    "How many species of zebras are there, and how do the physical and behavioral characteristics of these species reflect their adaptations to different environments across Africa?",
    "What is the primary diet of zebras, and how do their feeding habits and role as herbivores impact the ecosystems in which they live?",
    "Where are zebras commonly found, and how do their habitats influence their social structures, behaviors, and interactions with other species in the ecosystem?"
]



FinalEvalR_easyNN10, FinalEvalC_easyNN10, FinalEvalF_easyNN10, FinalResponseLength_easyNN10 = ResponseEvaluationMetrics(Queries_method=easy_queriesNN10)






In [ ]:
FinalR_easyNNN=pd.concat([FinalEvalR_easyNN1, FinalEvalR_easyNN2, FinalEvalR_easyNN3, FinalEvalR_easyNN4, FinalEvalR_easyNN5, FinalEvalR_easyNN6, FinalEvalR_easyNN7,
                         FinalEvalR_easyNN8, FinalEvalR_easyNN9, FinalEvalR_easyNN10], axis=0)
FinalC_easyNNN=pd.concat([FinalEvalC_easyNN1, FinalEvalC_easyNN2, FinalEvalC_easyNN3, FinalEvalC_easyNN4, FinalEvalC_easyNN5, FinalEvalC_easyNN6, FinalEvalC_easyNN7,
                         FinalEvalC_easyNN8, FinalEvalC_easyNN9, FinalEvalC_easyNN10], axis=0)

FinalF_easyNNN=pd.concat([FinalEvalF_easyNN1, FinalEvalF_easyNN2, FinalEvalF_easyNN3, FinalEvalF_easyNN4, FinalEvalF_easyNN5, FinalEvalF_easyNN6, FinalEvalF_easyNN7,
                         FinalEvalF_easyNN8, FinalEvalF_easyNN9, FinalEvalF_easyNN10], axis=0)

FinalResponseLength_easyNNN=pd.concat([FinalResponseLength_easyNN1, FinalResponseLength_easyNN2, FinalResponseLength_easyNN3, FinalResponseLength_easyNN4,
                                      FinalResponseLength_easyNN5, FinalResponseLength_easyNN6, FinalResponseLength_easyNN7, FinalResponseLength_easyNN8,
                                      FinalResponseLength_easyNN9, FinalResponseLength_easyNN10], axis=0)


In [ ]:
medium_queriesNN1 = [
    "What were the key points of the Emancipation Proclamation, issued by Abraham Lincoln during the American Civil War, and how did it influence the abolition of slavery in the Confederate states, the reaction of the Union army, and the broader implications for civil rights in the United States?",
    "Describe the events leading up to the assassination of Abraham Lincoln, including the conspiracy led by John Wilkes Booth, the motivations behind the assassination, the immediate aftermath, and its impact on the post-Civil War Reconstruction era and the course of American history.",
    "How did Abraham Lincoln's early life, including his upbringing in a log cabin in Kentucky, his self-education, his experiences as a lawyer in Illinois, and his debates with Stephen A. Douglas, influence his political career and his rise to the presidency during the Civil War?",
    "Explain the social structure of an ant colony, including the roles of the queen, workers, and soldiers, how these roles contribute to the survival and efficiency of the colony, the process of pheromone communication, and the division of labor in tasks like foraging, defense, and brood care.",
    "How do ants communicate with each other within their colonies, using pheromones and other chemical signals to coordinate activities such as foraging, defense, and caring for the queen's offspring, and how does this communication differ among different species of ants?",
    "What are the main predators of ants in the wild, including species like anteaters, birds, and other insects, and how do ants defend themselves against these predators through strategies like chemical defenses (venom, formic acid), physical barriers, and group behaviors?",
    "Explain the differences between Modern Standard Arabic, the formal language used in media, literature, and international diplomacy, and Classical Arabic, the language of the Quran and Islamic scholarship, and how these variations are used in different cultural and religious contexts.",
    "How has the Arabic language influenced other languages, particularly in the fields of science, mathematics, and literature during the Islamic Golden Age, and how Arabic loanwords have been integrated into languages such as Spanish, Turkish, and Persian?",
    "Describe the role of Arabic in the context of Islam, including its use as the liturgical language of the Quran, its significance in Islamic prayers and rituals (such as Salat), and its role in preserving Islamic jurisprudence, theology, and scholarly works throughout history.",
    "Explain the process of pollination and how bees, particularly honeybees, contribute to it by transferring pollen between flowers, which is essential for the reproduction of many plants, including key agricultural crops, and the production of fruits, vegetables, and seeds.",
    "What are the major threats to bee populations worldwide, including factors like pesticide use (neonicotinoids), habitat loss due to urbanization and agriculture, diseases such as Varroa mites and Nosema, and the phenomenon of Colony Collapse Disorder (CCD), and how do these threats impact global food security?",
    "Describe the lifecycle of a honeybee, including the stages from egg to larva, pupa, and adult bee, the roles different bees play within the hive (queen, workers, drones), and how environmental factors like temperature and food availability influence their development.",
    "Explain the significance of the hard exoskeleton in beetles, which provides protection, support, and helps prevent desiccation in various environments, how it contributes to their survival in diverse habitats, and the role of the exoskeleton in the evolutionary success of beetles.",
    "What are the main differences between beetle larvae and adult beetles in terms of their morphology, behavior, and ecological roles, including how larvae are often specialized for feeding and growth, while adults are adapted for dispersal and reproduction?",
    "Describe the role of beetles in the decomposition process, including how species like dung beetles and carrion beetles contribute to nutrient cycling by breaking down organic matter, their importance in ecosystems as recyclers of nutrients, and their interactions with other decomposers like fungi and bacteria.",
    "Explain the significance of the Forbidden City in Beijing's history, including its role as the imperial palace during the Ming and Qing dynasties, its architectural features like the Meridian Gate and Hall of Supreme Harmony, and its status as a symbol of China's imperial past and cultural heritage.",
    "How has Beijing's economy evolved in the past two decades, particularly with the rise of technology sectors like information technology and e-commerce, the growth of financial services, and the expansion of infrastructure projects like the Beijing Daxing International Airport, and what challenges does the city face in sustaining growth?",
    "Describe the impact of air pollution on Beijing's environment and public health, including the sources of pollution such as vehicle emissions and industrial activity, the effects on respiratory health and quality of life, and the measures being taken by the government to reduce pollution levels, such as implementing stricter emissions standards and promoting public transportation.",
    "Explain the process of metamorphosis in butterflies, including the stages of egg, larva (caterpillar), pupa (chrysalis), and adult, how hormonal changes regulate this transformation, and the significance of metamorphosis for the survival and reproduction of butterflies in various environments.",
    "How do butterflies contribute to pollination, and what role do they play in the ecosystem as both pollinators and prey for other species, including their interactions with flowering plants, their importance in maintaining plant diversity, and their role in food webs as a source of nutrition for birds and other predators?",
    "Describe the role of butterflies in their ecosystems, including their interactions with plants and other animals, their importance in maintaining biodiversity, and the threats they face from habitat loss, climate change, and pesticides, and how these threats affect their populations and ecological roles."
]


FinalEvalR_mediumNN1, FinalEvalC_mediumNN1, FinalEvalF_mediumNN1, FinalResponseLength_mediumNN1 = ResponseEvaluationMetrics(Queries_method=medium_queriesNN1)

medium_queriesNN2 = [
    "Describe the key domestic policies of Calvin Coolidge's administration, including his efforts to reduce taxes, shrink government spending, and support the growth of the American economy during the 1920s, and how these policies influenced the economic boom known as the Roaring Twenties.",
    "How did Calvin Coolidge handle the Boston Police Strike of 1919 during his tenure as Governor of Massachusetts, and how did his response to this strike, which emphasized law and order, help to shape his political career and public image, eventually leading to his rise to the presidency?",
    "What was Calvin Coolidge's approach to foreign policy during his presidency, including his administration's stance on isolationism, international trade, and efforts to maintain peace through agreements such as the Kellogg-Briand Pact, and how did these policies reflect the broader trends in U.S. foreign relations during the interwar period?",
    "Describe the significance of the Canadian Charter of Rights and Freedoms, a cornerstone of Canada's constitutional framework, and how it protects individual rights and freedoms, balances federal and provincial powers, and reflects the country's commitment to multiculturalism and human rights.",
    "How does Canada's healthcare system, known for its publicly funded, universal coverage, differ from that of the United States, particularly in terms of access to medical services, cost-effectiveness, and outcomes, and what challenges and debates surround the sustainability and efficiency of Canada's healthcare model?",
    "Explain the economic importance of the natural resources sector in Canada, including its contributions to the country's GDP, export revenues, and employment, as well as the environmental and social challenges associated with resource extraction industries such as mining, forestry, and oil and gas production.",
    "Explain the role of the cello in a string quartet, a key ensemble in classical music, and how it provides the harmonic foundation and supports the overall structure of the music, while also contributing to the melodic content through its rich, resonant tone, particularly in works by composers such as Beethoven and Shostakovich.",
    "Describe the differences between a cello and a double bass, two members of the string instrument family, focusing on their size, tuning, range, playing techniques, and roles in orchestral and chamber music settings, and how these differences influence their respective sounds and musical applications.",
    "What is the standard tuning of a cello, and how do the intervals between the strings (in fifths) allow for a wide range of expression and technical possibilities in various musical genres, from Baroque and Classical music to contemporary compositions and improvisational performances?",
    "Explain the differences between Simplified and Traditional Chinese characters, including the historical context behind the simplification process initiated in the People's Republic of China, how these writing systems are used in different regions (Mainland China, Taiwan, Hong Kong), and the implications for literacy, cultural preservation, and language education.",
    "How has the Chinese language influenced other East Asian languages, particularly in terms of vocabulary, writing systems, and literary traditions, and what are some examples of Chinese loanwords and characters that have been integrated into languages like Japanese, Korean, and Vietnamese?",
    "Describe the role of Chinese characters in Japanese and Korean writing systems, including the use of Kanji in Japanese and Hanja in Korean, how these characters function alongside native scripts (Hiragana, Katakana, Hangul), and the impact of Chinese characters on the development of written and spoken language in these cultures.",
    "Describe the physical characteristics of a cougar, a large and powerful predator, including its muscular build, sharp retractable claws, and keen sense of vision and hearing, which make it an effective hunter across a wide range of habitats from forests to deserts and mountains.",
    "Explain the hunting techniques used by cougars, including their stealthy stalking behavior, ability to ambush prey with powerful leaps, and use of their strong jaws and claws to deliver a fatal bite, and how these techniques allow them to take down prey much larger than themselves, such as deer and elk.",
    "What are the main threats to cougar populations in the wild, including habitat loss due to human encroachment, conflicts with livestock owners leading to retaliatory killings, road mortality, and the impact of reduced prey availability, and how conservation efforts are addressing these challenges to ensure the survival of this apex predator?",
    "Explain the role of cymbals in an orchestra, particularly in the percussion section, where they are used to create dramatic effects, accentuate climaxes, and enhance the overall texture of the music, and how different types of cymbals (e.g., crash, splash, sizzle) are selected based on the specific requirements of the composition.",
    "Describe the differences between crash cymbals and ride cymbals, two common types of cymbals used in both orchestral and modern music settings, focusing on their size, sound characteristics, playing techniques, and how they contribute to different musical textures and rhythms in a piece.",
    "How are cymbals used in a drum kit, a staple of modern music genres such as rock, jazz, and pop, and what roles do different cymbals (e.g., hi-hats, crashes, rides) play in creating rhythm, dynamics, and accents, and how do drummers use various techniques like crashing, riding, and splashing to achieve their desired sound?",
    "Explain the historical significance of Dhaka during the Mughal period, when it served as an important administrative and commercial center in Bengal, known for its thriving textile industry, particularly the production of fine muslin, and how this legacy continues to influence the city's cultural and economic identity today.",
    "How did the partition of India in 1947 affect Dhaka, leading to its transition from being part of British India to becoming the capital of East Pakistan, and eventually the capital of an independent Bangladesh, and what were the social, economic, and political impacts of these changes on the city's development?",
    "Describe the main economic activities in Dhaka, one of the fastest-growing cities in the world, focusing on key sectors such as the textile and garment industry, which is a major driver of the country's economy, the role of remittances from expatriate workers, and the challenges posed by rapid urbanization, poverty, and environmental degradation.",
    "Explain the lifecycle of a dragonfly, an insect known for its remarkable aerial abilities, including the stages of egg, nymph (larva), and adult, the importance of aquatic habitats for the nymph stage, and how the transformation into an adult involves a dramatic metamorphosis that prepares the dragonfly for its role as a predator in the ecosystem.",
    "Describe the differences between dragonflies and damselflies, two closely related groups of insects in the order Odonata, focusing on their physical characteristics (e.g., wing shape, body structure), behaviors, and ecological roles, and how these differences affect their interactions with the environment and other species.",
    "How do dragonflies contribute to their ecosystems, particularly as predators that help control mosquito populations, their role as prey for birds and other wildlife, and their importance as bioindicators of environmental health, especially in freshwater ecosystems where they breed and develop?"
]


FinalEvalR_mediumNN2, FinalEvalC_mediumNN2, FinalEvalF_mediumNN2, FinalResponseLength_mediumNN2 = ResponseEvaluationMetrics(Queries_method=medium_queriesNN2)



medium_queriesNN3 = [
    "Describe the differences between diving ducks and dabbling ducks, focusing on their feeding habits, physical adaptations, and preferred habitats, and how these differences impact their role in the ecosystem and their interactions with other waterfowl.",
    "How do ducks' beaks help them filter food from water, and what are the specific structural adaptations that allow different species of ducks to efficiently forage for different types of food in various aquatic environments?",
    "What are the main predators of ducks in different regions of the world, and how have ducks evolved behavioral and physical defenses to evade these predators, particularly during vulnerable life stages like nesting and molting?",
    "Describe the significance of the Suez Canal, a vital waterway connecting the Mediterranean Sea and the Red Sea, in Egypt's economy, global trade routes, and geopolitical strategy, and how its operation has influenced international relations since its completion in 1869.",
    "How did the ancient Egyptian civilization influence modern Egypt in terms of culture, religion, architecture, and social organization, and what are the enduring legacies of this ancient civilization in contemporary Egyptian society?",
    "What are the major economic activities in Egypt, focusing on sectors like agriculture, tourism, and manufacturing, and how do these industries contribute to the country’s GDP, employment, and international trade relations?",
    "Describe the artistic style of El Greco, particularly his use of elongated forms, intense colors, and dramatic lighting, and how these elements reflect the spiritual and emotional themes in his work, which were influenced by both his Greek heritage and his immersion in Spanish culture.",
    "How did El Greco's work influence the Baroque movement in art, particularly in terms of the use of emotion, movement, and contrast, and what aspects of his style were adopted or adapted by later Baroque artists?",
    "What are the main themes in El Greco's paintings, such as religious devotion, mysticism, and the human struggle between the divine and the earthly, and how do these themes manifest in his composition, use of color, and depiction of figures?",
    "Describe the social structure of elephant herds, focusing on the roles of matriarchs, the behavior of bulls, and the relationships between family groups, and how this complex social organization supports the survival and well-being of elephants in the wild.",
    "How do elephants communicate with each other through vocalizations, body language, and other sensory cues, and what role does communication play in coordinating group activities, maintaining social bonds, and responding to environmental challenges?",
    "What are the main threats to elephant populations in the wild, including poaching, habitat loss, and human-wildlife conflict, and what conservation strategies are being implemented to mitigate these threats and ensure the long-term survival of elephants?",
    "Describe the differences between Old English and Modern English in terms of vocabulary, grammar, and pronunciation, and how historical events like the Norman Conquest and the Great Vowel Shift contributed to these linguistic changes.",
    "How has the British Empire contributed to the spread of English globally, particularly through colonization, trade, and cultural exchange, and what are the long-term effects of this spread on the development of English as a global lingua franca?",
    "What are the main characteristics of English as a global lingua franca, including its role in international business, science, and diplomacy, and how has the use of English in these contexts influenced its vocabulary, syntax, and register?",
    "Describe the Finnish education system, widely regarded as one of the best in the world, focusing on its emphasis on equality, student well-being, and teacher professionalism, and how these principles have contributed to Finland's high educational outcomes.",
    "How does Finland's healthcare system compare to other Nordic countries in terms of accessibility, quality, and efficiency, and what are the key features of the Finnish welfare model that support the health and well-being of its population?",
    "What are the major industries in Finland, including technology, forestry, and manufacturing, and how do these sectors contribute to the country's economic growth, employment, and international trade relations?",
    "Describe the phonological characteristics of Finnish, focusing on its vowel harmony, extensive case system, and the role of long and short vowels and consonants, and how these features distinguish Finnish from other European languages.",
    "How has the Finnish language influenced other languages in the region, particularly through loanwords and linguistic contact with neighboring languages like Swedish and Russian, and what role has Finnish played in the cultural identity of the Finnish people?",
    "What are the main differences between Finnish and Estonian, two closely related Finno-Ugric languages, in terms of phonology, grammar, and vocabulary, and how have historical and cultural factors contributed to these linguistic divergences?",
    "Describe the historical development of the flute, from its origins in ancient civilizations to its current form as a key instrument in Western classical music, and how innovations in design and manufacturing have expanded its range and versatility.",
    "How does the embouchure, the way a flutist shapes their lips and directs the airstream, affect the sound of a flute in terms of pitch, tone quality, and dynamics, and what techniques do flutists use to master this crucial aspect of flute playing?",
    "What are the main differences between a flute and a recorder, two woodwind instruments with distinct histories and playing techniques, and how do these differences affect the sound, repertoire, and role of each instrument in various musical traditions?",
    "Describe the diet of foxes, focusing on their role as opportunistic omnivores that can adapt to a wide range of food sources in different environments, and how their feeding behavior varies depending on the availability of prey and seasonal changes.",
    "How do foxes adapt to urban environments, where they often come into close contact with humans, and what behavioral and ecological strategies do they use to survive and thrive in cities and suburbs?",
    "What are the main threats to fox populations, including habitat loss, hunting, and disease, and what conservation efforts are being implemented to protect foxes and maintain their populations in the wild?"
]


FinalEvalR_mediumNN3, FinalEvalC_mediumNN3, FinalEvalF_mediumNN3, FinalResponseLength_mediumNN3 = ResponseEvaluationMetrics(Queries_method=medium_queriesNN3)

medium_queriesNN4 = [
    "Describe the influence of Latin on the French language, particularly in terms of vocabulary, syntax, and grammar, and how the transformation from Vulgar Latin to Old French was shaped by historical events such as the fall of the Roman Empire and the subsequent Frankish rule.",
    "How did French become a global language, particularly through the processes of colonization and cultural exchange, and what are the long-term effects of the spread of French on the linguistic landscapes of regions like Africa, Southeast Asia, and the Caribbean?",
    "What are the main differences between Parisian French and Quebec French in terms of pronunciation, vocabulary, and grammar, and how have historical, cultural, and social factors contributed to the divergence between these two major varieties of French?",
    "Describe the domestic policies of Gerald Ford's administration, focusing on how he addressed the economic challenges of the 1970s, including inflation, unemployment, and the energy crisis, as well as his efforts to restore public trust in the aftermath of Watergate.",
    "How did Gerald Ford handle the aftermath of the Watergate scandal, particularly in terms of his decision to pardon Richard Nixon, and what were the political and public reactions to this decision, both at the time and in the context of his legacy?",
    "What were the key challenges faced by Gerald Ford during his presidency, including domestic issues like economic instability and social unrest, as well as foreign policy challenges such as the Vietnam War and détente with the Soviet Union?",
    "Describe the differences between High German and Low German dialects, particularly in terms of phonology, vocabulary, and regional distribution, and how these dialectal variations have been influenced by historical, cultural, and social factors.",
    "How has the German language influenced English, particularly during the early medieval period through the Anglo-Saxon invasions, and what are some examples of Germanic loanwords and linguistic structures that have been integrated into English?",
    "What are the main grammatical features of German, particularly in terms of noun declensions, verb conjugations, and word order, and how do these features compare to those of other Germanic languages like Dutch and English?",
    "Describe the significance of the Ashanti Empire in Ghana's history, particularly in terms of its military, economic, and cultural achievements, and how its legacy continues to influence modern Ghanaian society and national identity.",
    "How did Ghana achieve independence from the United Kingdom, particularly through the efforts of leaders like Kwame Nkrumah and the broader nationalist movement, and what were the key challenges and successes of the post-independence period?",
    "What are the major economic activities in Ghana, including agriculture, mining, and services, and how do these sectors contribute to the country's GDP, employment, and international trade, particularly in the context of regional and global markets?",
    "Describe the habitat requirements of giant pandas, focusing on their reliance on bamboo forests in China's mountainous regions, and how changes in their habitat due to deforestation and climate change pose challenges to their survival.",
    "How do giant pandas contribute to their ecosystem, particularly in terms of their role as a flagship species for conservation efforts, and how their presence helps to protect the biodiversity of the temperate forests they inhabit?",
    "What are the main threats to giant panda populations, including habitat loss, low reproductive rates, and human-wildlife conflict, and what conservation strategies are being implemented to address these threats and ensure the species' survival?",
    "Describe the social structure of giraffe herds, focusing on the role of females in leading groups and the behavior of males during mating season, and how these social dynamics contribute to the survival and reproduction of giraffes in the wild.",
    "How do giraffes use their necks in combat during male-male competition, and how has this behavior influenced the evolution of their unique anatomy, particularly in terms of neck length and strength?",
    "What are the main threats to giraffe populations in the wild, including habitat loss, poaching, and human-wildlife conflict, and what conservation efforts are being undertaken to protect these iconic animals and their habitats?",
    "Describe the social structure of gray wolf packs, focusing on the roles of the alpha pair, subordinate members, and pups, and how this hierarchical organization supports the pack's hunting strategies, territorial defense, and overall survival.",
    "How do gray wolves communicate with each other through vocalizations, body language, and scent marking, and what role does communication play in coordinating group activities, maintaining social bonds, and establishing dominance within the pack?",
    "What are the main threats to gray wolf populations, including habitat fragmentation, human persecution, and prey depletion, and how are conservation efforts addressing these challenges to ensure the long-term survival of wolf populations?",
    "Describe the domestic policies of Grover Cleveland's administration, particularly in terms of his approach to economic issues like the tariff debate and monetary policy, and how his policies reflected his conservative Democratic values.",
    "How did Grover Cleveland handle economic issues during his presidency, particularly during the Panic of 1893, and what measures did his administration take to address the economic downturn and its impact on the American public?",
    "What were the key challenges faced by Grover Cleveland during his presidency, including issues related to labor unrest, economic depression, and foreign policy, and how did he respond to these challenges in a way that shaped his legacy?",
    "Describe the differences between acoustic and electric guitars, focusing on their construction, sound production, and use in different musical genres, and how these differences influence the playing techniques and musical expression of guitarists.",
    "How has the guitar evolved over time, particularly in terms of changes in design, materials, and playing styles, and how have these innovations contributed to the guitar's prominence in various musical traditions around the world?",
    "What are the main techniques used in playing the guitar, including fingerpicking, strumming, and the use of a plectrum, and how do these techniques vary depending on the genre of music being performed, from classical to rock and jazz?"
]


FinalEvalR_mediumNN4, FinalEvalC_mediumNN4, FinalEvalF_mediumNN4, FinalResponseLength_mediumNN4 = ResponseEvaluationMetrics(Queries_method=medium_queriesNN4)


medium_queriesNN5 = [
    "Describe the main themes in Gustav Klimt's artwork, particularly in relation to his exploration of the human psyche, the tension between eroticism and mortality, and how these themes are reflected in his use of symbolism and intricate patterns.",
    "How did Klimt's work influence the Vienna Secession movement, particularly in terms of his rejection of traditional academic art and his embrace of modernist principles, and how did this influence the broader development of modern art in Europe?",
    "What are the characteristics of Klimt's 'Golden Phase', particularly in terms of his use of gold leaf, ornamental patterns, and the incorporation of Byzantine and classical motifs, and how did this period reflect his evolving artistic vision?",
    "Describe the significance of Indonesia's independence from the Netherlands, focusing on the political, social, and economic challenges the country faced in the aftermath of colonial rule, and how these challenges shaped its modern identity.",
    "How does Indonesia's geography affect its cultural diversity, particularly in terms of the vast number of islands, ethnic groups, and languages, and how does this diversity contribute to both the unity and complexity of the Indonesian nation?",
    "What are the main economic activities in Indonesia, particularly in terms of agriculture, mining, and tourism, and how do these sectors contribute to the country's GDP, employment, and regional development?",
    "Describe the significance of Newton's laws of motion, particularly in terms of their impact on the development of classical mechanics, and how these laws provided a framework for understanding the physical behavior of objects in motion.",
    "How did Newton contribute to the field of optics, particularly through his work on the nature of light and color, and how did his experiments with prisms and lenses challenge the prevailing theories of his time?",
    "What were the main ideas in Newton's 'Principia Mathematica', particularly in terms of the laws of motion, universal gravitation, and calculus, and how did this work lay the foundation for modern physics and mathematics?",
    "Describe the main characteristics of Pollock's 'drip technique', particularly in terms of its spontaneous and dynamic nature, and how this technique revolutionized the process of painting by emphasizing the physical act of creation.",
    "How did Pollock's work influence abstract expressionism, particularly in terms of his emphasis on the subconscious mind, the use of non-representational forms, and the exploration of new techniques and materials?",
    "What were the personal struggles that Pollock faced during his life, particularly in terms of his battle with alcoholism and mental health issues, and how did these struggles influence the themes and intensity of his artwork?",
    "Describe the main principles of the Monroe Doctrine, particularly in terms of its assertion of American influence in the Western Hemisphere, and how this doctrine shaped U.S. foreign policy and its relationships with European powers.",
    "How did Monroe's presidency contribute to the Era of Good Feelings, particularly in terms of his efforts to promote national unity, economic growth, and political stability, and how did this period reflect the broader trends of early 19th-century America?",
    "What were the key challenges faced by Monroe during his presidency, including issues related to sectionalism, the economy, and foreign policy, and how did his administration address these challenges to maintain the country's stability and growth?",
    "Describe the significance of Watt's separate condenser in steam engines, particularly in terms of its impact on the efficiency and practicality of steam power, and how this innovation contributed to the broader industrialization of Britain.",
    "How did Watt's inventions contribute to the Industrial Revolution, particularly in terms of their applications in factories, transportation, and mining, and how did they help to transform the economic and social landscape of the 18th and 19th centuries?",
    "What were the main challenges Watt faced in developing his steam engine, including technical difficulties, financial constraints, and competition from other inventors, and how did he overcome these obstacles to achieve success?",
    "Describe the main grammatical features of Japanese, including its use of particles, honorifics, and complex verb conjugations, and how these features reflect the cultural and social norms of Japanese society.",
    "How has the Japanese language evolved over time, particularly in terms of its borrowing of Chinese characters, the development of Kana scripts, and the influence of Western languages, and how has this evolution shaped modern Japanese?",
    "What are the differences between Hiragana and Katakana, particularly in terms of their origins, usage, and the role they play in the Japanese writing system, and how do these scripts complement the use of Kanji in written Japanese?",
    "Describe the main domestic policies of John Adams' administration, particularly in terms of his approach to strengthening the federal government, managing foreign relations, and addressing the challenges of a young and growing nation.",
    "How did John Adams handle the XYZ Affair, particularly in terms of his efforts to avoid war with France while maintaining American sovereignty, and how did this diplomatic crisis influence the domestic and foreign policies of his administration?",
    "What were the key challenges faced by John Adams during his presidency, including political opposition, foreign threats, and internal divisions, and how did he address these challenges to uphold the principles of the American Revolution?"
]

FinalEvalR_mediumNN5, FinalEvalC_mediumNN5, FinalEvalF_mediumNN5, FinalResponseLength_mediumNN5 = ResponseEvaluationMetrics(Queries_method=medium_queriesNN5)



medium_queriesNN6 = [
    "Describe the locomotion of kangaroos, particularly focusing on their unique mode of hopping, the role of their powerful hind legs, and how this form of movement is adapted to the arid and open landscapes of Australia.",
    "How do kangaroos adapt to their environment, particularly in terms of their physiological and behavioral adaptations to seasonal changes, water scarcity, and the availability of food in Australia's diverse ecosystems?",
    "What are the main threats to kangaroo populations, including habitat loss, hunting, and climate change, and how do these threats impact the conservation efforts aimed at protecting these iconic marsupials?",
    "Describe the social behavior of koalas, particularly in terms of their solitary nature, territoriality, and the role of vocalizations in maintaining social boundaries within their eucalyptus forest habitats.",
    "How do koalas adapt to their environment, particularly in terms of their specialized diet of eucalyptus leaves, their slow metabolic rate, and their adaptations for conserving water in often hot and dry conditions?",
    "What are the main threats to koala populations, including habitat destruction, disease, and climate change, and how do these threats impact the ongoing conservation efforts to protect this vulnerable species?",
    "Describe the main grammatical features of Korean, including its use of honorifics, sentence structure, and the role of particles in conveying meaning, and how these features reflect the cultural values and social hierarchies of Korean society.",
    "How has the Korean language evolved over time, particularly in terms of its historical development from Old Korean to Modern Korean, and the influence of Chinese, Japanese, and Western languages on its vocabulary and grammar?",
    "What are the differences between Hangul and Hanja, particularly in terms of their origins, usage, and the roles they play in modern Korean writing, and how do these scripts reflect the linguistic and cultural history of Korea?",
    "Describe the main themes in Leonardo da Vinci's artwork, particularly in terms of his exploration of human anatomy, nature, and the use of perspective, and how these themes reflect the broader intellectual and artistic trends of the Renaissance.",
    "How did Leonardo da Vinci contribute to the field of anatomy, particularly through his detailed anatomical drawings and dissections, and how did his work advance the understanding of the human body in the context of Renaissance science?",
    "What were the main ideas in Leonardo da Vinci's notebooks, particularly in terms of his studies on mechanics, hydraulics, and flight, and how do these ideas reflect his multidisciplinary approach to understanding the natural world?",
    "Describe the social structure of leopards, particularly in terms of their solitary nature, territoriality, and the role of scent marking and vocalizations in maintaining social boundaries within their habitats.",
    "How do leopards adapt to their environment, particularly in terms of their camouflage, climbing ability, and opportunistic feeding habits, and how do these adaptations enable them to thrive in a wide range of ecosystems?",
    "What are the main threats to leopard populations, including habitat loss, poaching, and human-wildlife conflict, and how do these threats impact the conservation efforts aimed at protecting this vulnerable species?",
    "Describe the political system of Liechtenstein, particularly in terms of its constitutional monarchy, the role of the prince and parliament, and how this system of governance reflects the country's history and relationship with neighboring countries.",
    "How does Liechtenstein's economy compare to other European countries, particularly in terms of its financial services sector, manufacturing industry, and the role of international trade, and how does this economy support the country's high standard of living?",
    "What are the major industries in Liechtenstein, including financial services, manufacturing, and tourism, and how do these industries contribute to the country's GDP, employment, and economic stability?",
    "Describe the historical significance of Lima, particularly in terms of its role as the capital of the Spanish Viceroyalty of Peru, and how this colonial history has shaped the city's cultural and architectural heritage.",
    "How does Lima's economy contribute to Peru's GDP, particularly in terms of its role as the country's commercial and financial center, and how do key industries such as manufacturing, services, and tourism drive economic growth in the city?",
    "What are the main cultural activities in Lima, including festivals, culinary traditions, and the arts, and how do these activities reflect the diverse cultural influences and historical legacy of the city?",
    "Describe the social structure of lion prides, particularly in terms of the roles of males, females, and cubs, and how this social organization influences their hunting strategies, territory defense, and survival in the wild.",
    "How do lions communicate with each other, particularly through vocalizations, scent marking, and body language, and how do these forms of communication play a crucial role in maintaining social bonds and territorial boundaries within prides?",
    "What are the main threats to lion populations, including habitat loss, human-wildlife conflict, and poaching, and how do these threats impact the conservation efforts aimed at protecting this iconic species?",
    "Describe the habitat requirements of lizards, including the importance of temperature regulation, access to food, and shelter, and how these requirements influence their distribution and behavior in different environments.",
    "How do lizards adapt to their environment, particularly in terms of their thermoregulatory behaviors, camouflage, and feeding strategies, and how do these adaptations enable them to survive in a wide range of habitats?",
    "What are the main threats to lizard populations, including habitat destruction, climate change, and invasive species, and how do these threats impact the conservation efforts aimed at protecting this diverse group of reptiles?"
]


FinalEvalR_mediumNN6, FinalEvalC_mediumNN6, FinalEvalF_mediumNN6, FinalResponseLength_mediumNN6 = ResponseEvaluationMetrics(Queries_method=medium_queriesNN6)


medium_queriesNN7 = [
    "Describe the molting process in lobsters, particularly focusing on the physiological changes that occur during molting, the role of calcium in shell formation, and how this process is critical for growth and survival in the marine environment.",
    "How do lobsters adapt to their environment, particularly in terms of their behaviors, physical adaptations, and ecological roles in different marine ecosystems, and how do these adaptations contribute to their success as a species?",
    "What are the main threats to lobster populations, including overfishing, habitat destruction, and climate change, and how do these threats impact the sustainability of lobster fisheries and marine ecosystems?",
    "Describe the historical significance of the Tower of London, particularly in terms of its role as a royal palace, prison, and symbol of British history, and how its architecture reflects the changes in English society over the centuries.",
    "How does London's economy contribute to the UK's GDP, particularly in terms of its financial services, tourism, and cultural industries, and how do these sectors support the city's status as a global economic hub?",
    "What are the main cultural activities in London, including festivals, museums, and the arts, and how do these activities reflect the city's diverse population and rich cultural heritage?",
    "Describe the main structural components of a lyre, including its body, strings, and tuning mechanisms, and how these components work together to produce the instrument's characteristic sound.",
    "How did the lyre contribute to ancient Greek culture, particularly in terms of its use in religious ceremonies, entertainment, and education, and how does its symbolism reflect the values of Greek society?",
    "What are the differences between a lyre and a harp, particularly in terms of their construction, playing techniques, and roles in musical traditions, and how do these differences reflect their cultural significance?",
    "Describe the main grammatical features of Malay, including its word order, use of particles, and verb structure, and how these features reflect the linguistic diversity and history of the Malay-speaking world.",
    "How has the Malay language evolved over time, particularly in terms of its interactions with other languages, the influence of trade and religion, and the development of modern Malay as a national language?",
    "What are the differences between Malay and Indonesian, particularly in terms of vocabulary, pronunciation, and grammar, and how do these differences reflect the distinct cultural and political histories of Malaysia and Indonesia?",
    "Describe the main themes in Michelangelo's artwork, particularly in terms of his exploration of human anatomy, religious themes, and the use of perspective, and how these themes reflect the broader intellectual trends of the Renaissance.",
    "How did Michelangelo contribute to the Renaissance, particularly in terms of his innovations in sculpture, painting, and architecture, and how did his work influence the development of Western art?",
    "What were the main techniques used by Michelangelo in his sculptures, particularly in terms of his approach to carving marble, the use of contrapposto, and the portrayal of human emotion and movement?",
    "Describe the domestic policies of Millard Fillmore's administration, particularly in terms of his approach to issues such as slavery, economic development, and foreign relations, and how these policies reflected the political challenges of his time.",
    "How did Millard Fillmore handle the issue of slavery during his presidency, particularly in terms of his support for the Compromise of 1850, and how did this stance impact his political legacy and the tensions leading up to the Civil War?",
    "What were the key challenges faced by Millard Fillmore during his presidency, including domestic and foreign policy issues, and how did his administration respond to these challenges?",
    "Describe the main themes in Rockwell's artwork, particularly in terms of his depiction of American life, cultural values, and social issues, and how his work reflects the changing social landscape of 20th century America.",
    "How did Rockwell's work reflect American society, particularly in terms of his portrayal of everyday life, the American Dream, and the social challenges of his time, and how has his work influenced the perception of American culture?",
    "What were the main techniques used by Rockwell in his paintings, including his approach to composition, use of color, and attention to detail, and how do these techniques contribute to the realism and emotional impact of his work?",
    "Describe the intelligence of octopuses, particularly in terms of their problem-solving abilities, use of tools, and complex behaviors, and how these traits reflect their adaptability and survival strategies in the marine environment.",
    "How do octopuses adapt to their environment, particularly in terms of their camouflage, flexible body structure, and behavioral strategies, and how do these adaptations enable them to thrive in diverse marine habitats?",
    "What are the main threats to octopus populations, including overfishing, habitat destruction, and climate change, and how do these threats impact their survival and the ecological balance of marine ecosystems?",
    "Describe the social behavior of otters, particularly in terms of their group dynamics, communication methods, and cooperative behaviors, and how these social interactions contribute to their survival and success in aquatic environments.",
    "How do otters adapt to their environment, particularly in terms of their physical adaptations for swimming, foraging techniques, and use of tools, and how do these adaptations support their role as predators in aquatic ecosystems?",
    "What are the main threats to otter populations, including habitat loss, pollution, and hunting, and how do these threats impact the conservation efforts aimed at protecting these charismatic animals?"
]


FinalEvalR_mediumNN7, FinalEvalC_mediumNN7, FinalEvalF_mediumNN7, FinalResponseLength_mediumNN7 = ResponseEvaluationMetrics(Queries_method=medium_queriesNN7)


medium_queriesNN8 = [
    "Describe the main themes in Picasso's artwork, focusing on his exploration of form, space, and the human condition, and how these themes evolved throughout his career from the Blue Period to Cubism.",
    "How did Picasso's work influence modern art, particularly in terms of his innovations in Cubism, his approach to abstract representation, and his impact on subsequent movements such as Surrealism and Abstract Expressionism?",
    "What were the main periods of Picasso's artistic career, including the Blue Period, Rose Period, and Cubism, and how do these periods reflect his evolving artistic vision and response to personal and historical events?",
    "Describe the breeding behavior of penguins, particularly in terms of their courtship rituals, nesting practices, and parental care, and how these behaviors contribute to their reproductive success in harsh environments.",
    "How do penguins adapt to their environment, particularly in terms of their physiological, behavioral, and social adaptations to cold climates and their ability to thrive in the extreme conditions of the Antarctic?",
    "What are the main threats to penguin populations, including climate change, overfishing, and habitat destruction, and how do these threats impact their survival and the conservation efforts to protect them?",
    "Describe the mechanism of sound production in a piano, particularly in terms of the role of hammers, strings, and the soundboard, and how these components work together to produce the instrument's characteristic tone.",
    "How has the design of the piano evolved over time, particularly in terms of changes in string tension, keyboard range, and materials used, and how have these innovations influenced the instrument's role in music performance?",
    "What are the main differences between a grand piano and an upright piano, particularly in terms of their sound production, action mechanism, and use in different musical settings?",
    "Describe the main themes in Renoir's artwork, particularly in terms of his focus on light, color, and the depiction of everyday life, and how these themes reflect the broader goals of the Impressionist movement.",
    "How did Renoir's work influence the Impressionist movement, particularly in terms of his approach to capturing the effects of light and atmosphere, and his focus on the beauty of ordinary scenes and human interaction?",
    "What were the main techniques used by Renoir in his paintings, including his use of loose brushwork, vibrant color palettes, and attention to light, and how do these techniques contribute to the sense of immediacy and emotion in his work?",
    "Describe the main themes in Mondrian's artwork, particularly in terms of his exploration of abstraction, geometry, and the use of primary colors, and how these themes reflect his philosophical ideas about harmony and order.",
    "How did Mondrian's work influence modern art, particularly in terms of his contributions to abstract art and Neoplasticism, and his impact on the development of minimalism and geometric abstraction?",
    "What were the main techniques used by Mondrian in his paintings, including his use of grid structures, balanced compositions, and limited color palettes, and how do these techniques reflect his pursuit of universal harmony and simplicity?",
    "Describe the social behavior of polar bears, particularly in terms of their solitary nature, hunting strategies, and interactions with other polar bears, and how these behaviors contribute to their survival in the Arctic environment.",
    "How do polar bears adapt to their environment, particularly in terms of their physical adaptations for insulation, hunting, and swimming, and how do these adaptations enable them to thrive in the harsh conditions of the Arctic?",
    "What are the main threats to polar bear populations, including climate change, loss of sea ice habitat, and human activities, and how do these threats impact their survival and the efforts to conserve them?",
    "Describe the main grammatical features of Portuguese, particularly in terms of its verb conjugation, gender agreement, and sentence structure, and how these features reflect its Latin origins and linguistic evolution.",
    "How has the Portuguese language evolved over time, particularly in terms of its historical development, the influence of other languages, and the divergence between European and Brazilian Portuguese?",
    "What are the differences between European Portuguese and Brazilian Portuguese, particularly in terms of pronunciation, vocabulary, and grammar, and how do these differences reflect the distinct cultural and historical contexts of each variant?",
    "Describe the historical significance of Qatar's independence, particularly in terms of its transition from a British protectorate to a sovereign state, and how this event shaped its modern political and economic development.",
    "How does Qatar's economy compare to other Gulf countries, particularly in terms of its reliance on natural gas, economic diversification efforts, and investments in infrastructure and education?",
    "What are the main cultural activities in Qatar, including traditional festivals, sports, and arts, and how do these activities reflect the country's efforts to preserve its cultural heritage while embracing modernization?",
    "Describe the historical significance of Singapore's independence, particularly in terms of its separation from Malaysia, its development as a sovereign state, and its transformation into a global economic powerhouse.",
    "How does Singapore's economy compare to other Southeast Asian countries, particularly in terms of its focus on trade, finance, and technology, and its role as a regional hub for innovation and business?",
    "What are the main cultural activities in Singapore, including its multicultural festivals, arts scene, and culinary traditions, and how do these activities reflect the city's diverse population and global outlook?"
]


FinalEvalR_mediumNN8, FinalEvalC_mediumNN8, FinalEvalF_mediumNN8, FinalResponseLength_mediumNN8 = ResponseEvaluationMetrics(Queries_method=medium_queriesNN8)




medium_queriesNN9 = [
    "Describe the main grammatical features of Spanish, particularly in terms of verb conjugation, gender agreement, and the use of definite and indefinite articles, and how these features reflect the language's Latin roots.",
    "How has the Spanish language evolved over time, particularly in terms of the influence of other languages, regional dialects, and the standardization processes that have shaped modern Spanish?",
    "What are the differences between Castilian Spanish and Latin American Spanish, particularly in terms of pronunciation, vocabulary, and regional variations, and how do these differences reflect the cultural diversity of the Spanish-speaking world?",
    "Describe the main grammatical features of Swahili, particularly in terms of its noun classes, verb structure, and the influence of Arabic on its vocabulary, and how these features reflect its role as a lingua franca in East Africa.",
    "How has the Swahili language evolved over time, particularly in terms of its origins, the influence of trade and colonialism, and the development of standard Swahili in modern education and media?",
    "What are the differences between Swahili and other Bantu languages, particularly in terms of syntax, phonology, and lexical borrowing, and how do these differences reflect Swahili's unique position as a bridge language in East Africa?",
    "Describe the social behavior of swans, particularly in terms of their mating rituals, territoriality, and parental care, and how these behaviors contribute to their survival and reproductive success in aquatic environments.",
    "How do swans adapt to their environment, particularly in terms of their physical adaptations for swimming, feeding, and surviving in temperate and cold climates?",
    "What are the main threats to swan populations, including habitat destruction, pollution, and climate change, and how do these threats impact their conservation status and the efforts to protect them?",
    "Describe the main grammatical features of Swedish, particularly in terms of its word order, verb conjugation, and use of definite and indefinite forms, and how these features reflect the language's Germanic origins.",
    "How has the Swedish language evolved over time, particularly in terms of the influence of other languages, historical changes in vocabulary and grammar, and the role of standardization in education and media?",
    "What are the differences between Swedish and Danish, particularly in terms of pronunciation, vocabulary, and syntax, and how do these differences reflect the distinct cultural and historical contexts of each language?",
    "Describe the domestic policies of Theodore Roosevelt's administration, particularly in terms of his approach to trust-busting, conservation, and social reform, and how these policies reflect his progressive ideals.",
    "How did Theodore Roosevelt handle the issue of conservation during his presidency, particularly in terms of his establishment of national parks, forests, and wildlife refuges, and how did these efforts contribute to the early environmental movement?",
    "What were the key challenges faced by Theodore Roosevelt during his presidency, particularly in terms of his handling of foreign policy, labor disputes, and the balance of power between the executive and legislative branches?",
    "Describe the social behavior of tigers, particularly in terms of their territoriality, hunting strategies, and interactions with other tigers, and how these behaviors contribute to their survival as solitary apex predators.",
    "How do tigers adapt to their environment, particularly in terms of their physical adaptations for stealth, strength, and endurance, and how do these adaptations enable them to thrive in a variety of habitats, from forests to grasslands?",
    "What are the main threats to tiger populations, including habitat loss, poaching, and human-wildlife conflict, and how do these threats impact their survival and the conservation efforts to protect them?",
    "Describe the mechanism of sound production in a trumpet, particularly in terms of the vibration of the lips, the role of the mouthpiece, and the resonance of the tubing, and how these elements work together to produce the instrument's distinctive sound.",
    "How has the design of the trumpet evolved over time, particularly in terms of changes in valve mechanisms, bore size, and materials used, and how have these innovations influenced the instrument's versatility and use in different musical genres?",
    "What are the main differences between a trumpet and a cornet, particularly in terms of their shape, sound, and use in different musical settings, and how do these differences reflect their distinct roles in brass ensembles?",
    "Describe the main grammatical features of Turkish, particularly in terms of its agglutinative structure, vowel harmony, and the use of suffixes to indicate tense, mood, and case, and how these features reflect its Turkic origins.",
    "How has the Turkish language evolved over time, particularly in terms of the influence of Arabic and Persian, the language reforms of the 20th century, and the development of modern Turkish in education and media?",
    "What are the differences between Turkish and Azerbaijani, particularly in terms of pronunciation, vocabulary, and syntax, and how do these differences reflect the distinct cultural and historical contexts of each language?",
    "Describe the habitat requirements of turtles, particularly in terms of their need for access to water, basking sites, and nesting areas, and how these requirements vary across different species and environments.",
    "How do turtles adapt to their environment, particularly in terms of their physical adaptations for protection, feeding, and reproduction, and how do these adaptations enable them to survive in diverse habitats, from deserts to oceans?",
    "What are the main threats to turtle populations, including habitat destruction, climate change, and human exploitation, and how do these threats impact their survival and the conservation efforts to protect them?"
]


FinalEvalR_mediumNN9, FinalEvalC_mediumNN9, FinalEvalF_mediumNN9, FinalResponseLength_mediumNN9 = ResponseEvaluationMetrics(Queries_method=medium_queriesNN9)


medium_queriesNN10 = [
    "Describe the domestic policies of Ulysses S. Grant's administration, particularly in terms of his approach to Reconstruction, civil rights, and economic development, and how these policies reflected the challenges of the post-Civil War era.",
    "How did Grant handle the issue of Reconstruction during his presidency, particularly in terms of his efforts to protect the rights of freedmen, enforce federal laws in the South, and address the political and social challenges of reunifying the nation?",
    "What were the key challenges faced by Grant during his presidency, particularly in terms of corruption scandals, economic crises, and the ongoing struggles over Reconstruction, and how did these challenges impact his legacy?",
    "Describe the historical significance of Uruguay's independence, particularly in terms of its struggle for sovereignty from Spanish and Brazilian control, and how this history has shaped Uruguay's national identity and political development.",
    "How does Uruguay's economy compare to other South American countries, particularly in terms of its reliance on agriculture, exports, and its position within regional trade agreements such as MERCOSUR?",
    "What are the main cultural activities in Uruguay, particularly in terms of the country's traditions in music, dance, and literature, and how do these activities reflect Uruguay's diverse cultural heritage?",
    "Describe the main grammatical features of Vietnamese, particularly in terms of its tonal system, word order, and the use of classifiers, and how these features reflect the language's Austroasiatic roots and historical influences.",
    "How has the Vietnamese language evolved over time, particularly in terms of the influence of Chinese, French, and regional dialects, and how these influences have shaped modern Vietnamese as it is spoken today?",
    "What are the differences between Northern and Southern Vietnamese dialects, particularly in terms of pronunciation, vocabulary, and usage, and how do these differences reflect the historical and cultural diversity of Vietnam?",
    "Describe the main themes in van Gogh's artwork, particularly in terms of his exploration of color, emotion, and the human condition, and how these themes reflect his personal struggles and artistic vision.",
    "How did van Gogh's work influence modern art, particularly in terms of his impact on the development of expressionism, abstraction, and the use of color in conveying emotion and psychological depth?",
    "What were the main periods of van Gogh's artistic career, particularly in terms of his early works in the Netherlands, his time in Paris, and his final years in the south of France, and how do these periods reflect his artistic evolution?",
    "Describe the mechanism of sound production in a violin, particularly in terms of the interaction between the strings, bow, and body of the instrument, and how these elements work together to produce the violin's distinctive sound.",
    "How has the design of the violin evolved over time, particularly in terms of changes in shape, materials, and construction techniques, and how have these innovations influenced the instrument's sound and role in music?",
    "What are the main differences between a violin and a viola, particularly in terms of size, tuning, and sound, and how do these differences influence their roles in orchestral and chamber music settings?",
    "Describe the domestic policies of Woodrow Wilson's administration, particularly in terms of his approach to progressive reforms, economic regulation, and social issues, and how these policies reflected the challenges of the early 20th century.",
    "How did Wilson handle the issue of World War I during his presidency, particularly in terms of his efforts to maintain neutrality, his eventual decision to enter the war, and his role in shaping the post-war order through initiatives such as the League of Nations?",
    "What were the key challenges faced by Wilson during his presidency, particularly in terms of domestic opposition to his policies, the challenges of leading the nation during a global conflict, and the political and social changes of the Progressive Era?",
    "Describe the mechanism of sound production in a xylophone, particularly in terms of the vibration of the bars, the role of the resonators, and how these elements contribute to the instrument's unique tone and resonance.",
    "How has the design of the xylophone evolved over time, particularly in terms of changes in materials, tuning systems, and construction techniques, and how have these innovations influenced the instrument's use in different musical genres?",
    "What are the main differences between a xylophone and a marimba, particularly in terms of their range, tone, and construction, and how do these differences reflect their distinct roles in classical, jazz, and world music?",
    "Describe the social behavior of zebras, particularly in terms of their herd dynamics, communication methods, and interactions with other species, and how these behaviors contribute to their survival in the wild.",
    "How do zebras adapt to their environment, particularly in terms of their physical adaptations for grazing, avoiding predators, and coping with seasonal changes in their habitats?",
    "What are the main threats to zebra populations, including habitat loss, poaching, and competition with livestock, and how do these threats impact their conservation status and the efforts to protect them?"
]


FinalEvalR_mediumNN10, FinalEvalC_mediumNN10, FinalEvalF_mediumNN10, FinalResponseLength_mediumNN10 = ResponseEvaluationMetrics(Queries_method=medium_queriesNN10)


FinalR_mediumNNN=pd.concat([FinalEvalR_mediumNN1, FinalEvalR_mediumNN2, FinalEvalR_mediumNN3, FinalEvalR_mediumNN4, FinalEvalR_mediumNN5, FinalEvalR_mediumNN6, FinalEvalR_mediumNN7,
                         FinalEvalR_mediumNN8, FinalEvalR_mediumNN9, FinalEvalR_mediumNN10], axis=0)
FinalC_mediumNNN=pd.concat([FinalEvalC_mediumNN1, FinalEvalC_mediumNN2, FinalEvalC_mediumNN3, FinalEvalC_mediumNN4, FinalEvalC_mediumNN5, FinalEvalC_mediumNN6, FinalEvalC_mediumNN7,
                         FinalEvalC_mediumNN8, FinalEvalC_mediumNN9, FinalEvalC_mediumNN10], axis=0)

FinalF_mediumNNN=pd.concat([FinalEvalF_mediumNN1, FinalEvalF_mediumNN2, FinalEvalF_mediumNN3, FinalEvalF_mediumNN4, FinalEvalF_mediumNN5, FinalEvalF_mediumNN6, FinalEvalF_mediumNN7,
                         FinalEvalF_mediumNN8, FinalEvalF_mediumNN9, FinalEvalF_mediumNN10], axis=0)

FinalResponseLength_mediumNNN=pd.concat([FinalResponseLength_mediumNN1, FinalResponseLength_mediumNN2, FinalResponseLength_mediumNN3, FinalResponseLength_mediumNN4,
                                      FinalResponseLength_mediumNN5, FinalResponseLength_mediumNN6, FinalResponseLength_mediumNN7, FinalResponseLength_mediumNN8,
                                      FinalResponseLength_mediumNN9, FinalResponseLength_mediumNN10], axis=0)




In [ ]:

hard_queriesNN1 = [
    "Analyze Abraham Lincoln's impact on the Civil War strategies of the Union, including his decisions as Commander-in-Chief, his interactions with generals like Ulysses S. Grant and William Tecumseh Sherman, his role in the implementation of the Anaconda Plan, and his influence on military tactics that led to key victories such as the Battle of Gettysburg and the Siege of Vicksburg.",
    "Compare and contrast Abraham Lincoln's political views with those of his contemporaries, such as Stephen A. Douglas, Jefferson Davis, and Frederick Douglass, particularly regarding issues like slavery, states' rights, the preservation of the Union, and the concept of popular sovereignty, and how these differing views shaped the political landscape of the mid-19th century United States.",
    "Discuss the legal and constitutional challenges Abraham Lincoln faced during his presidency, including his suspension of habeas corpus, the use of executive orders like the Emancipation Proclamation, his efforts to pass the Thirteenth Amendment, his struggles with the Supreme Court, and the broader implications of his actions for the balance of powers between the executive and legislative branches.",
    "Describe the various roles ants play in their ecosystems, such as their contributions to soil aeration through their tunneling activities, seed dispersal (myrmecochory), predation on other insects, mutualistic relationships with aphids and plants, and their impact on nutrient cycling, and how these roles are vital for maintaining the health and stability of ecosystems in different biomes.",
    "Analyze the evolutionary adaptations that have made ants successful, including their complex social structures with division of labor, the development of specialized castes (workers, soldiers, queens), their ability to produce and respond to pheromones for communication, their use of chemical defenses like formic acid, and their ability to form symbiotic relationships with other species, such as fungi and plants.",
    "Compare the reproductive strategies of different ant species, such as the differences between monogynous (single queen) and polygynous (multiple queens) colonies, the various strategies ants use to establish new colonies through swarming or budding, and the role of nuptial flights in gene flow and population dynamics among ant colonies.",
    "Analyze the morphological structure of Arabic verbs, including the root-and-pattern system that underlies verb conjugations, the formation of derived forms (binyanim) to express different tenses, moods, and voices, the use of inflection to indicate person, number, and gender, and how these features are utilized in Classical Arabic, Modern Standard Arabic, and various dialects.",
    "Discuss the challenges and strategies in teaching Arabic as a second language, considering the complexities of the Arabic script (including the use of diacritics), the differences between spoken dialects and Modern Standard Arabic, the importance of mastering pronunciation and phonology, the role of immersion and cultural context in language acquisition, and the use of technology in language learning.",
    "Compare and contrast the various dialects of Arabic spoken across different regions, such as Egyptian Arabic, Levantine Arabic, and Gulf Arabic, including differences in pronunciation, vocabulary, grammar, and syntax, and how these dialects have been influenced by historical, social, and cultural factors, as well as the impact of media and migration on the spread and evolution of these dialects.",
    "Analyze the impact of Colony Collapse Disorder on global agriculture, including the economic consequences of declining bee populations for crop pollination, the potential risks to food security, the environmental factors contributing to CCD such as pesticide exposure, pathogens, and habitat loss, and the ongoing research into causes and potential solutions, including efforts to breed more resilient bee strains and improve beekeeping practices.",
    "Discuss the differences in behavior and social structure between honey bees and bumblebees, particularly in terms of their nesting habits (e.g., honeybees in hives versus bumblebees in ground nests), colony sizes, reproductive strategies, roles in pollination, and how these differences affect their interactions with plants and their contributions to ecosystems, as well as their responses to environmental stressors.",
    "Compare the efficiency of bees as pollinators to other insects, such as butterflies, beetles, and moths, considering factors like their foraging behavior, flower preferences, the mechanics of pollen transfer, the role of body size and hair density in pollen collection, and their importance in the pollination of agricultural crops and wild plants, as well as the economic value of their pollination services.",
    "Analyze the evolutionary adaptations of beetles that have led to their diversity, such as the development of specialized mouthparts for feeding (e.g., mandibles in carnivorous beetles, proboscis in nectar-feeding beetles), their hard exoskeletons (elytra) that provide protection and reduce water loss, their ability to undergo complete metamorphosis (holometabolism), and their capacity to exploit a wide range of ecological niches, from forests to deserts.",
    "Discuss the ecological impact of beetles on agriculture, including both the beneficial roles they play in pollination, pest control, and soil health, as well as the damage caused by species that are agricultural pests (e.g., Colorado potato beetle, boll weevil), the economic costs of pest control measures, and the challenges of balancing beetle conservation with the protection of crops.",
    "Compare the reproductive strategies of different beetle species, such as the variations in egg-laying practices (e.g., laying eggs in plant tissues, soil, or decaying wood), larval development (e.g., feeding on plants, prey, or detritus), the use of chemical defenses (e.g., toxic secretions) to protect offspring, and the role of parental care in species like burying beetles, and how these strategies contribute to the success and diversity of beetles.",
    "Analyze the role of Beijing in China's political landscape, including its function as the capital city and the seat of the central government, the location of major political institutions such as the National People's Congress and the Communist Party headquarters, its significance in shaping China's domestic and foreign policies, and its historical role as the center of Chinese civilization during various dynasties.",
    "Discuss the cultural significance of Beijing's traditional architecture, such as the siheyuan courtyards, the Forbidden City, and the Temple of Heaven, how these structures reflect the city's historical and cultural heritage, their role in Confucian and imperial rituals, and the challenges of preserving these sites in the face of rapid urbanization and modernization.",
    "Compare the urban development of Beijing with other major global cities, particularly in terms of infrastructure development, population growth, transportation networks (e.g., the Beijing Subway, high-speed rail), environmental challenges (e.g., air pollution, water scarcity), and the balance between modernization and the preservation of historical sites, and how these factors influence Beijing's global standing.",
    "Analyze the impact of habitat loss on butterfly populations, including the effects of deforestation, urbanization, and climate change on butterfly species and their migratory patterns, the role of conservation efforts in protecting critical habitats, the impact of invasive species on native butterfly populations, and the implications for biodiversity and ecosystem health.",
    "Discuss the differences in lifecycle and behavior between butterflies and moths, including their feeding habits, patterns of activity (diurnal vs. nocturnal), the role of mimicry and camouflage in predator avoidance, differences in wing structure and resting posture, and their roles in different ecosystems as pollinators, prey, and indicators of environmental health.",
    "Compare the migration patterns of different butterfly species, such as the Monarch butterfly (Danaus plexippus), the Painted Lady (Vanessa cardui), and the Red Admiral (Vanessa atalanta), including the distances they travel, the environmental cues that trigger migration, the challenges they face during migration (e.g., habitat loss, climate change), and the role of conservation initiatives in supporting their migratory routes."
]


FinalEvalR_hardNN1, FinalEvalC_hardNN1, FinalEvalF_hardNN1, FinalResponseLength_hardNN1 = ResponseEvaluationMetrics(Queries_method=hard_queriesNN1)


hard_queriesNN2 = [
    "Analyze the economic impact of Calvin Coolidge's tax policies, particularly the Revenue Acts of 1924 and 1926, which significantly reduced federal income taxes, and discuss how these policies influenced the economic expansion of the 1920s, the stock market boom, and the eventual onset of the Great Depression, considering the broader context of laissez-faire economics and fiscal conservatism during his administration.",
    "Compare Calvin Coolidge's presidency with that of his predecessor, Warren G. Harding, focusing on their approaches to government regulation, economic policy, and the handling of scandals, such as the Teapot Dome scandal, and how Coolidge's emphasis on honesty, frugality, and business-friendly policies differed from Harding's more relaxed and scandal-ridden administration.",
    "Discuss the long-term effects of Calvin Coolidge's administration on the Republican Party, particularly his influence on the party's ideology regarding limited government, fiscal conservatism, and support for business interests, and how his legacy shaped the party's platform in subsequent decades, including the New Deal era and the rise of modern conservatism.",
    "Analyze the impact of Canadian multiculturalism policies on its society, particularly the implementation of the Canadian Multiculturalism Act of 1988, which aimed to preserve and enhance cultural diversity, and discuss the challenges and successes of these policies in fostering social cohesion, combating racism and discrimination, and integrating immigrants into Canadian society.",
    "Discuss the role of Canada in international peacekeeping missions, including its contributions to United Nations peacekeeping operations since the 1950s, the development of the 'peacekeeping' concept by Canadian diplomat Lester B. Pearson, and the impact of these missions on Canada's international reputation, foreign policy, and military strategy.",
    "Compare and contrast the political systems of Canada and the United Kingdom, focusing on their shared history as part of the British Commonwealth, the similarities and differences in their parliamentary systems, the role of the monarchy in each country, and how historical events such as the Statute of Westminster and the Canada Act influenced their political development and sovereignty.",
    "Analyze the development of cello technique from the Baroque to the modern era, considering the contributions of key composers and performers, such as J.S. Bach, Ludwig van Beethoven, and Pablo Casals, and how advancements in instrument construction, changes in performance practice, and the evolution of musical styles have influenced the technical demands and expressive capabilities of the cello.",
    "Discuss the contributions of Pablo Casals to cello performance, particularly his revival of the Bach Cello Suites, his innovations in cello technique, his influence as a teacher, and his role in promoting the cello as a solo instrument, and how his legacy has shaped the careers of subsequent generations of cellists.",
    "Compare the use of the cello in classical and contemporary music, focusing on its role in traditional orchestral and chamber music versus its application in modern genres such as jazz, pop, and experimental music, and how composers and performers have expanded the expressive range and technical possibilities of the cello through innovative techniques and collaborations.",
    "Analyze the linguistic structure of Mandarin Chinese grammar, focusing on the role of word order (subject-verb-object), the use of particles to indicate tense and aspect, the lack of inflection for tense or number, and the importance of context and tone in conveying meaning, and how these features compare to other languages in the Sino-Tibetan language family.",
    "Discuss the challenges of translating Chinese idioms (chengyu) into English, considering the cultural, historical, and linguistic context in which these idioms developed, the difficulties in preserving their metaphorical and often concise nature, and the strategies translators use to convey their meaning and nuance in a language with a different grammatical and cultural framework.",
    "Compare the phonological systems of Mandarin and Cantonese, two major Chinese dialects, focusing on differences in tonal inventory, syllable structure, and vowel and consonant sounds, and how these phonological differences influence mutual intelligibility, regional identity, and the preservation of linguistic diversity within the broader Chinese-speaking community.",
    "Analyze the role of cougars in their ecosystems, particularly as apex predators that help regulate prey populations and maintain the balance of various species in their habitats, the impact of their hunting behavior on the distribution and behavior of prey species, and the broader ecological consequences of their decline or removal from certain areas.",
    "Discuss the conservation strategies employed to protect cougars, focusing on efforts to preserve and restore their habitats, reduce human-wildlife conflict, and promote coexistence through public education and wildlife corridors, and how these strategies address the challenges posed by habitat fragmentation, hunting, and urban expansion.",
    "Compare the behavior of cougars with that of other big cats like lions and tigers, focusing on differences in social structure (solitary versus social), hunting techniques, prey selection, territoriality, and interactions with humans, and how these behavioral differences reflect the ecological niches each species occupies within their respective environments.",
    "Analyze the evolution of cymbal manufacturing techniques, considering the historical development of cymbal production from ancient times to the present, the impact of technological advancements on the precision and consistency of cymbal crafting, and the role of artisans and manufacturers like Zildjian and Sabian in shaping the modern cymbal sound used in various musical genres.",
    "Discuss the cultural significance of cymbals in different musical traditions, from their use in religious and ceremonial contexts in ancient civilizations to their role in classical orchestras, jazz bands, and contemporary music, and how the symbolism, sound, and playing techniques of cymbals vary across cultures and musical styles.",
    "Compare the sound characteristics of different cymbal alloys, such as B20 bronze, B8 bronze, and nickel-silver, focusing on how the composition, thickness, and manufacturing process of each alloy affects the tone, sustain, and projection of the cymbal, and how these characteristics influence a musician's choice of cymbals for different musical contexts.",
    "Analyze the urban challenges faced by Dhaka, the capital of Bangladesh, due to rapid population growth, focusing on issues such as traffic congestion, air and water pollution, inadequate infrastructure, housing shortages, and the strain on public services, and how these challenges are being addressed through urban planning, policy reforms, and international cooperation.",
    "Discuss the impact of climate change on Dhaka's infrastructure, particularly the increased frequency of flooding, rising sea levels, and extreme weather events, and how these environmental challenges threaten the city's economy, public health, and social stability, and the measures being taken to enhance resilience through infrastructure improvements and disaster preparedness.",
    "Compare the development of Dhaka with that of other major South Asian cities, focusing on similarities and differences in urbanization patterns, economic growth, challenges related to poverty and inequality, and the role of government policies and international aid in shaping the trajectory of urban development in the region.",
    "Analyze the evolutionary adaptations of dragonflies for flight, focusing on the unique structure of their wings, the role of their compound eyes in detecting movement, their highly developed flight muscles, and their ability to hover, accelerate rapidly, and change direction in mid-air, and how these adaptations have made them effective predators and survivors in various environments.",
    "Discuss the cultural significance of dragonflies in different societies, from their symbolic meanings in Japanese and Native American cultures to their representation in art, literature, and folklore, and how these cultural perceptions have influenced attitudes towards dragonflies and their conservation in various regions.",
    "Compare the hunting strategies of dragonflies with those of other aerial predators, such as birds and bats, focusing on differences in hunting techniques, prey selection, sensory adaptations, and the ecological roles each plays in their respective ecosystems, and how these strategies contribute to the balance of their environments."
]


FinalEvalR_hardNN2, FinalEvalC_hardNN2, FinalEvalF_hardNN2, FinalResponseLength_hardNN2 = ResponseEvaluationMetrics(Queries_method=hard_queriesNN2)


hard_queriesNN3 = [
    "Analyze the migratory patterns of ducks in the Northern Hemisphere, focusing on the environmental factors that influence their seasonal movements, the challenges they face during migration, and the conservation efforts to protect critical stopover sites and wintering grounds.",
    "Discuss the economic importance of ducks in agriculture, particularly in the context of rice farming, where ducks are used in integrated pest management systems, and how these practices contribute to sustainable farming and rural livelihoods.",
    "Compare and contrast the plumage differences between male and female ducks, and how sexual dimorphism in color and patterning is related to mating strategies, predator avoidance, and environmental adaptation in different species.",
    "Analyze the political changes in Egypt over the past century, including the impact of colonialism, the rise of nationalism, and the transitions between monarchy, republicanism, and military rule, and how these changes have shaped modern Egyptian society and governance.",
    "Discuss the impact of tourism on Egypt's economy, focusing on the role of historical sites like the pyramids and the temples of Luxor in attracting international visitors, and how tourism contributes to both economic growth and cultural preservation.",
    "Compare the cultural differences between urban and rural areas in Egypt, including variations in social norms, family structures, and economic activities, and how these differences reflect the broader processes of modernization and globalization in the country.",
    "Analyze the use of color in El Greco's artwork, particularly his distinctive palette of vibrant blues, greens, and reds, and how his color choices enhance the emotional and spiritual intensity of his compositions, setting him apart from his contemporaries.",
    "Discuss the significance of El Greco's contributions to Renaissance art, particularly in the context of his innovative use of perspective, his synthesis of Byzantine and Western traditions, and his influence on later movements like Mannerism and Baroque.",
    "Compare El Greco's style with that of his contemporaries, such as Titian and Caravaggio, focusing on their differing approaches to composition, use of light and shadow, and the portrayal of religious and mythological themes.",
    "Analyze the role of elephants in their ecosystems, particularly as keystone species that influence the structure and composition of their habitats through their feeding behavior, seed dispersal, and creation of waterholes, and how their presence benefits other species.",
    "Discuss the conservation efforts to protect elephants, including anti-poaching initiatives, habitat restoration projects, and transboundary wildlife corridors, and the challenges these efforts face from human-wildlife conflict, illegal trade, and political instability.",
    "Compare the behaviors of African and Asian elephants, focusing on differences in social organization, communication, and responses to environmental pressures, and how these differences are shaped by their distinct evolutionary histories and habitats.",
    "Analyze the impact of English on linguistic diversity worldwide, including the spread of English as a global lingua franca, its role in displacing indigenous languages, and the cultural implications of its dominance in international communication.",
    "Discuss the challenges of teaching English as a second language, particularly in non-English-speaking countries, and how factors like cultural differences, linguistic distance, and access to resources affect language acquisition and proficiency.",
    "Compare the phonological differences between British and American English, focusing on variations in vowel pronunciation, consonant usage, and stress patterns, and how these differences reflect the historical and social developments of the two dialects.",
    "Analyze the impact of Finland's welfare state on its economy, including the effects of universal healthcare, education, and social security programs on economic growth, income inequality, and overall quality of life in Finland.",
    "Discuss the role of technology and innovation in Finland's development, focusing on key sectors like information technology, biotechnology, and renewable energy, and how Finland's investment in research and development has positioned it as a leader in these fields.",
    "Compare the cultural practices of the Sami people with those of other Finnish communities, particularly in terms of traditional livelihoods, language preservation, and the challenges of balancing cultural heritage with modernization.",
    "Analyze the grammatical structure of Finnish verbs, focusing on the use of inflection to convey tense, mood, person, and number, and how this complex system differs from that of other European languages, particularly those in the Indo-European family.",
    "Discuss the challenges of translating Finnish literature into English, particularly in capturing the nuances of Finnish grammar, vocabulary, and cultural references, and how these challenges affect the global reception of Finnish literary works.",
    "Compare the dialects of Finnish spoken in different regions, focusing on differences in pronunciation, vocabulary, and grammar, and how these dialects reflect the historical and cultural diversity within Finland.",
    "Analyze the acoustical properties of the flute, particularly how its cylindrical bore, key mechanism, and materials affect its sound production, range, and timbre, and how these properties have been optimized in modern flute design.",
    "Discuss the role of the flute in various musical traditions, from its use in Western classical music to its significance in folk and indigenous music around the world, and how the instrument’s versatility has contributed to its widespread popularity.",
    "Compare the techniques used in playing the Western concert flute and the Indian bansuri, focusing on differences in embouchure, finger positioning, and breath control, and how these techniques reflect the distinct musical traditions of each instrument.",
    "Analyze the role of foxes in folklore and culture, focusing on their symbolic representation as cunning and adaptable creatures in myths, stories, and religious beliefs across different societies, and how these representations influence human perceptions of foxes.",
    "Discuss the evolutionary adaptations of foxes for hunting, including their keen senses, stealthy movement, and opportunistic feeding behavior, and how these adaptations have enabled them to thrive in a wide range of environments.",
    "Compare the behaviors of different fox species in their natural habitats, focusing on variations in social organization, territoriality, and reproductive strategies, and how these behaviors are shaped by ecological factors and evolutionary pressures."
]


FinalEvalR_hardNN3, FinalEvalC_hardNN3, FinalEvalF_hardNN3, FinalResponseLength_hardNN3 = ResponseEvaluationMetrics(Queries_method=hard_queriesNN3)


hard_queriesNN4 = [
    "Analyze the impact of French on other languages worldwide, particularly through processes like language borrowing, creolization, and linguistic imperialism, and how French has influenced the development of languages in former French colonies and global cultural exchanges.",
    "Discuss the challenges of learning French for non-native speakers, particularly in terms of pronunciation, verb conjugation, and gendered nouns, and how these challenges vary depending on the learner's linguistic background and the teaching methods used.",
    "Compare the phonological systems of French and Italian, focusing on differences in vowel quality, stress patterns, and consonant clusters, and how these differences reflect the distinct historical and cultural developments of the two Romance languages.",
    "Analyze the impact of Gerald Ford's economic policies, particularly in terms of his efforts to curb inflation, stabilize the economy during the recession of the 1970s, and address the energy crisis, and how these policies influenced the economic trajectory of the United States.",
    "Discuss the significance of Gerald Ford's pardon of Richard Nixon, focusing on the legal, political, and ethical implications of the decision, and how it affected Ford's presidency, public opinion, and the broader process of national reconciliation after Watergate.",
    "Compare Gerald Ford's presidency with that of his successor, Jimmy Carter, focusing on their respective approaches to domestic and foreign policy challenges, leadership styles, and the legacies they left in American political history.",
    "Analyze the historical development of the German language, particularly in terms of its evolution from Old High German to Modern Standard German, and how factors like the Protestant Reformation, the printing press, and national unification influenced its standardization and spread.",
    "Discuss the challenges of learning German for English speakers, particularly in terms of complex grammar, gendered nouns, and case system, and how these challenges can be addressed through effective language learning strategies and resources.",
    "Compare the phonological systems of German and Dutch, focusing on differences in vowel sounds, consonant shifts, and stress patterns, and how these differences reflect the historical and regional divergence between the two closely related Germanic languages.",
    "Analyze the impact of Ghana's colonial history on its modern society, particularly in terms of the legacy of British rule on its political institutions, education system, and economic development, and how post-colonial Ghana has navigated these challenges in building a national identity.",
    "Discuss the role of Ghana in African politics, particularly as a pioneer of Pan-Africanism and a leader in the fight for independence across the continent, and how Ghana's foreign policy has evolved to reflect its changing role in regional and international affairs.",
    "Compare the cultural practices of different ethnic groups in Ghana, focusing on variations in language, religion, and social organization, and how these practices contribute to the rich cultural diversity and unity within the modern Ghanaian state.",
    "Analyze the conservation efforts to protect giant pandas, focusing on initiatives like habitat preservation, breeding programs, and international collaborations, and how these efforts have contributed to the stabilization and growth of panda populations in the wild and in captivity.",
    "Discuss the impact of climate change on giant panda habitats, particularly in terms of changes in bamboo availability, temperature, and precipitation patterns, and how these environmental shifts pose challenges to panda conservation and management strategies.",
    "Compare the behaviors of giant pandas in the wild and in captivity, focusing on differences in diet, social interactions, and reproductive success, and how these differences inform conservation practices and the ethical considerations of breeding programs.",
    "Analyze the evolutionary adaptations of giraffes for their height, particularly in terms of their cardiovascular system, neck vertebrae, and feeding behavior, and how these adaptations have enabled them to exploit ecological niches in the savannas of Africa.",
    "Discuss the role of giraffes in their ecosystems, particularly in terms of their impact on vegetation structure, seed dispersal, and interactions with other herbivores and predators, and how their presence influences the biodiversity and health of their habitats.",
    "Compare the different subspecies of giraffes and their geographic distributions, focusing on variations in coat pattern, size, and behavior, and how these differences reflect the evolutionary pressures and environmental conditions in different regions of Africa.",
    "Analyze the role of gray wolves in their ecosystems, particularly as apex predators that regulate prey populations and maintain the balance of the ecosystems they inhabit, and how their presence or absence affects trophic cascades and biodiversity.",
    "Discuss the conservation efforts to protect gray wolves, including reintroduction programs, legal protections, and public education campaigns, and the challenges these efforts face from human-wolf conflicts, habitat fragmentation, and political opposition.",
    "Compare the behaviors of gray wolves in different regions, focusing on variations in hunting strategies, social structure, and interactions with other species, and how these behaviors are shaped by environmental conditions and human activities.",
    "Analyze the impact of Grover Cleveland's economic policies, particularly in terms of his approach to tariffs, currency reform, and government spending, and how these policies reflected his conservative Democratic values and influenced the economic debates of his time.",
    "Discuss the significance of Grover Cleveland's vetoes, particularly in terms of his use of the veto power to oppose what he saw as unnecessary government spending and overreach, and how this aspect of his presidency shaped his reputation as a defender of limited government.",
    "Compare Grover Cleveland's presidency with that of his predecessors, focusing on differences in political philosophy, leadership style, and responses to the major issues of the day, and how these differences influenced the course of American history in the late 19th century.",
    "Analyze the role of the guitar in different musical genres, from classical to jazz to rock, and how its versatility, expressive range, and cultural significance have made it one of the most popular and influential instruments in the world.",
    "Discuss the impact of guitar technology on modern music, particularly in terms of the development of electric guitars, amplifiers, and effects pedals, and how these innovations have expanded the creative possibilities for musicians and transformed popular music.",
    "Compare the playing styles of different famous guitarists, focusing on differences in technique, tone, and musical interpretation, and how these styles have influenced the evolution of guitar music and the development of new genres and subgenres."
]


FinalEvalR_hardNN4, FinalEvalC_hardNN4, FinalEvalF_hardNN4, FinalResponseLength_hardNN4 = ResponseEvaluationMetrics(Queries_method=hard_queriesNN4)



hard_queriesNN5 = [
    "Analyze the impact of Gustav Klimt's work on modern art, particularly in terms of his influence on later movements such as Symbolism and Art Nouveau, and how his innovative use of color, form, and symbolism continues to inspire contemporary artists.",
    "Discuss the role of eroticism in Klimt's paintings, particularly in terms of its representation of human desire, vulnerability, and mortality, and how these themes are intertwined with his exploration of the sacred and the profane.",
    "Compare the techniques used by Klimt in his portraits and landscapes, focusing on differences in composition, use of color, and symbolic elements, and how these techniques reflect his evolving artistic vision and the cultural context of his time.",
    "Analyze the political changes in Indonesia since its independence, focusing on the transition from Sukarno's Guided Democracy to Suharto's New Order, and how these changes have shaped Indonesia's political landscape, governance, and civil society.",
    "Discuss the impact of tourism on Indonesia's economy, particularly in terms of its contributions to GDP, employment, and regional development, as well as the challenges it poses in terms of environmental sustainability and cultural preservation.",
    "Compare the cultural practices of different ethnic groups in Indonesia, focusing on variations in language, religion, and social organization, and how these practices contribute to the rich cultural diversity and unity within the modern Indonesian state.",
    "Analyze the impact of Newton's work on the Scientific Revolution, particularly in terms of his contributions to the fields of physics, mathematics, and astronomy, and how his ideas challenged the prevailing Aristotelian worldview and laid the foundation for modern science.",
    "Discuss the challenges Newton faced in his scientific career, including his disputes with other scientists, his struggles with mental health, and the societal expectations of his time, and how these challenges influenced his work and legacy.",
    "Compare the scientific contributions of Newton and Galileo, focusing on their respective roles in the development of modern physics, their methods of scientific inquiry, and the controversies they faced in challenging the established scientific and religious authorities.",
    "Analyze the impact of Jackson Pollock's work on modern art, particularly in terms of his role in the development of abstract expressionism, and how his innovative techniques and focus on the process of creation have influenced subsequent generations of artists.",
    "Discuss the role of automatism in Pollock's paintings, particularly in terms of its connection to Surrealist ideas of the subconscious mind, and how Pollock's use of spontaneous gestures and unplanned compositions challenged traditional notions of artistic control.",
    "Compare Pollock's techniques with those of other abstract expressionists, focusing on differences in their approaches to form, color, and composition, and how these differences reflect the diverse influences and personal styles within the movement.",
    "Analyze the impact of the Monroe Doctrine on American foreign policy, particularly in terms of its long-term implications for U.S. relations with Latin America and Europe, and how it has been interpreted and applied in different historical contexts.",
    "Discuss the significance of the Missouri Compromise during Monroe's presidency, particularly in terms of its role in managing sectional tensions over slavery, and how it foreshadowed the broader conflicts that would eventually lead to the Civil War.",
    "Compare Monroe's presidency with that of his predecessor, James Madison, focusing on differences in their approaches to foreign policy, economic issues, and national unity, and how these differences influenced the trajectory of the early American republic.",
    "Analyze the impact of James Watt's work on modern engineering, particularly in terms of his contributions to the development of steam power, and how his innovations laid the foundation for the industrialization of Europe and the global economy.",
    "Discuss the collaboration between James Watt and Matthew Boulton, particularly in terms of their partnership's impact on the commercialization and widespread adoption of steam engine technology, and how their collaboration exemplifies the synergy between innovation and entrepreneurship.",
    "Compare the steam engines of James Watt and Thomas Newcomen, focusing on differences in design, efficiency, and practical applications, and how Watt's improvements addressed the limitations of Newcomen's earlier model and revolutionized industrial processes.",
    "Analyze the influence of Chinese on the Japanese writing system, particularly in terms of the adaptation of Kanji and the development of Hiragana and Katakana, and how these scripts reflect the linguistic and cultural exchange between China and Japan.",
    "Discuss the challenges of translating Japanese literature into English, particularly in terms of preserving the nuances of the Japanese language, cultural context, and literary style, and how these challenges are addressed by translators and scholars.",
    "Compare the phonological systems of Japanese and Korean, focusing on differences in vowel harmony, consonant clusters, and intonation patterns, and how these differences reflect the distinct linguistic histories and cultural influences of the two languages.",
    "Analyze the impact of the Alien and Sedition Acts on Adams' presidency, particularly in terms of their influence on civil liberties, political opposition, and public opinion, and how these controversial laws shaped the legacy of Adams' administration.",
    "Discuss the significance of John Adams' contributions to American independence, particularly in terms of his role in drafting the Declaration of Independence, securing foreign alliances, and advocating for the principles of republican government during the Revolutionary War.",
    "Compare John Adams' presidency with that of his successor, Thomas Jefferson, focusing on differences in their political philosophies, leadership styles, and responses to key issues like foreign policy, domestic governance, and the balance of power between the federal and state governments."
]


FinalEvalR_hardNN5, FinalEvalC_hardNN5, FinalEvalF_hardNN5, FinalResponseLength_hardNN5 = ResponseEvaluationMetrics(Queries_method=hard_queriesNN5)



hard_queriesNN6 = [
    "Analyze the role of kangaroos in Australian culture, particularly in terms of their symbolic representation in indigenous mythology, national identity, and how they are featured in art, literature, and media.",
    "Discuss the impact of habitat loss on kangaroo populations, particularly in terms of urbanization, agricultural expansion, and climate change, and how these factors are driving changes in their distribution, behavior, and population dynamics.",
    "Compare the different species of kangaroos and their geographic distributions, focusing on the ecological niches they occupy, their adaptations to various environments, and how these differences contribute to the biodiversity of Australia's fauna.",
    "Analyze the conservation efforts to protect koalas, particularly in terms of habitat preservation, disease management, and public awareness campaigns, and how these efforts are addressing the threats posed by climate change, urbanization, and deforestation.",
    "Discuss the impact of climate change on koala habitats, particularly in terms of changes in temperature, rainfall patterns, and the availability of eucalyptus leaves, and how these changes are affecting koala populations and their long-term survival prospects.",
    "Compare the behaviors of koalas in the wild and in captivity, particularly in terms of their feeding habits, social interactions, and health outcomes, and how these differences provide insights into the challenges of conserving this species in its natural habitat.",
    "Analyze the influence of Chinese on the Korean writing system, particularly in terms of the historical introduction and adaptation of Chinese characters (Hanja) in Korea, and how this influence has shaped the development of the Korean language and literacy practices.",
    "Discuss the challenges of translating Korean literature into English, particularly in terms of preserving the nuances of the Korean language, cultural context, and literary style, and how these challenges are addressed by translators and scholars.",
    "Compare the phonological systems of Korean and Japanese, focusing on differences in vowel harmony, consonant clusters, and intonation patterns, and how these differences reflect the distinct linguistic histories and cultural influences of the two languages.",
    "Analyze the impact of Leonardo da Vinci's work on the Renaissance, particularly in terms of his contributions to art, science, and engineering, and how his multidisciplinary approach exemplified the Renaissance ideal of the 'universal man'.",
    "Discuss the challenges Leonardo da Vinci faced in his scientific career, particularly in terms of the societal and religious constraints of his time, and how these challenges influenced his work and legacy as both an artist and a scientist.",
    "Compare the artistic techniques of Leonardo da Vinci and Michelangelo, focusing on their approaches to human anatomy, use of perspective, and treatment of religious themes, and how these differences reflect their personal styles and the broader artistic trends of the High Renaissance.",
    "Analyze the role of leopards in their ecosystems, particularly in terms of their role as apex predators, their impact on prey populations, and how their behavior and habitat use contribute to the overall biodiversity and stability of their ecosystems.",
    "Discuss the conservation efforts to protect leopards, particularly in terms of anti-poaching initiatives, habitat preservation, and community engagement programs, and how these efforts are addressing the threats posed by habitat loss, human-wildlife conflict, and illegal wildlife trade.",
    "Compare the behaviors of leopards in different regions, focusing on differences in their hunting strategies, social interactions, and habitat use, and how these differences reflect the adaptability and resilience of this species in diverse environments.",
    "Analyze the impact of Liechtenstein's banking sector on its economy, particularly in terms of its role as a financial hub, the regulatory framework that supports the sector, and how this industry contributes to the country's GDP, employment, and international reputation.",
    "Discuss the role of Liechtenstein in international relations, particularly in terms of its diplomatic relationships, participation in international organizations, and how its status as a small, neutral country influences its foreign policy and international standing.",
    "Compare the cultural practices of Liechtenstein with those of neighboring countries, focusing on differences and similarities in language, traditions, and social norms, and how these cultural exchanges have shaped the identity and heritage of Liechtenstein.",
    "Analyze the urban development of Lima over the past century, particularly in terms of its growth as a metropolitan area, the challenges of urban planning, and how these developments have influenced the city's infrastructure, economy, and social dynamics.",
    "Discuss the impact of tourism on Lima's economy, particularly in terms of its contributions to GDP, employment, and cultural exchange, as well as the challenges it poses in terms of environmental sustainability and cultural preservation.",
    "Compare the social issues in Lima with those in other major South American cities, focusing on challenges such as urban poverty, inequality, and access to public services, and how these issues are being addressed through public policy and community initiatives.",
    "Analyze the role of lions in their ecosystems, particularly in terms of their role as apex predators, their impact on prey populations, and how their behavior and social structure contribute to the overall biodiversity and stability of their ecosystems.",
    "Discuss the conservation efforts to protect lions, particularly in terms of anti-poaching initiatives, habitat preservation, and community engagement programs, and how these efforts are addressing the threats posed by habitat loss, human-wildlife conflict, and illegal wildlife trade.",
    "Compare the behaviors of lions in different regions, focusing on differences in their social structures, hunting strategies, and interactions with other species, and how these differences reflect the adaptability and resilience of this species in diverse environments.",
    "Analyze the evolutionary adaptations of lizards, particularly in terms of their physiological and behavioral traits that enable them to thrive in a wide range of environments, and how these adaptations have contributed to their diversification and ecological success.",
    "Discuss the role of lizards in their ecosystems, particularly in terms of their roles as predators, prey, and contributors to soil health and pest control, and how their interactions with other species influence the dynamics of their ecosystems.",
    "Compare the behaviors of different lizard species in their natural habitats, focusing on differences in their feeding strategies, reproductive behaviors, and responses to environmental challenges, and how these differences reflect the ecological diversity of this group of reptiles."
]

FinalEvalR_hardNN6, FinalEvalC_hardNN6, FinalEvalF_hardNN6, FinalResponseLength_hardNN6 = ResponseEvaluationMetrics(Queries_method=hard_queriesNN6)


hard_queriesNN7 = [
    "Analyze the impact of lobster fishing on marine ecosystems, particularly in terms of its effects on lobster populations, the health of benthic habitats, and the broader implications for marine biodiversity and food webs.",
    "Discuss the economic importance of lobsters in global seafood markets, including the economic value of lobster fisheries, trade dynamics, and the challenges of ensuring sustainable practices in the face of increasing demand and environmental change.",
    "Compare the anatomical differences between clawed lobsters and spiny lobsters, particularly in terms of their exoskeleton structure, claw morphology, and sensory adaptations, and how these differences reflect their ecological roles and evolutionary histories.",
    "Analyze the impact of the Industrial Revolution on London's development, particularly in terms of its urban expansion, the transformation of its economic base, and the social and environmental challenges that arose during this period.",
    "Discuss the role of London in global finance, particularly in terms of its position as a financial center, the development of its financial institutions, and the impact of globalization on its economy and influence.",
    "Compare the social issues in London with those in other major European cities, focusing on challenges such as housing affordability, income inequality, and social integration, and how these issues are being addressed through policy and urban planning.",
    "Analyze the evolution of the lyre in different cultures, particularly in terms of its variations in design, use in religious and cultural practices, and how its symbolic meaning has evolved over time in different societies.",
    "Discuss the role of the lyre in ancient rituals and ceremonies, particularly in terms of its use in religious worship, funerary practices, and cultural celebrations, and how these functions reflect the spiritual and social values of ancient civilizations.",
    "Compare the playing techniques of the lyre and the lute, particularly in terms of their string configurations, tuning methods, and the role of the performer in creating music, and how these differences reflect the instruments' cultural contexts.",
    "Analyze the influence of Arabic on the Malay language, particularly in terms of the introduction of loanwords, the influence of Islamic scholarship, and how these linguistic exchanges have shaped the development of Malay over time.",
    "Discuss the challenges of translating Malay literature into English, particularly in terms of preserving the nuances of Malay idioms, cultural references, and the linguistic diversity within Malay, and how these challenges are addressed by translators.",
    "Compare the phonological systems of Malay and Javanese, focusing on differences in vowel harmony, consonant clusters, and intonation patterns, and how these differences reflect the distinct linguistic and cultural histories of the two languages.",
    "Analyze the impact of Michelangelo's work on art history, particularly in terms of his innovations in sculpture, painting, and architecture, and how his work influenced the development of Western art and the Renaissance ideals of beauty and humanism.",
    "Discuss the challenges Michelangelo faced during his career, particularly in terms of the political and religious pressures of his time, and how these challenges influenced his artistic choices and legacy.",
    "Compare the artistic techniques of Michelangelo and Leonardo da Vinci, focusing on their approaches to depicting the human form, use of perspective, and treatment of religious themes, and how these differences reflect their personal styles and the broader artistic trends of the Renaissance.",
    "Analyze the impact of Millard Fillmore's foreign policies, particularly in terms of his approach to trade, diplomacy, and international relations, and how these policies influenced the United States' position in the global community.",
    "Discuss the significance of the Compromise of 1850 during Fillmore's presidency, particularly in terms of its impact on the sectional tensions between the North and South, and how it influenced the political landscape leading up to the Civil War.",
    "Compare Fillmore's presidency with that of his predecessor, Zachary Taylor, particularly in terms of their approaches to domestic and foreign policy, and how their leadership styles and political ideologies influenced the direction of the country.",
    "Analyze the impact of Norman Rockwell's work on American culture, particularly in terms of his portrayal of American life, values, and social issues, and how his illustrations have influenced the nation's cultural identity and artistic heritage.",
    "Discuss the challenges Rockwell faced during his career, particularly in terms of balancing artistic integrity with commercial success, and how these challenges influenced the themes and style of his work.",
    "Compare the artistic styles of Norman Rockwell and other American illustrators, focusing on differences in their approaches to composition, use of color, and depiction of American life, and how these differences reflect their individual artistic visions and the broader trends in American art.",
    "Analyze the role of octopuses in marine ecosystems, particularly in terms of their role as predators, their interactions with prey and other species, and how their behaviors and adaptations contribute to the ecological balance of their habitats.",
    "Discuss the significance of octopus camouflage abilities, particularly in terms of their use of chromatophores, behavioral adaptations, and neural control, and how these abilities have evolved to enhance their survival and hunting strategies in diverse marine environments.",
    "Compare the behaviors of different octopus species in their natural habitats, focusing on differences in their hunting strategies, social interactions, and responses to environmental challenges, and how these differences reflect the ecological diversity of this group of cephalopods.",
    "Analyze the role of otters in their ecosystems, particularly in terms of their role as apex predators, their impact on prey populations, and how their behaviors and habitat use contribute to the overall biodiversity and stability of their ecosystems.",
    "Discuss the conservation efforts to protect otters, particularly in terms of anti-poaching initiatives, habitat preservation, and community engagement programs, and how these efforts are addressing the threats posed by habitat loss, pollution, and human-wildlife conflict.",
    "Compare the behaviors of different otter species in their natural habitats, focusing on differences in their social structures, foraging strategies, and interactions with other species, and how these differences reflect the adaptability and resilience of this group of mammals."
]


FinalEvalR_hardNN7, FinalEvalC_hardNN7, FinalEvalF_hardNN7, FinalResponseLength_hardNN7 = ResponseEvaluationMetrics(Queries_method=hard_queriesNN7)


hard_queriesNN8 = [
    "Analyze the impact of Cubism on Picasso's work, particularly in terms of his deconstruction of form, use of fragmented perspectives, and how this movement challenged traditional notions of representation in art.",
    "Discuss the significance of Picasso's 'Guernica,' particularly in terms of its depiction of the horrors of war, its political and social commentary, and its influence on both contemporary and later art and activism.",
    "Compare the techniques used by Picasso and Georges Braque in their Cubist works, particularly in terms of their approach to breaking down forms, the use of collage, and how their collaboration and competition shaped the development of Cubism.",
    "Analyze the role of penguins in their ecosystems, particularly in terms of their position as predators and prey, their impact on marine food webs, and how changes in their populations affect the broader ecosystem.",
    "Discuss the impact of climate change on penguin habitats, particularly in terms of changes in sea ice extent, food availability, and breeding success, and how these factors influence their long-term survival.",
    "Compare the behaviors of different penguin species in their natural habitats, particularly in terms of their breeding, feeding, and social interactions, and how these behaviors reflect the ecological diversity of this avian group.",
    "Analyze the role of the piano in classical music, particularly in terms of its use in solo performance, chamber music, and orchestral settings, and how the instrument's versatility has influenced its prominence in Western music.",
    "Discuss the impact of piano technology on modern music, particularly in terms of innovations in digital pianos, sound synthesis, and how these advancements have expanded the possibilities for piano performance and composition.",
    "Compare the playing techniques of different famous pianists, particularly in terms of their approach to touch, phrasing, and interpretation, and how these techniques reflect their individual artistic personalities and contributions to the piano repertoire.",
    "Analyze the impact of Renoir's work on modern art, particularly in terms of his influence on the development of color theory, his approach to capturing light and movement, and his legacy in the transition from Impressionism to modernism.",
    "Discuss the challenges Renoir faced during his career, particularly in terms of his struggles with arthritis, his evolving artistic style, and how these challenges influenced his later works and reputation.",
    "Compare the artistic techniques of Renoir and Claude Monet, particularly in terms of their use of color, light, and brushwork, and how these techniques reflect their individual interpretations of the Impressionist aesthetic.",
    "Analyze the impact of Mondrian's work on abstract art, particularly in terms of his development of Neoplasticism, his influence on the De Stijl movement, and his legacy in the evolution of geometric abstraction and minimalism.",
    "Discuss the significance of Mondrian's 'Composition with Red, Blue, and Yellow,' particularly in terms of its use of primary colors, grid structure, and how it exemplifies his quest for universal harmony and balance in art.",
    "Compare the artistic techniques of Mondrian and Theo van Doesburg, particularly in terms of their approach to abstraction, use of color, and how their collaboration and differences shaped the development of the De Stijl movement.",
    "Analyze the role of polar bears in their ecosystems, particularly in terms of their impact on prey populations, their role as apex predators, and how their behavior and ecology influence the Arctic food web.",
    "Discuss the impact of climate change on polar bear habitats, particularly in terms of the loss of sea ice, changes in prey availability, and how these environmental changes threaten their survival and conservation efforts.",
    "Compare the behaviors of polar bears in different regions, particularly in terms of their hunting strategies, denning practices, and how regional differences in climate and habitat influence their behavior and ecology.",
    "Analyze the influence of Latin on the Portuguese language, particularly in terms of its lexical, phonological, and grammatical features, and how the language's evolution reflects the broader history of Romance languages.",
    "Discuss the challenges of translating Portuguese literature into English, particularly in terms of capturing the nuances of idiomatic expressions, cultural references, and the distinct linguistic features of Portuguese.",
    "Compare the phonological systems of Portuguese and Spanish, particularly in terms of their vowel systems, consonant clusters, and stress patterns, and how these similarities and differences reflect their shared Latin roots.",
    "Analyze the impact of Qatar's natural gas reserves on its economy, particularly in terms of revenue generation, economic diversification, and how these resources have shaped its development and geopolitical influence.",
    "Discuss the role of Qatar in international politics, particularly in terms of its foreign policy strategies, mediation efforts in regional conflicts, and its influence within international organizations such as the United Nations and OPEC.",
    "Compare the social development of Qatar with that of neighboring countries, particularly in terms of education, healthcare, and social welfare programs, and how these factors contribute to its human development and social stability.",
    "Analyze the impact of Singapore's port on its economy, particularly in terms of trade, logistics, and how its strategic location has positioned Singapore as a global shipping hub and economic powerhouse.",
    "Discuss the role of Singapore in international trade, particularly in terms of its free trade agreements, its role in global supply chains, and how its economic policies have facilitated its integration into the global economy.",
    "Compare the social development of Singapore with that of neighboring countries, particularly in terms of its education system, healthcare, and housing policies, and how these factors contribute to its high standard of living and social cohesion."
]


FinalEvalR_hardNN8, FinalEvalC_hardNN8, FinalEvalF_hardNN8, FinalResponseLength_hardNN8 = ResponseEvaluationMetrics(Queries_method=hard_queriesNN8)



hard_queriesNN9 = [
    "Analyze the influence of Arabic on the Spanish language, particularly in terms of lexical borrowing, phonological changes, and the impact of the Islamic presence in Spain on the development of the Spanish lexicon.",
    "Discuss the challenges of translating Spanish literature into English, particularly in terms of capturing the nuances of Spanish idiomatic expressions, cultural references, and the distinct linguistic features of Spanish.",
    "Compare the phonological systems of Spanish and Portuguese, particularly in terms of their vowel systems, consonant clusters, and stress patterns, and how these similarities and differences reflect their shared Latin roots.",
    "Analyze the influence of Arabic on the Swahili language, particularly in terms of its lexical and syntactic features, and how the historical interactions between Arab traders and East African communities shaped the development of Swahili.",
    "Discuss the challenges of translating Swahili literature into English, particularly in terms of preserving the cultural context, idiomatic expressions, and the unique syntactic structures of Swahili.",
    "Compare the phonological systems of Swahili and Somali, particularly in terms of their consonant inventories, vowel harmony, and tonal patterns, and how these similarities and differences reflect their distinct linguistic and cultural histories.",
    "Analyze the role of swans in their ecosystems, particularly in terms of their impact on aquatic vegetation, their role as both predators and prey, and how their migratory patterns influence the ecological balance of wetland habitats.",
    "Discuss the significance of swans in mythology and culture, particularly in terms of their symbolic associations with beauty, purity, and transformation, and how these cultural meanings have been expressed in literature, art, and folklore.",
    "Compare the behaviors of different swan species in their natural habitats, particularly in terms of their mating rituals, territoriality, and parental care, and how these behaviors reflect the ecological diversity of the Anatidae family.",
    "Analyze the influence of German on the Swedish language, particularly in terms of lexical borrowing, syntactic structures, and the historical impact of the Hanseatic League and other German-speaking communities on the development of Swedish.",
    "Discuss the challenges of translating Swedish literature into English, particularly in terms of capturing the nuances of Swedish idiomatic expressions, cultural references, and the distinct linguistic features of Swedish.",
    "Compare the phonological systems of Swedish and Norwegian, particularly in terms of their vowel systems, consonant clusters, and intonation patterns, and how these similarities and differences reflect their shared North Germanic roots.",
    "Analyze the impact of Theodore Roosevelt's foreign policies, particularly in terms of his role in negotiating the end of the Russo-Japanese War, his approach to the construction of the Panama Canal, and how his policies shaped the U.S.'s role as a global power.",
    "Discuss the significance of the Panama Canal during Roosevelt's presidency, particularly in terms of its strategic importance for global trade, its impact on U.S.-Latin American relations, and the engineering challenges involved in its construction.",
    "Compare Roosevelt's presidency with that of his successor, William Howard Taft, particularly in terms of their approaches to trust-busting, conservation, and foreign policy, and how these differences reflect their distinct leadership styles and political philosophies.",
    "Analyze the role of tigers in their ecosystems, particularly in terms of their impact on prey populations, their role as apex predators, and how their presence or absence influences the ecological balance of their habitats.",
    "Discuss the impact of poaching on tiger populations, particularly in terms of the demand for tiger parts in traditional medicine, the challenges of enforcing anti-poaching laws, and the conservation strategies employed to protect these endangered animals.",
    "Compare the behaviors of different tiger subspecies in their natural habitats, particularly in terms of their hunting strategies, territoriality, and social interactions, and how these behaviors reflect the ecological diversity of the Panthera tigris species.",
    "Analyze the role of the trumpet in classical music, particularly in terms of its use in orchestral and chamber music, its evolution from natural trumpets to valved instruments, and its influence on the development of Western music.",
    "Discuss the impact of trumpet technology on modern music, particularly in terms of the development of new materials, valve mechanisms, and the use of electronic trumpets in contemporary music genres.",
    "Compare the playing techniques of different famous trumpeters, particularly in terms of their approach to tone production, articulation, and interpretation, and how these techniques reflect their individual artistic styles and contributions to the trumpet repertoire.",
    "Analyze the influence of Arabic on the Turkish language, particularly in terms of lexical borrowing, the adoption of Arabic script before the Latin alphabet reform, and the impact of Islamic culture on the development of Turkish.",
    "Discuss the challenges of translating Turkish literature into English, particularly in terms of preserving the cultural context, idiomatic expressions, and the unique syntactic structures of Turkish.",
    "Compare the phonological systems of Turkish and Persian, particularly in terms of their vowel harmony, consonant clusters, and stress patterns, and how these similarities and differences reflect their distinct linguistic and cultural histories.",
    "Analyze the evolutionary adaptations of turtles, particularly in terms of their shell structure, respiratory systems, and reproductive strategies, and how these adaptations have enabled them to survive in diverse environments over millions of years.",
    "Discuss the role of turtles in their ecosystems, particularly in terms of their impact on vegetation, their role as both predators and prey, and how their presence or absence influences the ecological balance of their habitats.",
    "Compare the behaviors of different turtle species in their natural habitats, particularly in terms of their feeding habits, nesting behaviors, and social interactions, and how these behaviors reflect the ecological diversity of the Testudines order."
]


FinalEvalR_hardNN9, FinalEvalC_hardNN9, FinalEvalF_hardNN9, FinalResponseLength_hardNN9 = ResponseEvaluationMetrics(Queries_method=hard_queriesNN9)




hard_queriesNN10 = [
    "Analyze the impact of Grant's foreign policies, particularly in terms of his efforts to expand U.S. influence in the Caribbean and Latin America, and how these policies reflected the broader goals of American expansionism in the post-Civil War era.",
    "Discuss the significance of the Civil Rights Act of 1875 during Grant's presidency, particularly in terms of its provisions for equal rights and the challenges of enforcing civil rights legislation in the face of growing opposition from Southern states.",
    "Compare Grant's presidency with that of his predecessor, Andrew Johnson, particularly in terms of their approaches to Reconstruction, civil rights, and relations with Congress, and how these differences reflected the political and social challenges of the time.",
    "Analyze the impact of agriculture on Uruguay's economy, particularly in terms of the country's reliance on livestock, grain production, and exports, and how these sectors contribute to Uruguay's position within the global agricultural market.",
    "Discuss the role of Uruguay in international politics, particularly in terms of its contributions to peacekeeping, its leadership in regional organizations such as MERCOSUR, and its diplomatic relations with other nations.",
    "Compare the social development of Uruguay with that of neighboring countries, particularly in terms of education, healthcare, and social welfare, and how these factors contribute to Uruguay's reputation as one of South America's most stable and prosperous nations.",
    "Analyze the influence of Chinese on the Vietnamese language, particularly in terms of lexical borrowing, the use of Chinese characters in classical texts, and how these influences reflect the historical and cultural interactions between China and Vietnam.",
    "Discuss the challenges of translating Vietnamese literature into English, particularly in terms of preserving the cultural context, idiomatic expressions, and the unique syntactic structures of Vietnamese.",
    "Compare the phonological systems of Vietnamese and Thai, particularly in terms of their tonal systems, vowel inventories, and consonant clusters, and how these similarities and differences reflect their distinct linguistic and cultural histories.",
    "Analyze the impact of van Gogh's mental health on his work, particularly in terms of how his struggles with depression, anxiety, and psychosis influenced the themes, colors, and emotional intensity of his paintings.",
    "Discuss the significance of van Gogh's 'Starry Night', particularly in terms of its use of color, composition, and symbolic elements, and how the painting reflects van Gogh's unique artistic vision and personal experiences.",
    "Compare the techniques used by van Gogh and Paul Gauguin, particularly in terms of their approaches to color, brushwork, and composition, and how their artistic collaboration and eventual conflict influenced their respective styles.",
    "Analyze the role of the violin in classical music, particularly in terms of its use as a solo instrument, its role in orchestral and chamber music, and how its repertoire has evolved from the Baroque period to the present day.",
    "Discuss the impact of violin technology on modern music, particularly in terms of the development of electric violins, amplification techniques, and how these innovations have expanded the violin's role in genres such as rock, jazz, and electronic music.",
    "Compare the playing techniques of different famous violinists, particularly in terms of their approaches to tone production, articulation, and interpretation, and how these techniques reflect their individual artistic styles and contributions to the violin repertoire.",
    "Analyze the impact of Wilson's foreign policies, particularly in terms of his role in shaping the post-World War I order, his promotion of the League of Nations, and how his policies influenced the development of U.S. diplomacy in the 20th century.",
    "Discuss the significance of the League of Nations during Wilson's presidency, particularly in terms of its goals, successes, and failures, and how Wilson's advocacy for the League reflected his vision for a new international order.",
    "Compare Wilson's presidency with that of his predecessor, William Howard Taft, particularly in terms of their approaches to foreign policy, domestic reform, and the challenges of leading the nation during a time of global conflict.",
    "Analyze the role of the xylophone in orchestral music, particularly in terms of its use in modern compositions, its contribution to the texture and color of the orchestra, and how its role has evolved from the 19th century to the present day.",
    "Discuss the impact of xylophone technology on modern music, particularly in terms of the development of synthetic materials, electronic xylophones, and how these innovations have expanded the instrument's versatility and use in contemporary genres.",
    "Compare the playing techniques of different famous xylophonists, particularly in terms of their approaches to mallet selection, articulation, and interpretation, and how these techniques reflect their contributions to the development of the xylophone repertoire.",
    "Analyze the role of zebras in their ecosystems, particularly in terms of their impact on grassland dynamics, their role as prey for large carnivores, and how their grazing behavior influences plant community structure.",
    "Discuss the impact of habitat loss on zebra populations, particularly in terms of the challenges of conservation in fragmented landscapes, the effects of human-wildlife conflict, and the strategies employed to protect zebra habitats.",
    "Compare the behaviors of different zebra species in their natural habitats, particularly in terms of their social structures, migration patterns, and interactions with other species, and how these behaviors reflect the ecological diversity of the Equidae family."
]


FinalEvalR_hardNN10, FinalEvalC_hardNN10, FinalEvalF_hardNN10, FinalResponseLength_hardNN10 = ResponseEvaluationMetrics(Queries_method=hard_queriesNN10)


FinalR_hardNNN=pd.concat([FinalEvalR_hardNN1, FinalEvalR_hardNN2, FinalEvalR_hardNN3, FinalEvalR_hardNN4, FinalEvalR_hardNN5, FinalEvalR_hardNN6, FinalEvalR_hardNN7,
                         FinalEvalR_hardNN8, FinalEvalR_hardNN9, FinalEvalR_hardNN10], axis=0)
FinalC_hardNNN=pd.concat([FinalEvalC_hardNN1, FinalEvalC_hardNN2, FinalEvalC_hardNN3, FinalEvalC_hardNN4, FinalEvalC_hardNN5, FinalEvalC_hardNN6, FinalEvalC_hardNN7,
                         FinalEvalC_hardNN8, FinalEvalC_hardNN9, FinalEvalC_hardNN10], axis=0)

FinalF_hardNNN=pd.concat([FinalEvalF_hardNN1, FinalEvalF_hardNN2, FinalEvalF_hardNN3, FinalEvalF_hardNN4, FinalEvalF_hardNN5, FinalEvalF_hardNN6, FinalEvalF_hardNN7,
                         FinalEvalF_hardNN8, FinalEvalF_hardNN9, FinalEvalF_hardNN10], axis=0)

FinalResponseLength_hardNNN=pd.concat([FinalResponseLength_hardNN1, FinalResponseLength_hardNN2, FinalResponseLength_hardNN3, FinalResponseLength_hardNN4,
                                      FinalResponseLength_hardNN5, FinalResponseLength_hardNN6, FinalResponseLength_hardNN7, FinalResponseLength_hardNN8,
                                      FinalResponseLength_hardNN9, FinalResponseLength_hardNN10], axis=0)



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_difficulty_index_comparison(df, mean_colors, std_df, custom_legend_labels=None, save_as_pdf=False, pdf_filename='plot.pdf'):
    """
    Plots bar graphs comparing the mean values for different query difficulties and index types,
    with error bars representing the standard deviation.

    Parameters:
    - df: DataFrame containing the mean values.
    - std_df: DataFrame containing the standard deviation values.
    - mean_colors: Dictionary mapping index types to colors.
    - custom_legend_labels: Dictionary mapping index types to custom legend labels (optional).
    - save_as_pdf: Boolean flag to save the plot as a PDF.
    - pdf_filename: Filename to save the PDF if save_as_pdf is True.
    """
    # Set the positions and width for the bars
    x = np.arange(len(df['Query Difficulty'].unique()))
    width = 0.25

    fig, ax = plt.subplots(figsize=(24, 10))

    for i, idx_type in enumerate(df['Index Type'].unique()):
        means = df[df['Index Type'] == idx_type]['Mean']
        std_devs = std_df[std_df['Index Type'] == idx_type]['Std']
        label = custom_legend_labels[idx_type] if custom_legend_labels else idx_type
        ax.bar(x + i*width - width, means, width, yerr=std_devs, label=label, color=mean_colors[idx_type], capsize=5, linewidth=3)

    ax.set_ylabel('Mean Value', fontsize=40)
    ax.set_xticks(x)
    ax.set_ylim(0, 1.25)
    ax.set_xticklabels(df['Query Difficulty'].unique(), fontsize=40)
    ax.tick_params(axis='y', labelsize=40)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, fontsize=40)

    plt.tight_layout()
    if save_as_pdf:
        plt.savefig(pdf_filename)

    plt.show()



In [ ]:
# Standard deviation values for each group
std_easy = Final_N_easyF.std()
std_medium = Final_N_mediumF.std()
std_hard =Final_N_hardF.std()

# Mean values for each group
mean_easy = Final_N_easyF.mean()
mean_medium = Final_N_mediumF.mean()
mean_hard = Final_N_hardF.mean()

# Create a DataFrame for plotting
data = {
    'Query Difficulty': ['Easy query', 'Easy query', 'Easy query', 'Medium query', 'Medium query', 'Medium query', 'Hard query', 'Hard query', 'Hard query'],
    'Index Type': ['Vector Index', 'Knowledge Index', 'Both', 
                   'Vector Index', 'Knowledge Index', 'Both', 
                   'Vector Index', 'Knowledge Index', 'Both'],
    'Mean': np.concatenate([mean_easy.values, mean_medium.values, mean_hard.values])
}

# Create a DataFrame for standard deviation (error bars)
error_data = {
    'Query Difficulty': ['Easy query', 'Easy query', 'Easy query', 'Medium query', 'Medium query', 'Medium query', 'Hard query', 'Hard query', 'Hard query'],
    'Index Type': ['Vector Index', 'Knowledge Index', 'Both', 
                   'Vector Index', 'Knowledge Index', 'Both', 
                   'Vector Index', 'Knowledge Index', 'Both'],
    'Std': np.concatenate([std_easy.values, std_medium.values, std_hard.values])
}

plot_df = pd.DataFrame(data)
error_df = pd.DataFrame(error_data)

# Define colors for each index type
mean_colors = {
    'Vector Index': 'blue',
    'Knowledge Index': 'green',
    'Both': 'red'
}

# Define custom legend labels
custom_legend_labels = {
    'Vector Index': 'Vector Index',
    'Knowledge Index': 'Knowledge Index',
    'Both': 'Vector + Knowledge'
}

# Example usage
plot_difficulty_index_comparison(plot_df, mean_colors, std_df=error_df, custom_legend_labels=custom_legend_labels, save_as_pdf=True, pdf_filename='try.pdf')



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def plot_index_comparison(df, std_df, mean_colors, custom_legend_labels=None, save_as_pdf=False, pdf_filename='plot.pdf'):
    """
    Plots bar graphs comparing the mean values for different index types, with error bars representing the standard deviation.

    Parameters:
    - df: DataFrame containing the mean values.
    - std_df: DataFrame containing the standard deviation values.
    - mean_colors: Dictionary mapping index types to colors.
    - custom_legend_labels: Dictionary mapping index types to custom legend labels (optional).
    - save_as_pdf: Boolean flag to save the plot as a PDF file (default: False).
    - pdf_filename: Filename for the saved PDF file (default: 'plot.pdf').
    """
    fig, ax = plt.subplots(figsize=(24, 10))

    xticks_labels = []
    x_positions = np.arange(len(df['Index Type']))

    for i, idx_type in enumerate(df['Index Type']):
        mean_value = df[df['Index Type'] == idx_type]['Mean'].values[0]
        std_dev = std_df[std_df['Index Type'] == idx_type]['Std'].values[0]
        label = custom_legend_labels[idx_type] if custom_legend_labels else idx_type
        
        # Plot the bar with the error bar
        ax.bar(x_positions[i], mean_value, color=mean_colors[idx_type], yerr=std_dev, capsize=10, label=label, linewidth=3)
        xticks_labels.append(label)
        
    ax.set_xlabel('Index Type', fontsize=40)
    ax.set_ylabel('Mean Value', fontsize=40)
    ax.set_xticks(x_positions)
    ax.set_ylim(0, 1.25)
    ax.set_yticks(np.arange(0, 1.26, 0.25))
    ax.tick_params(axis='y', labelsize=40)
    ax.set_xticklabels(xticks_labels, fontsize=40)  # Increase font size of x ticks
    ax.tick_params(axis='y', labelsize=40)  # Add font size to y ticks

    plt.tight_layout()
    
    if save_as_pdf:
        plt.savefig(pdf_filename)
    
    plt.show()



FFinalR_N=pd.concat([FinalR_easy, FinalR_medium, FinalR_hard], axis=0)
FFinalC_N=pd.concat([FinalC_easy, FinalC_medium, FinalC_hard], axis=0)
FFinalF_N=pd.concat([FinalF_easy, FinalF_medium, FinalF_hard], axis=0)


# Standard deviation values for each group
std_F = FFinalF_N.std()
std_C = FFinalC_N.std()
std_R = FFinalR_N.std()

# Mean values for each group
mean_F = FFinalF_N.mean()
mean_C = FFinalC_N.mean()
mean_R = FFinalR_N.mean()


# FFinalR_N=pd.concat([FFinal_N_easyR, FFinal_N_mediumR, FFinal_N_hardR], axis=0)
# FFinalC_N=pd.concat([FFinal_N_easyC, FFinal_N_mediumC, FFinal_N_hardC], axis=0)
# FFinalF_N=pd.concat([FFinal_N_easyF, FFinal_N_mediumF, FFinal_N_hardF], axis=0)


# # Standard deviation values for each group
# std_F = FFinalF_N.std()
# std_C = FFinalC_N.std()
# std_R = FFinalR_N.std()

# # Mean values for each group
# mean_F = FFinalF_N.mean()
# mean_C = FFinalC_N.mean()
# mean_R = FFinalR_N.mean()

# Create a DataFrame for plotting
data = {
    #'Query Difficulty': ['Easy query', 'Easy query', 'Easy query', 'Medium query', 'Medium query', 'Medium query', 'Hard query', 'Hard query', 'Hard query'],
    'Index Type': ['Vector Index', 'Knowledge Index', 'Both'],
    'Mean': mean_F.values
}

# Create a DataFrame for standard deviation (error bars)
error_data = {
    #'Query Difficulty': ['Easy query', 'Easy query', 'Easy query', 'Medium query', 'Medium query', 'Medium query', 'Hard query', 'Hard query', 'Hard query'],
    'Index Type': ['Vector Index', 'Knowledge Index', 'Both'],
    'Std': std_F.values
}

plot_df = pd.DataFrame(data)
error_df = pd.DataFrame(error_data)

# Define colors for each index type
mean_colors = {
    'Vector Index': 'blue',
    'Knowledge Index': 'green',
    'Both': 'red'
}

# Define custom legend labels
custom_legend_labels = {
    'Vector Index': 'Vector Index',
    'Knowledge Index': 'Knowledge Index',
    'Both': 'Vector + Knowledge'
}

# Example usage
# Example usage with the provided data
plot_index_comparison(plot_df, error_df, mean_colors, custom_legend_labels=custom_legend_labels, save_as_pdf=True, pdf_filename='FmeanF.pdf')


